# CMSE 201 Project 'Bebovision'

**Name:** Brendan Hunt  
**Section:** 2  
**Question:** Can I use my personal match data to build a model that decides what the best decision would be given a match scenario?

**What is it?**  
I'm creating a data analysis tool for 'League of Legends. It will analyze x number of recent matches, graph the average K/D/A, winrate, and many other statistics that are necessary for analyzing match history.  

**Where am I collecting data?**  
I am collecting data using the Riot Developer API, it's a publicly available API that lets me access player info, match info and many other things, thus removing the need for a large external dataset. I have left a lot of code below to display how I am accessing and filtering the data.  

**Possible problems when reviewing assignment**  
I will leave everything in a ran state, but if you try to run it, it may not work as the API key used to access the data resets every day.  
  
**Link to Riot API**  
https://developer.riotgames.com/

**When Running Code**  
Much of this code will take significant amount of time to run due to the rate limits imposed by RiotAPI. I will provide my finalized datasets as CSVs.

In [1]:
#Necessary imports
import requests
import pandas as pd
import json
import time 
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np  
from api_key import x

In [5]:
api_key = x #this is the key used to access the API
name = 'master yi gaming'
region = 'na1'
def summoner_info(name,region): #this function returns necessary info for my account to be used later
    api_url = f'https://{region}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}?api_key={api_key}'
    response = requests.get(api_url)
    if response.status_code == 200:
        player_info = response.json()
        print(player_info)
        player_info_organized = json.dumps(player_info,indent=4)
        print(player_info_organized)
        account_id = player_info['accountId']
        player_id = player_info['id']
        puuid = player_info['puuid']
        #summoner_name = player_info['name']
        icon_id = player_info['profileIconId']
        summoner_level = player_info['summonerLevel']
        data = {
            'account_id': account_id,
            'player_id': player_id,
            'puuid': puuid,
            #'summoner_name': summoner_name,
            'icon_id': icon_id,
            'summoner_level': summoner_level
        }
        return data
    else:
        print(f"Failed to fetch summoner data. Status code: {response.status_code}")
player_data = summoner_info(name,region)
print(player_data)

{'id': 'FbVVb308EBzZpYOV64ffZSXKyQSPZurpON0fom5zr4e3jdRz', 'accountId': 'y3NNbcdpYFk_v5lH-fslzuEPd5s9tetR9kJnS2ElsK3CsU_MUF-tg7yQ', 'puuid': 'd7nZKupaoJ5Bacm6MnBq2NmDcvdzDeDZBDKQW2MLn5zv6DMjU_MfCQj7yKWh68rEr00pUGN7BlBjzw', 'profileIconId': 6068, 'revisionDate': 1713315092706, 'summonerLevel': 280}
{
    "id": "FbVVb308EBzZpYOV64ffZSXKyQSPZurpON0fom5zr4e3jdRz",
    "accountId": "y3NNbcdpYFk_v5lH-fslzuEPd5s9tetR9kJnS2ElsK3CsU_MUF-tg7yQ",
    "puuid": "d7nZKupaoJ5Bacm6MnBq2NmDcvdzDeDZBDKQW2MLn5zv6DMjU_MfCQj7yKWh68rEr00pUGN7BlBjzw",
    "profileIconId": 6068,
    "revisionDate": 1713315092706,
    "summonerLevel": 280
}
{'account_id': 'y3NNbcdpYFk_v5lH-fslzuEPd5s9tetR9kJnS2ElsK3CsU_MUF-tg7yQ', 'player_id': 'FbVVb308EBzZpYOV64ffZSXKyQSPZurpON0fom5zr4e3jdRz', 'puuid': 'd7nZKupaoJ5Bacm6MnBq2NmDcvdzDeDZBDKQW2MLn5zv6DMjU_MfCQj7yKWh68rEr00pUGN7BlBjzw', 'icon_id': 6068, 'summoner_level': 280}


In [ ]:
def get_puuid(summoner_name,region,api_key): #pulls specifically the puuid, necessary to find player matches
    api_url = f'https://{region}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}?api_key={api_key}'
    print(api_url)
    response = requests.get(api_url)
    player_info = response.json()
    puuid = player_info['puuid']
    return puuid

In [ ]:
puuid = player_data['puuid']
print(puuid)

In [ ]:
def get_match_ids(puuid, mass_region, no_games, queue_id, api_key):
    api_url = (
        "https://" +
        mass_region +
        ".api.riotgames.com/lol/match/v5/matches/by-puuid/" +
        puuid + 
        "/ids?start=0" + 
        "&count=" +
        str(no_games) + 
        "&queue=" + 
        str(queue_id) + 
        "&api_key=" + 
        api_key
    )
    resp = requests.get(api_url)
    match_ids = resp.json()
    print(match_ids)
    return match_ids

In [ ]:
queue_id = 400
mass_region = 'americas'
no_games = '50'
print(get_match_ids(puuid,mass_region,no_games,queue_id,api_key))

In [ ]:
def get_match_data(match_id,mass_region,api_key): #overall match data for every player
    api_url = f'https://{mass_region}.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}'
    while True:
        resp = requests.get(api_url)
        
        
        if resp.status_code == 429:
            print("Rate Limit hit, sleeping for 10 seconds")
            time.sleep(10)
           
            continue
            
        match_data = resp.json()
        return match_data     


In [ ]:
def find_player_data(match_data,puuid): #pulls my data from a given match index (match_id)
    print("Match data:", match_data)
    participants = match_data['metadata']['participants']
    player_index = participants.index(puuid)
    player_data = match_data['info']['participants'][player_index]
    return player_data

In [ ]:
def master_function(summoner_name,region,mass_region,no_games,queue_id,api_key):
    puuid = get_puuid(summoner_name, region, api_key)
    match_ids = get_match_ids(puuid, mass_region, no_games, queue_id, api_key)
    df = gather_all_data(puuid, match_ids, mass_region, api_key)
    return df

In [ ]:
summoner_name = 'master yi gaming'
region = 'na1'
mass_region = 'americas'
no_games = 5
queue_id = 400

df = master_function(summoner_name, region, mass_region, no_games, queue_id, api_key)
match_ids = get_match_ids(puuid, mass_region, no_games, queue_id, api_key)
match_id = match_ids[0]
match_data = get_match_data(match_id, mass_region, api_key)
player_data = find_player_data(match_data,puuid)
formatted_data = json.dumps(match_data, indent=4)
print(match_data)
metadata = match_data['metadata']
participants = metadata['participants']


In [ ]:
all_participants = []

# Loop through your match IDs
for match_id in match_ids:
    match_data = get_match_data(match_id, mass_region, api_key)

    if 'metadata' in match_data and 'participants' in match_data['metadata']:
        participants = match_data['metadata']['participants']
        all_participants.extend(participants)
puuids = all_participants  
all_participants_larger_set = []
for puuid in puuids:
    match_ids_larger_set = get_match_ids(puuid, mass_region, no_games, queue_id, api_key)
    for match_id in match_ids_larger_set:
        match_data_larger_set = get_match_data(match_id, mass_region, api_key)

        if 'metadata' in match_data_larger_set and 'participants' in match_data_larger_set['metadata']:
            participants_larger_set = match_data_larger_set['metadata']['participants']
            all_participants_larger_set.extend(participants_larger_set)
print(len(all_participants_larger_set))
#This bit of code here was used to loop through my match history to garner a larger set of players and pull more matches



In [ ]:
print(all_participants_larger_set)
print(type(all_participants_larger_set))
partlistcondensed = list(set(all_participants_larger_set))
uniqueparts = []
for item in partlistcondensed:
    if item not in uniqueparts:
        uniqueparts.append(item)

print(uniqueparts)
print(len(uniqueparts))

In [ ]:
#df = pd.DataFrame({'PUUID': all_participants_larger_set})
#csv_file_path = 'puuids.csv'
#df.to_csv(csv_file_path, index=False)
print(match_ids_larger_set)

In [ ]:
print(all_participants_larger_set)
df = pd.DataFrame({'PUUID': all_participants_larger_set})

# Specify the file path for the CSV file
csv_file_path = 'puuids.csv'

# Save the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)

print(f'CSV file "{csv_file_path}" created successfully.')

In [ ]:
def get_recent_matchlist(puuid, region, api_key, num_matches=5):
    url = f"https://{mass_region}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count={num_matches}&api_key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        print(response.json())
        return response.json()
    elif response.status_code == 429:
        # Rate limit exceeded, wait for a few seconds and retry
        retry_after = int(response.headers['Retry-After'])
        print(f"Rate limit exceeded. Waiting for {retry_after} seconds...")
        time.sleep(retry_after)
        return get_recent_matchlist(puuid, region, api_key, num_matches)
    else:
        print(f"Failed to retrieve matchlist for PUUID {puuid}. Status code: {response.status_code}")
        return []

puuids = uniqueparts

region = "na1"  # Change to the appropriate region

# Initialize a list to store match IDs
all_match_ids = []

# Loop through PUUIDs and get recent match IDs
for puuid in puuids:
    recent_match_ids = get_recent_matchlist(puuid, region, api_key, num_matches=4)
    all_match_ids.extend(recent_match_ids)

# Print the list of match IDs
print("Match IDs from recent 2 matches of all players:")
for match_id in all_match_ids:
    print(match_id)

print(type(all_match_ids))

In [ ]:
print(type(all_match_ids))
print(len(all_match_ids))
presortedmatchids = list(set(all_match_ids))
unique_match_ids = []
for item in presortedmatchids:
    if item not in unique_match_ids:
        unique_match_ids.append(item)

print(unique_match_ids)
print(len(unique_match_ids))

print(unique_match_ids)
df = pd.DataFrame({'Match IDs': unique_match_ids})

# Specify the file path for the CSV file
csv_file_path = 'matchids.csv'

# Save the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)





In [ ]:
for match_id in unique_match_ids:
    
    match_data = get_match_data(match_id, mass_region, api_key) 

    print(f"Match ID: {match_id}")
    print(f"Game Duration: {match_data['info']['gameDuration']} seconds")
    print("Participants:")
    for participant in match_data['info']['participants']:
        print(f"Champion: {participant['championName']}, KDA: {participant['kills']}/{participant['deaths']}/{participant['assists']}")

In [ ]:
df = pd.read_csv('matchids.csv')
match_id = df.iloc[1, 0]
data = get_match_data(match_id,mass_region,api_key)
if 'metadata' in data:
    for key in data['metadata'].keys():
        print(key)


if 'info' in data:
    for key in data['info'].keys():
        print(key)

print(data)

In [ ]:
match_ids = pd.read_csv('matchids.csv',skiprows=1)
print(match_ids)
#gather_all_data(puuid,match_ids,mass_region,api_key)

In [ ]:
def find_player_data(match_data,puuid): #pulls my data from a given match index (match_id)
    print("Match data:", match_data)
    participants = match_data['metadata']['participants']
    player_index = participants.index(puuid)
    player_data = match_data['info']['participants'][player_index]
    return player_data

In [ ]:
import os
def gather_all_data(match_ids, mass_region, api_key,csv_file_path):
    columns = [
        'match_id', 'puuid', 'champion', 'kills', 'deaths', 'assists', 'win', 
        'item0', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6', 
        'vision_score', 'gameLength', 'goldPerMinute', 'totalCS', 'csPerMinute', 
        
    ]
    data = {key: [] for key in columns}
    
    for match_id in match_ids:
        match_data = get_match_data(match_id, mass_region, api_key)
        if 'status' in match_data and match_data['status'].get('status_code') == 404:
            print(f"Data not found for match ID: {match_id}")
            continue  
        game_duration = match_data.get('info', {}).get('gameDuration', 0)
        if 'metadata' in match_data and 'participants' in match_data['metadata']:
            for puuid in match_data['metadata']['participants']:
                player_data = find_player_data(match_data, puuid)

                
                data['match_id'].append(match_id)
                data['puuid'].append(puuid)
                data['champion'].append(player_data.get('championName', 'Unknown'))
                data['kills'].append(player_data.get('kills', 0))
                data['deaths'].append(player_data.get('deaths', 0))
                data['assists'].append(player_data.get('assists', 0))
                data['win'].append(player_data.get('win', False))
                data['gameLength'].append(game_duration)  #
                gold_earned = player_data.get('goldEarned', 0)
                gold_per_minute = (gold_earned / game_duration) * 60 if game_duration > 0 else 0
                data['goldPerMinute'].append(gold_per_minute)
                total_cs = player_data.get('totalMinionsKilled', 0) + player_data.get('neutralMinionsKilled', 0)
                cs_per_minute = total_cs / (game_duration/60) if game_duration > 0 else 0
                data['totalCS'].append(total_cs)
                data['csPerMinute'].append(cs_per_minute)
                for i in range(7):
                    data[f'item{i}'].append(player_data.get(f'item{i}', 'None'))
                data['vision_score'].append(player_data.get('visionScore', 0))
        df = pd.DataFrame(data, columns=columns)

       
        if not os.path.isfile(csv_file_path):
            
            df.to_csv(csv_file_path, index=False, mode='w')
        else:
           
            df.to_csv(csv_file_path, index=False, mode='a', header=False)

        
        data = {key: [] for key in columns}        
        
        print({key: len(value) for key, value in data.items()})

    df = pd.DataFrame(data)
    return df


In [ ]:
csv_file_path = 'results.csv'
df = pd.read_csv('matchids.csv')
match_ids = df['Match IDs'].tolist()[:3300]
print(match_ids)
result_df = gather_all_data(match_ids,mass_region,api_key,csv_file_path)
result_df = result_df.drop(columns=['match_id','puuid'])



In [ ]:
with open('item.json') as file:
    item_data = json.load(file)

# Function to get item name by ID
def get_item_name(item_id):
    return item_data['data'][str(item_id)]['name'] if str(item_id) in item_data['data'] else 'Unknown'

# Example usage
item_name = get_item_name(1011)  # Replace 1001 with the item ID
print(item_name)

In [ ]:

# Load the item data from the JSON file
with open('item.json') as file:
    item_data = json.load(file)

# Function to get item name by ID
def get_item_name(item_id):
    return item_data['data'][str(item_id)]['name'] if str(item_id) in item_data['data'] else 'Unknown'

most_common_items_df = pd.read_csv('most_common_items_by_champion_sorted.csv')

# Converting item IDs into item names for the top 20 champions
for index, row in most_common_items_df.iloc[:20].iterrows():
    item_ids = row['Most Common Items'].split(', ')
    item_names = [get_item_name(int(item_id)) for item_id in item_ids]
    most_common_items_df.at[index, 'Most Common Items'] = ', '.join(item_names)

# Displaying the most common items for the top 20 champions
pd.set_option('display.max_colwidth', None)

top_10_champions = most_common_items_df.head(10)
next_10_champions = most_common_items_df.iloc[10:20]

# Displaying the two halves side by side
display_side_by_side = pd.concat([top_10_champions.reset_index(drop=True), 
                                  next_10_champions.reset_index(drop=True)], 
                                 axis=1)

display_side_by_side.columns = ['Champion', 'Most Common Items', 'Champion', 'Most Common Items']
display_side_by_side


In [ ]:
df = pd.read_csv('results.csv')
results = df.drop(columns=['match_id','puuid'])
print(results)

In [ ]:
item_columns = ['item0', 'item1', 'item2', 'item3','item4','item5','item6']
melted_df = results.melt(id_vars=['win'], value_vars=item_columns, var_name='item_slot', value_name='item_id')

# Filter out rows where item_id is 0, if needed
melted_df = melted_df[melted_df['item_id'] != 0]

# Count wins and total games for each item
item_stats = melted_df.groupby('item_id').agg(total_games=('item_id', 'count'), wins=('win', 'sum'))

# Calculate win rate
item_stats['win_rate'] = item_stats['wins'] / item_stats['total_games']

print(item_stats)

item_stats.to_csv('item_stats.csv', index=True)

In [ ]:
winrates = results.groupby('champion')['win'].mean().reset_index()
winrates.rename(columns={'win': 'winrate'}, inplace=True)
print(winrates)
average_cs_per_minute = results.groupby('champion')['csPerMinute'].mean().reset_index()
print(average_cs_per_minute)
results['deaths'] = results['deaths'].replace(0,1)
results['ka_d_ratio'] = results['kills'] / results['deaths']
average_ka_d_per_champion = results.groupby('champion')['ka_d_ratio'].mean().reset_index()
print(average_ka_d_per_champion)
pickrates = results['champion'].value_counts() / 3300  
pickrates = pickrates.reset_index()
pickrates.columns = ['champion', 'pickrate']
print(pickrates)

In [ ]:
combined_df = winrates.merge(average_cs_per_minute, on='champion') 
combined_df = combined_df.merge(average_ka_d_per_champion, on='champion')
combined_df = combined_df.merge(pickrates, on='champion')
combined_df.to_csv('overalldata.csv', index=False)


In [ ]:
combined_df_sorted = combined_df.sort_values(by='pickrate', ascending=False)
plt.figure(figsize=(20, 8))
sns.barplot(x='champion', y='pickrate', data=combined_df_sorted, palette='viridis')
plt.ylabel('Pickrate')
plt.xlabel('Champion')
plt.title('Top Champions by Pickrate')
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig('pickrate.png')
plt.show()
top_20_pickrate = combined_df_sorted.head(20)
#find top 20 highest picked champs
for index, row in top_20_pickrate.iterrows():
    champion = row['champion']
    pick_rate = round(row['pickrate'], 2)
    print(f"Champion: {champion}, PickRate: {pick_rate}")


In [ ]:
combined_df_sorted = combined_df.sort_values(by='ka_d_ratio', ascending=False)
plt.figure(figsize=(20, 8))
sns.barplot(x='champion', y='ka_d_ratio', data=combined_df_sorted, palette='viridis')
plt.ylabel('KDA')
plt.xlabel('Champion')
plt.title('Average Champion KDA')
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig('avgchampkda.png')
plt.show()


top_20_pickrate = combined_df_sorted.head(20)
for index, row in top_20_pickrate.iterrows():
    champion = row['champion']
    ka_d = round(row['ka_d_ratio'], 2)
    print(f"Champion: {champion}, KDA: {ka_d}")




In [ ]:
combined_df_sorted = combined_df.sort_values(by='csPerMinute', ascending=False)
plt.figure(figsize=(20, 8))
sns.barplot(x='champion', y='csPerMinute', data=combined_df_sorted, palette='viridis',width=1)
plt.ylabel('csPerMinute')
plt.xlabel('Champion')
plt.title('Average Champion Creep Score (CS) Per Minute')
plt.xticks(rotation=90,fontsize=8)
plt.savefig('avgcschamp')
plt.show()


top_20_pickrate = combined_df_sorted.head(20)
for index, row in top_20_pickrate.iterrows():
    champion = row['champion']
    cs_PerMinute = round(row['csPerMinute'], 2)
    print(f"Champion: {champion}, CS Per Minute: {cs_PerMinute}")




In [ ]:
data['totalItems'] = (data[['item0', 'item1', 'item2', 'item3', 'item4', 'item5']] != 0).sum(axis=1)
item_winrate = data.groupby('totalItems')['win'].mean()
plt.figure(figsize=(10, 6))
sns.barplot(x=item_winrate.index, y=item_winrate.values,palette = 'Set1')
plt.title('Average Win Rate by Total Number of Items Acquired')
plt.xlabel('Total Items Acquired')
plt.ylabel('Average Win Rate')
plt.savefig('totalitemswr.png')
plt.show()



In [ ]:
cs_bins_fine = pd.cut(data['csPerMinute'], bins=18)  
cs_winrate_fine = data.groupby(cs_bins_fine)['win'].mean()
plt.figure(figsize=(14, 7))
sns.barplot(x=cs_winrate_fine.index, y=cs_winrate_fine.values,palette='Set1')
plt.xticks(rotation=45)
plt.title('Average Win Rate by CS Per Minute')
plt.xlabel('CS Per Minute (Finer Bins)')
plt.ylabel('Average Win Rate')
plt.savefig('wrbycspm.png')
plt.show()


In [ ]:
vision_bins = pd.cut(data['vision_score'], bins=10)
vision_winrate = data.groupby(vision_bins)['win'].mean()
plt.figure(figsize=(10, 10))
sns.barplot(x=vision_winrate.index, y=vision_winrate.values,palette='Set2')
plt.xticks(rotation=45)
plt.title('Average Win Rate by Vision Score')
plt.xlabel('Vision Score')
plt.ylabel('Average Win Rate')
plt.savefig('visionscorewr.png')
plt.show()



In [ ]:
file_path_results = 'results.csv'
file_path_overalldata = 'overalldata.csv'
results_data = pd.read_csv(file_path_results)
overall_data = pd.read_csv(file_path_overalldata)

results_data['win'] = results_data['win'].astype(int)

merged_data = pd.merge(results_data, overall_data[['champion', 'winrate']], on='champion', how='left')

grouped_data = merged_data.groupby(['champion', 'gameLength'])['win'].mean().reset_index()

bin_size = 300 
grouped_data['gameLengthBin'] = (grouped_data['gameLength'] // bin_size) * bin_size
grouped_data_binned = grouped_data.groupby(['champion', 'gameLengthBin'])['win'].mean().reset_index()

all_champions = grouped_data['champion'].unique()
champion_groups_40 = [all_champions[i:i + 40] for i in range(0, len(all_champions), 40)]

def plot_champion_group(champion_group, data, title_suffix,save_filename):
    plt.figure(figsize=(15, 8))
    for champion in champion_group:
        champion_data = data[data['champion'] == champion]
        sns.lineplot(data=champion_data, x='gameLengthBin', y='win', label=champion)
    plt.title(f'Win Rate Over Binned Game Length for Champions: Group {title_suffix}')
    plt.xlabel('Game Length (Seconds)')
    plt.ylabel('Win Rate')
    plt.legend(title='Champion')
    plt.savefig(save_filename)
    plt.show()

for i, group in enumerate(champion_groups_40, 1):
    save_filename = f'{i}win_rate.png'
    plot_champion_group(group, grouped_data_binned, i,save_filename)

In [ ]:

all_champions_data = grouped_data_binned.copy()

pivot_table = all_champions_data.pivot(index='gameLengthBin', columns='champion', values='win')

pivot_table


In [ ]:
# Redefining the function to exclude item '0' (unused item slot) and find the most common items for each champion

def most_common_items_by_champion_excluding_zero(data, item_cols):
    common_items = {}
    for champion in data['champion'].unique():
        champion_data = data[data['champion'] == champion]
        item_counts = champion_data[item_cols].apply(pd.Series.value_counts).sum(axis=1).sort_values(ascending=False)
        item_counts = item_counts.drop(0, errors='ignore')  # Dropping item '0'
        common_items[champion] = item_counts.index[0:3]  # Top 3 most common items
    return common_items

most_common_items_excluding_zero = most_common_items_by_champion_excluding_zero(results_data, item_columns)

formatted_output = {champion: list(items) for champion, items in most_common_items_excluding_zero.items()}

readable_output = {champion: ', '.join(map(str, items)) for champion, items in formatted_output.items()}

df = pd.DataFrame.from_dict(readable_output, orient='index', columns=['Most Common Items'])

df.sort_index(inplace=True)

df.to_csv('most_common_items_by_champion_sorted.csv')


In [ ]:
#graph kda to winrate
kad_bins = pd.cut(results['ka_d_ratio'], bins=20)

kad_winrate = results.groupby(kad_bins)['win'].mean()

plt.figure(figsize=(12, 8))
sns.lineplot(x=kad_winrate.index.astype(str), y=kad_winrate.values)
plt.xticks(rotation=45)
plt.title('Average Win Rate by KA/D Ratio Bins')
plt.xlabel('KA/D Ratio Bins')
plt.ylabel('Average Win Rate')
plt.savefig('KDAwin.png')
plt.show()




In [ ]:
data = pd.read_csv('results.csv')
data.head()


In [ ]:
data = pd.read_csv('results.csv')
data = data.drop(columns=['puuid', 'match_id','champion'])
data['win'] = data['win'].astype(int)
correlation_matrix_with_win = data.corr()
plt.figure(figsize=(14, 10))
sns.heatmap(correlation_matrix_with_win, annot=True, cmap='coolwarm')
plt.title("Correlation Matrix of Game Data Including Win Rate")
plt.savefig('corrmatrix.png')
plt.show()

correlation_matrix_with_win



In [ ]:
sns.set(style="whitegrid")
data_numeric = data.select_dtypes(include=[float, int])
plt.figure(figsize=(10, 6))
sns.boxplot(x='win', y='goldPerMinute', data=data_numeric,palette = 'Set1')
plt.title('Box Plot of Gold Per Minute by Win/Loss')
plt.xlabel('Loss(0) , Win(1)')
plt.ylabel('Gold Per Minute')

plt.savefig('gpmboxplot.png')
plt.show()



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler

results_data['deaths'].replace(0, 1, inplace=True)
results_data['ka_d_ratio'] = (results_data['kills'] + results_data['assists']) / results_data['deaths']
results_data['ka_d_ratio'].replace([np.inf, -np.inf], np.nan, inplace=True)
results_data.dropna(inplace=True)
print("Data description:\n", results_data[features].describe())
X = results_data[features]
y = results_data['win']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_scaled, y_train)
y_pred = rf_classifier.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", report)

#this here can be ignored i know machine learning is a cmse202 topic i just wanted to challenge myself

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# Define a function representing the rate of change of kills for a team
def kill_rate(t, kills):
    # You can define the rate of change based on in-game factors
    # For simplicity, let's assume a constant rate for this example
    rate = 0.5  # Kills per minute
    return rate

# Define the time span of the game
t_span = (0, 30)  # Start at 0 minutes and end at 30 minutes

# Initial number of kills for the team
initial_kills = 0

# Solve the differential equation
sol = solve_ivp(kill_rate, t_span, [initial_kills], t_eval=np.linspace(0, 30, 100))

# Extract the time and kills values from the solution
t_values = sol.t
kills_values = sol.y[0]

# Plot the results
plt.figure(figsize=(8, 6))
plt.plot(t_values, kills_values, label='Kills for the Team', color='blue')
plt.xlabel('Time (minutes)')
plt.ylabel('Kills')
plt.title('Modeling Kills Over Time')
plt.legend()
plt.grid(True)
plt.savefig('killstime.png')
plt.show()



In [ ]:
def metric_change(M, t, b):
   
    dMdt = b * t
    return dMdt


b = 0.1  
M0 = 10  
M = odeint(metric_change, M0, t, args=(b,))
plt.figure(figsize=(10, 6))
plt.plot(t, M, label='Game Metric over Time (e.g., goldPerMinute, totalCS)')
plt.xlabel('Time (minutes)')
plt.ylabel('Game Metric')
plt.title('Impact of Game Length on Game Metrics')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
def performance_evolution(K, t, c):
   
    dKdt = c * t
    return dKdt

# Constants and Initial Conditions
c = 0.02  # Assuming a gradual improvement rate
K0 = 1    # Initial performance metric (e.g., KDA ratio, CS per minute) at the start

# Time points (representing different stages of a player's career or number of games played)
t = np.linspace(0, 200, 500)

K = odeint(performance_evolution, K0, t, args=(c,))

plt.figure(figsize=(10, 6))
plt.plot(t, K, label='Player Performance Metric over Time')
plt.xlabel('Time (Number of Games Played)')
plt.ylabel('Performance Metric (e.g., KDA, CS/min)')
plt.title('Evolution of Player Performance Over Time')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import statsmodels.api as sm

file_paths = {
    'item_stats': 'item_stats.csv',
    'most_common_items_by_champion': 'most_common_items_by_champion_sorted.csv',
    'overall_data': 'overalldata.csv',
    'results': 'results.csv'
}

dataframes = {name: pd.read_csv(path) for name, path in file_paths.items()}

agg_funcs_results = {
    'kills': 'mean',
    'deaths': 'mean',
    'assists': 'mean',
    'win': 'mean',  
    'vision_score': 'mean',
    'gameLength': 'mean',
    'goldPerMinute': 'mean',
    'totalCS': 'mean',
    'csPerMinute': 'mean'
}
results_agg = dataframes['results'].groupby('champion').agg(agg_funcs_results).reset_index()

merged_data = pd.merge(dataframes['overall_data'], results_agg, on='champion', how='left')

Y = merged_data['winrate'] 

X = merged_data[['csPerMinute_x', 'ka_d_ratio', 'pickrate', 'kills', 'deaths', 'assists', 
                 'vision_score', 'gameLength', 'goldPerMinute', 'totalCS', 'csPerMinute_y']]

X = sm.add_constant(X)

model = sm.OLS(Y, X).fit()

model_summary = model.summary()
model_summary


Some code was used from this github page to help me with learning how to use an API. 
https://github.com/JackWillz/The-Riot-API-Introduction-by-iTero-Gaming/blob/main/The%20Riot%20API%20Introductory%20Guide%20by%20iTero%20Gaming.ipynb

Here is an example json from one of the 3400 matches that I collected from. 

Match data: {'metadata': {'dataVersion': '2', 'matchId': 'NA1_4835192603', 'participants': ['yBm2l04adqR8oeG8WCcNZOflvVp9Q4ZcGSm-wXoGZhSUNstQih8jHK-uFw1hKK14VxzVxNtQnx11FA', 'PY7iMB_4ohr2vmtOB-GaUAFMb6owPm9ek4DJUBYt4IoEhxIvz3RrwACfnRDNv8uicE4HxaW-TiAtzg', 'vr_-Loy-Fc_iHTYyWWa383FyLCw6c0iKLnVREVa6Vtab4bAqMcE9XLbFaxnsgqwT5GTjELIuhm7Eyw', '34wqa3OKa6UWHSPIvs3i0BKlIYmR-dmz3C7uLs9o58K6O-ZnRjoBRJ4km59gerGJ546Fcr7S0ysaxw', 'IwYjkxE9AlLTMZ_9Mf6YVpyGcQYOeQq2azNWgQhU2BXEm0vZn6Bvrxj7jNIEr-iVQAAxAqQFa4jDHg', 'bHhAY7DSUICKSCwWzf2EeKhtFsUq4fEOXZM4BsXK4CIXbAW0z-N2I-2yls25q9G__fugkZVhF7h_Pw', 'th3FWRsIG8MtnJZg-BaRLYqcMc5fggO4dl7z6lBCtR8fGtSmncvKqX2-G-Hm81htOfAsgqs5V393Dw', 'KyRnbjcRmlrkQ7fShSbsBPJUhUkVTZ2GNZrM8wJSeqm0ld97UrrEKnE1UIT-pDsOn0RDqM83ukk_hg', 'yFM3imQ-mYn4XTiMFpOx8CQouAiGQTec1u24Ou7I-6kX8_oMBJTfPUG1MPbm_gI16oAeq-pWNnLgZQ', '1JzHac7vWznlDGVAlQMtHfJMJr39gI0_akJzwWBC9o2AJIp4GGbAbhhqMJYV-zgfCTE7pty72aAEwg']}, 'info': {'gameCreation': 1700458100222, 'gameDuration': 2158, 'gameEndTimestamp': 1700460278905, 'gameId': 4835192603, 'gameMode': 'CLASSIC', 'gameName': 'teambuilder-match-4835192603', 'gameStartTimestamp': 1700458120677, 'gameType': 'MATCHED_GAME', 'gameVersion': '13.22.541.9804', 'mapId': 11, 'participants': [{'allInPings': 0, 'assistMePings': 3, 'assists': 6, 'baitPings': 0, 'baronKills': 1, 'basicPings': 0, 'bountyLevel': 0, 'challenges': {'12AssistStreakCount': 0, 'abilityUses': 289, 'acesBefore15Minutes': 0, 'alliedJungleMonsterKills': 99, 'baronTakedowns': 1, 'blastConeOppositeOpponentCount': 0, 'bountyGold': 975, 'buffsStolen': 0, 'completeSupportQuestInTime': 0, 'controlWardTimeCoverageInRiverOrEnemyHalf': 0.797709301141369, 'controlWardsPlaced': 6, 'damagePerMinute': 1329.491672858909, 'damageTakenOnTeamPercentage': 0.28612535073262485, 'dancedWithRiftHerald': 0, 'deathsByEnemyChamps': 10, 'dodgeSkillShotsSmallWindow': 20, 'doubleAces': 0, 'dragonTakedowns': 1, 'earliestBaron': 1657.7682029, 'earliestDragonTakedown': 469.07374519999996, 'earlyLaningPhaseGoldExpAdvantage': 0, 'effectiveHealAndShielding': 0, 'elderDragonKillsWithOpposingSoul': 0, 'elderDragonMultikills': 0, 'enemyChampionImmobilizations': 21, 'enemyJungleMonsterKills': 0, 'epicMonsterKillsNearEnemyJungler': 0, 'epicMonsterKillsWithin30SecondsOfSpawn': 0, 'epicMonsterSteals': 0, 'epicMonsterStolenWithoutSmite': 0, 'firstTurretKilled': 0, 'flawlessAces': 0, 'fullTeamTakedown': 0, 'gameLength': 2158.4759347, 'goldPerMinute': 494.45858346960796, 'hadOpenNexus': 0, 'highestChampionDamage': 1, 'immobilizeAndKillWithAlly': 6, 'initialBuffCount': 2, 'initialCrabCount': 2, 'jungleCsBefore10Minutes': 74.00000008940697, 'junglerKillsEarlyJungle': 0, 'junglerTakedownsNearDamagedEpicMonster': 0, 'kTurretsDestroyedBeforePlatesFall': 0, 'kda': 2.2, 'killAfterHiddenWithAlly': 2, 'killParticipation': 0.7333333333333333, 'killedChampTookFullTeamDamageSurvived': 0, 'killingSprees': 3, 'killsNearEnemyTurret': 5, 'killsOnLanersEarlyJungleAsJungler': 0, 'killsOnRecentlyHealedByAramPack': 0, 'killsUnderOwnTurret': 1, 'killsWithHelpFromEpicMonster': 1, 'knockEnemyIntoTeamAndKill': 0, 'landSkillShotsEarlyGame': 1, 'laneMinionsFirst10Minutes': 0, 'laningPhaseGoldExpAdvantage': 0, 'legendaryCount': 0, 'lostAnInhibitor': 0, 'maxCsAdvantageOnLaneOpponent': 58.750000059604645, 'maxKillDeficit': 0, 'maxLevelLeadLaneOpponent': 2, 'mejaisFullStackInTime': 0, 'moreEnemyJungleThanOpponent': -53.00000008940697, 'multiKillOneSpell': 0, 'multiTurretRiftHeraldCount': 0, 'multikills': 3, 'multikillsAfterAggressiveFlash': 1, 'mythicItemUsed': 6631, 'outerTurretExecutesBefore10Minutes': 0, 'outnumberedKills': 6, 'outnumberedNexusKill': 0, 'perfectDragonSoulsTaken': 0, 'perfectGame': 0, 'pickKillWithAlly': 14, 'poroExplosions': 0, 'quickCleanse': 0, 'quickFirstTurret': 0, 'quickSoloKills': 0, 'riftHeraldTakedowns': 0, 'saveAllyFromDeath': 0, 'scuttleCrabKills': 3, 'skillshotsDodged': 130, 'skillshotsHit': 50, 'snowballsHit': 0, 'soloBaronKills': 0, 'soloKills': 6, 'stealthWardsPlaced': 2, 'survivedSingleDigitHpCount': 0, 'survivedThreeImmobilizesInFight': 8, 'takedownOnFirstTurret': 0, 'takedowns': 22, 'takedownsAfterGainingLevelAdvantage': 0, 'takedownsBeforeJungleMinionSpawn': 0, 'takedownsFirstXMinutes': 5, 'takedownsInAlcove': 0, 'takedownsInEnemyFountain': 0, 'teamBaronKills': 1, 'teamDamagePercentage': 0.3034331241406508, 'teamElderDragonKills': 0, 'teamRiftHeraldKills': 0, 'tookLargeDamageSurvived': 0, 'turretPlatesTaken': 0, 'turretTakedowns': 0, 'turretsTakenWithRiftHerald': 0, 'twentyMinionsIn3SecondsCount': 0, 'twoWardsOneSweeperCount': 0, 'unseenRecalls': 0, 'visionScoreAdvantageLaneOpponent': -0.11555755138397217, 'visionScorePerMinute': 0.9264663167538986, 'wardTakedowns': 3, 'wardTakedownsBefore20M': 1, 'wardsGuarded': 1}, 'champExperience': 19539, 'champLevel': 18, 'championId': 56, 'championName': 'Nocturne', 'championTransform': 0, 'commandPings': 20, 'consumablesPurchased': 7, 'damageDealtToBuildings': 0, 'damageDealtToObjectives': 27275, 'damageDealtToTurrets': 0, 'damageSelfMitigated': 72737, 'dangerPings': 0, 'deaths': 10, 'detectorWardsPlaced': 6, 'doubleKills': 3, 'dragonKills': 1, 'eligibleForProgression': True, 'enemyMissingPings': 12, 'enemyVisionPings': 3, 'firstBloodAssist': False, 'firstBloodKill': False, 'firstTowerAssist': False, 'firstTowerKill': False, 'gameEndedInEarlySurrender': False, 'gameEndedInSurrender': True, 'getBackPings': 3, 'goldEarned': 17787, 'goldSpent': 17650, 'holdPings': 0, 'individualPosition': 'JUNGLE', 'inhibitorKills': 0, 'inhibitorTakedowns': 0, 'inhibitorsLost': 2, 'item0': 6631, 'item1': 3111, 'item2': 3156, 'item3': 6696, 'item4': 6333, 'item5': 6694, 'item6': 3364, 'itemsPurchased': 28, 'killingSprees': 6, 'kills': 16, 'lane': 'JUNGLE', 'largestCriticalStrike': 0, 'largestKillingSpree': 3, 'largestMultiKill': 2, 'longestTimeSpentLiving': 790, 'magicDamageDealt': 7835, 'magicDamageDealtToChampions': 2969, 'magicDamageTaken': 14131, 'missions': {'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0}, 'needVisionPings': 0, 'neutralMinionsKilled': 171, 'nexusKills': 0, 'nexusLost': 1, 'nexusTakedowns': 0, 'objectivesStolen': 0, 'objectivesStolenAssists': 0, 'onMyWayPings': 13, 'participantId': 1, 'pentaKills': 0, 'perks': {'statPerks': {'defense': 5003, 'flex': 5008, 'offense': 5005}, 'styles': [{'description': 'primaryStyle', 'selections': [{'perk': 8008, 'var1': 116, 'var2': 33, 'var3': 0}, {'perk': 9111, 'var1': 2039, 'var2': 440, 'var3': 0}, {'perk': 9104, 'var1': 13, 'var2': 20, 'var3': 0}, {'perk': 8014, 'var1': 1227, 'var2': 0, 'var3': 0}], 'style': 8000}, {'description': 'subStyle', 'selections': [{'perk': 8138, 'var1': 18, 'var2': 0, 'var3': 0}, {'perk': 8106, 'var1': 5, 'var2': 0, 'var3': 0}], 'style': 8100}]}, 'physicalDamageDealt': 266332, 'physicalDamageDealtToChampions': 43491, 'physicalDamageTaken': 30914, 'placement': 0, 'playerAugment1': 0, 'playerAugment2': 0, 'playerAugment3': 0, 'playerAugment4': 0, 'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0, 'playerSubteamId': 0, 'profileIcon': 5675, 'pushPings': 0, 'puuid': 'yBm2l04adqR8oeG8WCcNZOflvVp9Q4ZcGSm-wXoGZhSUNstQih8jHK-uFw1hKK14VxzVxNtQnx11FA', 'quadraKills': 0, 'riotIdName': '', 'riotIdTagline': '', 'role': 'NONE', 'sightWardsBoughtInGame': 0, 'spell1Casts': 141, 'spell2Casts': 45, 'spell3Casts': 68, 'spell4Casts': 35, 'subteamPlacement': 0, 'summoner1Casts': 4, 'summoner1Id': 4, 'summoner2Casts': 20, 'summoner2Id': 11, 'summonerId': '3acL0Rdj_8MNNSo0lwD4JQL15Ik_KjMI4dJrSdbu0kLxIZQ', 'summonerLevel': 374, 'summonerName': 'Yessss Sirrr ', 'teamEarlySurrendered': False, 'teamId': 100, 'teamPosition': 'JUNGLE', 'timeCCingOthers': 311, 'timePlayed': 2158, 'totalAllyJungleMinionsKilled': 141, 'totalDamageDealt': 318237, 'totalDamageDealtToChampions': 47827, 'totalDamageShieldedOnTeammates': 0, 'totalDamageTaken': 52251, 'totalEnemyJungleMinionsKilled': 0, 'totalHeal': 18751, 'totalHealsOnTeammates': 0, 'totalMinionsKilled': 72, 'totalTimeCCDealt': 1213, 'totalTimeSpentDead': 398, 'totalUnitsHealed': 1, 'tripleKills': 0, 'trueDamageDealt': 44069, 'trueDamageDealtToChampions': 1366, 'trueDamageTaken': 7205, 'turretKills': 0, 'turretTakedowns': 0, 'turretsLost': 10, 'unrealKills': 0, 'visionClearedPings': 0, 'visionScore': 33, 'visionWardsBoughtInGame': 6, 'wardsKilled': 3, 'wardsPlaced': 8, 'win': False}, {'allInPings': 0, 'assistMePings': 9, 'assists': 3, 'baitPings': 0, 'baronKills': 0, 'basicPings': 0, 'bountyLevel': 0, 'challenges': {'12AssistStreakCount': 0, 'abilityUses': 647, 'acesBefore15Minutes': 0, 'alliedJungleMonsterKills': 3, 'baronTakedowns': 1, 'blastConeOppositeOpponentCount': 0, 'bountyGold': 800, 'buffsStolen': 0, 'completeSupportQuestInTime': 0, 'controlWardsPlaced': 0, 'damagePerMinute': 1083.557185778894, 'damageTakenOnTeamPercentage': 0.22598831396423713, 'dancedWithRiftHerald': 0, 'deathsByEnemyChamps': 12, 'dodgeSkillShotsSmallWindow': 11, 'doubleAces': 0, 'dragonTakedowns': 0, 'earliestBaron': 1657.7682029, 'earlyLaningPhaseGoldExpAdvantage': 0, 'effectiveHealAndShielding': 0, 'elderDragonKillsWithOpposingSoul': 0, 'elderDragonMultikills': 0, 'enemyChampionImmobilizations': 12, 'enemyJungleMonsterKills': 0, 'epicMonsterKillsNearEnemyJungler': 0, 'epicMonsterKillsWithin30SecondsOfSpawn': 0, 'epicMonsterSteals': 0, 'epicMonsterStolenWithoutSmite': 0, 'firstTurretKilled': 0, 'flawlessAces': 0, 'fullTeamTakedown': 0, 'gameLength': 2158.4759347, 'getTakedownsInAllLanesEarlyJungleAsLaner': 0, 'goldPerMinute': 349.06166838604037, 'hadOpenNexus': 0, 'immobilizeAndKillWithAlly': 1, 'initialBuffCount': 0, 'initialCrabCount': 0, 'jungleCsBefore10Minutes': 0, 'junglerTakedownsNearDamagedEpicMonster': 0, 'kTurretsDestroyedBeforePlatesFall': 0, 'kda': 0.5833333333333334, 'killAfterHiddenWithAlly': 1, 'killParticipation': 0.23333333333333334, 'killedChampTookFullTeamDamageSurvived': 0, 'killingSprees': 0, 'killsNearEnemyTurret': 0, 'killsOnOtherLanesEarlyJungleAsLaner': 0, 'killsOnRecentlyHealedByAramPack': 0, 'killsUnderOwnTurret': 1, 'killsWithHelpFromEpicMonster': 0, 'knockEnemyIntoTeamAndKill': 0, 'landSkillShotsEarlyGame': 4, 'laneMinionsFirst10Minutes': 72, 'laningPhaseGoldExpAdvantage': 0, 'legendaryCount': 0, 'lostAnInhibitor': 0, 'maxCsAdvantageOnLaneOpponent': 48, 'maxKillDeficit': 0, 'maxLevelLeadLaneOpponent': 1, 'mejaisFullStackInTime': 0, 'moreEnemyJungleThanOpponent': 0, 'multiKillOneSpell': 0, 'multiTurretRiftHeraldCount': 0, 'multikills': 0, 'multikillsAfterAggressiveFlash': 0, 'mythicItemUsed': 6653, 'outerTurretExecutesBefore10Minutes': 0, 'outnumberedKills': 0, 'outnumberedNexusKill': 0, 'perfectDragonSoulsTaken': 0, 'perfectGame': 0, 'pickKillWithAlly': 5, 'poroExplosions': 0, 'quickCleanse': 0, 'quickFirstTurret': 0, 'quickSoloKills': 0, 'riftHeraldTakedowns': 0, 'saveAllyFromDeath': 0, 'scuttleCrabKills': 0, 'skillshotsDodged': 71, 'skillshotsHit': 21, 'snowballsHit': 0, 'soloBaronKills': 0, 'soloKills': 2, 'stealthWardsPlaced': 12, 'survivedSingleDigitHpCount': 0, 'survivedThreeImmobilizesInFight': 6, 'takedownOnFirstTurret': 0, 'takedowns': 7, 'takedownsAfterGainingLevelAdvantage': 0, 'takedownsBeforeJungleMinionSpawn': 0, 'takedownsFirstXMinutes': 2, 'takedownsInAlcove': 1, 'takedownsInEnemyFountain': 0, 'teamBaronKills': 1, 'teamDamagePercentage': 0.24730289687255055, 'teamElderDragonKills': 0, 'teamRiftHeraldKills': 0, 'tookLargeDamageSurvived': 0, 'turretPlatesTaken': 0, 'turretTakedowns': 0, 'turretsTakenWithRiftHerald': 0, 'twentyMinionsIn3SecondsCount': 0, 'twoWardsOneSweeperCount': 0, 'unseenRecalls': 0, 'visionScoreAdvantageLaneOpponent': -0.24054467678070068, 'visionScorePerMinute': 0.6805494736539871, 'wardTakedowns': 2, 'wardTakedownsBefore20M': 2, 'wardsGuarded': 0}, 'champExperience': 16642, 'champLevel': 17, 'championId': 69, 'championName': 'Cassiopeia', 'championTransform': 0, 'commandPings': 0, 'consumablesPurchased': 3, 'damageDealtToBuildings': 0, 'damageDealtToObjectives': 11116, 'damageDealtToTurrets': 0, 'damageSelfMitigated': 23139, 'dangerPings': 0, 'deaths': 12, 'detectorWardsPlaced': 0, 'doubleKills': 0, 'dragonKills': 0, 'eligibleForProgression': True, 'enemyMissingPings': 9, 'enemyVisionPings': 0, 'firstBloodAssist': False, 'firstBloodKill': False, 'firstTowerAssist': False, 'firstTowerKill': False, 'gameEndedInEarlySurrender': False, 'gameEndedInSurrender': True, 'getBackPings': 6, 'goldEarned': 12557, 'goldSpent': 11450, 'holdPings': 0, 'individualPosition': 'TOP', 'inhibitorKills': 0, 'inhibitorTakedowns': 0, 'inhibitorsLost': 2, 'item0': 3040, 'item1': 6653, 'item2': 3116, 'item3': 1058, 'item4': 1058, 'item5': 0, 'item6': 3340, 'itemsPurchased': 21, 'killingSprees': 0, 'kills': 4, 'lane': 'JUNGLE', 'largestCriticalStrike': 0, 'largestKillingSpree': 0, 'largestMultiKill': 1, 'longestTimeSpentLiving': 336, 'magicDamageDealt': 142566, 'magicDamageDealtToChampions': 35481, 'magicDamageTaken': 20471, 'missions': {'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0}, 'needVisionPings': 1, 'neutralMinionsKilled': 4, 'nexusKills': 0, 'nexusLost': 1, 'nexusTakedowns': 0, 'objectivesStolen': 0, 'objectivesStolenAssists': 0, 'onMyWayPings': 6, 'participantId': 2, 'pentaKills': 0, 'perks': {'statPerks': {'defense': 5003, 'flex': 5008, 'offense': 5008}, 'styles': [{'description': 'primaryStyle', 'selections': [{'perk': 8010, 'var1': 928, 'var2': 0, 'var3': 0}, {'perk': 8009, 'var1': 3947, 'var2': 0, 'var3': 0}, {'perk': 9105, 'var1': 21, 'var2': 30, 'var3': 0}, {'perk': 8299, 'var1': 1829, 'var2': 0, 'var3': 0}], 'style': 8000}, {'description': 'subStyle', 'selections': [{'perk': 8226, 'var1': 250, 'var2': 2017, 'var3': 0}, {'perk': 8210, 'var1': 9, 'var2': 0, 'var3': 0}], 'style': 8200}]}, 'physicalDamageDealt': 21357, 'physicalDamageDealtToChampions': 3429, 'physicalDamageTaken': 19979, 'placement': 0, 'playerAugment1': 0, 'playerAugment2': 0, 'playerAugment3': 0, 'playerAugment4': 0, 'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0, 'playerSubteamId': 0, 'profileIcon': 11, 'pushPings': 0, 'puuid': 'PY7iMB_4ohr2vmtOB-GaUAFMb6owPm9ek4DJUBYt4IoEhxIvz3RrwACfnRDNv8uicE4HxaW-TiAtzg', 'quadraKills': 0, 'riotIdName': '', 'riotIdTagline': '', 'role': 'NONE', 'sightWardsBoughtInGame': 0, 'spell1Casts': 187, 'spell2Casts': 37, 'spell3Casts': 407, 'spell4Casts': 16, 'subteamPlacement': 0, 'summoner1Casts': 6, 'summoner1Id': 4, 'summoner2Casts': 5, 'summoner2Id': 12, 'summonerId': 'XzKi7SrTEcXzb4uXfz-VWXO3SMgGG-faQOAip9lB4rthsK8tT5i9Hdo99w', 'summonerLevel': 39, 'summonerName': 'Icky Viktor', 'teamEarlySurrendered': False, 'teamId': 100, 'teamPosition': 'TOP', 'timeCCingOthers': 54, 'timePlayed': 2158, 'totalAllyJungleMinionsKilled': 4, 'totalDamageDealt': 164317, 'totalDamageDealtToChampions': 38980, 'totalDamageShieldedOnTeammates': 0, 'totalDamageTaken': 41269, 'totalEnemyJungleMinionsKilled': 0, 'totalHeal': 5872, 'totalHealsOnTeammates': 0, 'totalMinionsKilled': 226, 'totalTimeCCDealt': 473, 'totalTimeSpentDead': 417, 'totalUnitsHealed': 1, 'tripleKills': 0, 'trueDamageDealt': 393, 'trueDamageDealtToChampions': 70, 'trueDamageTaken': 818, 'turretKills': 0, 'turretTakedowns': 0, 'turretsLost': 10, 'unrealKills': 0, 'visionClearedPings': 0, 'visionScore': 24, 'visionWardsBoughtInGame': 0, 'wardsKilled': 2, 'wardsPlaced': 12, 'win': False}, {'allInPings': 0, 'assistMePings': 2, 'assists': 7, 'baitPings': 0, 'baronKills': 0, 'basicPings': 0, 'bountyLevel': 0, 'challenges': {'12AssistStreakCount': 0, 'abilityUses': 327, 'acesBefore15Minutes': 0, 'alliedJungleMonsterKills': 0, 'baronTakedowns': 0, 'blastConeOppositeOpponentCount': 0, 'bountyGold': 0, 'buffsStolen': 0, 'completeSupportQuestInTime': 0, 'controlWardTimeCoverageInRiverOrEnemyHalf': 0.753495733287408, 'controlWardsPlaced': 3, 'damagePerMinute': 895.9700743750896, 'damageTakenOnTeamPercentage': 0.1957433771205483, 'dancedWithRiftHerald': 0, 'deathsByEnemyChamps': 18, 'dodgeSkillShotsSmallWindow': 35, 'doubleAces': 0, 'dragonTakedowns': 0, 'earliestBaron': 1657.7682029, 'earlyLaningPhaseGoldExpAdvantage': 0, 'effectiveHealAndShielding': 0, 'elderDragonKillsWithOpposingSoul': 0, 'elderDragonMultikills': 0, 'enemyChampionImmobilizations': 15, 'enemyJungleMonsterKills': 0, 'epicMonsterKillsNearEnemyJungler': 0, 'epicMonsterKillsWithin30SecondsOfSpawn': 0, 'epicMonsterSteals': 0, 'epicMonsterStolenWithoutSmite': 0, 'firstTurretKilled': 0, 'flawlessAces': 0, 'fullTeamTakedown': 0, 'gameLength': 2158.4759347, 'getTakedownsInAllLanesEarlyJungleAsLaner': 0, 'goldPerMinute': 352.98218638392865, 'hadOpenNexus': 0, 'immobilizeAndKillWithAlly': 3, 'initialBuffCount': 0, 'initialCrabCount': 0, 'jungleCsBefore10Minutes': 0, 'junglerTakedownsNearDamagedEpicMonster': 0, 'kTurretsDestroyedBeforePlatesFall': 0, 'kda': 0.4444444444444444, 'killAfterHiddenWithAlly': 2, 'killParticipation': 0.26666666666666666, 'killedChampTookFullTeamDamageSurvived': 0, 'killingSprees': 0, 'killsNearEnemyTurret': 0, 'killsOnOtherLanesEarlyJungleAsLaner': 0, 'killsOnRecentlyHealedByAramPack': 0, 'killsUnderOwnTurret': 0, 'killsWithHelpFromEpicMonster': 0, 'knockEnemyIntoTeamAndKill': 0, 'landSkillShotsEarlyGame': 20, 'laneMinionsFirst10Minutes': 68, 'laningPhaseGoldExpAdvantage': 0, 'legendaryCount': 0, 'lostAnInhibitor': 0, 'maxCsAdvantageOnLaneOpponent': 20, 'maxKillDeficit': 0, 'maxLevelLeadLaneOpponent': 1, 'mejaisFullStackInTime': 0, 'moreEnemyJungleThanOpponent': 0, 'multiKillOneSpell': 0, 'multiTurretRiftHeraldCount': 0, 'multikills': 0, 'multikillsAfterAggressiveFlash': 0, 'mythicItemUsed': 6653, 'outerTurretExecutesBefore10Minutes': 0, 'outnumberedKills': 0, 'outnumberedNexusKill': 0, 'perfectDragonSoulsTaken': 0, 'perfectGame': 0, 'pickKillWithAlly': 7, 'playedChampSelectPosition': 1, 'poroExplosions': 0, 'quickCleanse': 0, 'quickFirstTurret': 0, 'quickSoloKills': 0, 'riftHeraldTakedowns': 0, 'saveAllyFromDeath': 0, 'scuttleCrabKills': 0, 'skillshotsDodged': 252, 'skillshotsHit': 103, 'snowballsHit': 0, 'soloBaronKills': 0, 'soloKills': 0, 'stealthWardsPlaced': 13, 'survivedSingleDigitHpCount': 0, 'survivedThreeImmobilizesInFight': 0, 'takedownOnFirstTurret': 0, 'takedowns': 8, 'takedownsAfterGainingLevelAdvantage': 0, 'takedownsBeforeJungleMinionSpawn': 0, 'takedownsFirstXMinutes': 3, 'takedownsInAlcove': 0, 'takedownsInEnemyFountain': 0, 'teamBaronKills': 1, 'teamDamagePercentage': 0.20448943333322883, 'teamElderDragonKills': 0, 'teamRiftHeraldKills': 0, 'tookLargeDamageSurvived': 0, 'turretPlatesTaken': 1, 'turretTakedowns': 2, 'turretsTakenWithRiftHerald': 0, 'twentyMinionsIn3SecondsCount': 0, 'twoWardsOneSweeperCount': 0, 'unseenRecalls': 0, 'visionScoreAdvantageLaneOpponent': 0.02863776683807373, 'visionScorePerMinute': 1.2558606946186222, 'wardTakedowns': 4, 'wardTakedownsBefore20M': 3, 'wardsGuarded': 1}, 'champExperience': 16730, 'champLevel': 17, 'championId': 74, 'championName': 'Heimerdinger', 'championTransform': 0, 'commandPings': 1, 'consumablesPurchased': 10, 'damageDealtToBuildings': 3837, 'damageDealtToObjectives': 6511, 'damageDealtToTurrets': 3837, 'damageSelfMitigated': 16102, 'dangerPings': 0, 'deaths': 18, 'detectorWardsPlaced': 3, 'doubleKills': 0, 'dragonKills': 0, 'eligibleForProgression': True, 'enemyMissingPings': 4, 'enemyVisionPings': 0, 'firstBloodAssist': False, 'firstBloodKill': False, 'firstTowerAssist': False, 'firstTowerKill': False, 'gameEndedInEarlySurrender': False, 'gameEndedInSurrender': True, 'getBackPings': 1, 'goldEarned': 12698, 'goldSpent': 11275, 'holdPings': 0, 'individualPosition': 'MIDDLE', 'inhibitorKills': 0, 'inhibitorTakedowns': 0, 'inhibitorsLost': 2, 'item0': 2003, 'item1': 1056, 'item2': 3157, 'item3': 3165, 'item4': 6653, 'item5': 3020, 'item6': 3340, 'itemsPurchased': 24, 'killingSprees': 0, 'kills': 1, 'lane': 'MIDDLE', 'largestCriticalStrike': 0, 'largestKillingSpree': 0, 'largestMultiKill': 1, 'longestTimeSpentLiving': 223, 'magicDamageDealt': 179651, 'magicDamageDealtToChampions': 31095, 'magicDamageTaken': 19591, 'missions': {'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0}, 'needVisionPings': 0, 'neutralMinionsKilled': 0, 'nexusKills': 0, 'nexusLost': 1, 'nexusTakedowns': 0, 'objectivesStolen': 0, 'objectivesStolenAssists': 0, 'onMyWayPings': 3, 'participantId': 3, 'pentaKills': 0, 'perks': {'statPerks': {'defense': 5003, 'flex': 5008, 'offense': 5007}, 'styles': [{'description': 'primaryStyle', 'selections': [{'perk': 8229, 'var1': 2231, 'var2': 0, 'var3': 0}, {'perk': 8226, 'var1': 250, 'var2': 1372, 'var3': 0}, {'perk': 8210, 'var1': 13, 'var2': 0, 'var3': 0}, {'perk': 8237, 'var1': 1265, 'var2': 0, 'var3': 0}], 'style': 8200}, {'description': 'subStyle', 'selections': [{'perk': 8135, 'var1': 320, 'var2': 4, 'var3': 0}, {'perk': 8139, 'var1': 1191, 'var2': 0, 'var3': 0}], 'style': 8100}]}, 'physicalDamageDealt': 15007, 'physicalDamageDealtToChampions': 1136, 'physicalDamageTaken': 14864, 'placement': 0, 'playerAugment1': 0, 'playerAugment2': 0, 'playerAugment3': 0, 'playerAugment4': 0, 'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0, 'playerSubteamId': 0, 'profileIcon': 21, 'pushPings': 0, 'puuid': 'vr_-Loy-Fc_iHTYyWWa383FyLCw6c0iKLnVREVa6Vtab4bAqMcE9XLbFaxnsgqwT5GTjELIuhm7Eyw', 'quadraKills': 0, 'riotIdName': '', 'riotIdTagline': '', 'role': 'SOLO', 'sightWardsBoughtInGame': 0, 'spell1Casts': 123, 'spell2Casts': 114, 'spell3Casts': 72, 'spell4Casts': 18, 'subteamPlacement': 0, 'summoner1Casts': 4, 'summoner1Id': 4, 'summoner2Casts': 4, 'summoner2Id': 12, 'summonerId': '2OYBKao4wuXO3WVqfy58hycAmwlBe3mEV-mP2ky7nYscfAo', 'summonerLevel': 447, 'summonerName': 'Russ2k', 'teamEarlySurrendered': False, 'teamId': 100, 'teamPosition': 'MIDDLE', 'timeCCingOthers': 28, 'timePlayed': 2158, 'totalAllyJungleMinionsKilled': 0, 'totalDamageDealt': 196298, 'totalDamageDealtToChampions': 32232, 'totalDamageShieldedOnTeammates': 0, 'totalDamageTaken': 35745, 'totalEnemyJungleMinionsKilled': 0, 'totalHeal': 1002, 'totalHealsOnTeammates': 0, 'totalMinionsKilled': 212, 'totalTimeCCDealt': 378, 'totalTimeSpentDead': 594, 'totalUnitsHealed': 1, 'tripleKills': 0, 'trueDamageDealt': 1640, 'trueDamageDealtToChampions': 0, 'trueDamageTaken': 1290, 'turretKills': 2, 'turretTakedowns': 2, 'turretsLost': 10, 'unrealKills': 0, 'visionClearedPings': 0, 'visionScore': 45, 'visionWardsBoughtInGame': 3, 'wardsKilled': 4, 'wardsPlaced': 16, 'win': False}, {'allInPings': 0, 'assistMePings': 0, 'assists': 5, 'baitPings': 0, 'baronKills': 0, 'basicPings': 0, 'bountyLevel': 0, 'challenges': {'12AssistStreakCount': 0, 'abilityUses': 368, 'acesBefore15Minutes': 0, 'alliedJungleMonsterKills': 3, 'baronTakedowns': 1, 'blastConeOppositeOpponentCount': 0, 'bountyGold': 300, 'buffsStolen': 0, 'completeSupportQuestInTime': 0, 'controlWardsPlaced': 0, 'damagePerMinute': 935.6875515875076, 'damageTakenOnTeamPercentage': 0.18680074198641475, 'dancedWithRiftHerald': 0, 'deathsByEnemyChamps': 7, 'dodgeSkillShotsSmallWindow': 18, 'doubleAces': 0, 'dragonTakedowns': 0, 'earliestBaron': 1657.7682029, 'earlyLaningPhaseGoldExpAdvantage': 0, 'effectiveHealAndShielding': 195.80001831054688, 'elderDragonKillsWithOpposingSoul': 0, 'elderDragonMultikills': 0, 'enemyChampionImmobilizations': 0, 'enemyJungleMonsterKills': 0, 'epicMonsterKillsNearEnemyJungler': 0, 'epicMonsterKillsWithin30SecondsOfSpawn': 0, 'epicMonsterSteals': 0, 'epicMonsterStolenWithoutSmite': 0, 'firstTurretKilled': 0, 'flawlessAces': 0, 'fullTeamTakedown': 0, 'gameLength': 2158.4759347, 'getTakedownsInAllLanesEarlyJungleAsLaner': 0, 'goldPerMinute': 413.9642878386732, 'hadOpenNexus': 0, 'immobilizeAndKillWithAlly': 0, 'initialBuffCount': 0, 'initialCrabCount': 0, 'jungleCsBefore10Minutes': 0, 'junglerTakedownsNearDamagedEpicMonster': 0, 'kTurretsDestroyedBeforePlatesFall': 0, 'kda': 2, 'killAfterHiddenWithAlly': 1, 'killParticipation': 0.4666666666666667, 'killedChampTookFullTeamDamageSurvived': 0, 'killingSprees': 1, 'killsNearEnemyTurret': 0, 'killsOnOtherLanesEarlyJungleAsLaner': 0, 'killsOnRecentlyHealedByAramPack': 0, 'killsUnderOwnTurret': 3, 'killsWithHelpFromEpicMonster': 0, 'knockEnemyIntoTeamAndKill': 0, 'landSkillShotsEarlyGame': 13, 'laneMinionsFirst10Minutes': 55, 'laningPhaseGoldExpAdvantage': 0, 'legendaryCount': 0, 'lostAnInhibitor': 0, 'maxCsAdvantageOnLaneOpponent': 9, 'maxKillDeficit': 0, 'maxLevelLeadLaneOpponent': 2, 'mejaisFullStackInTime': 0, 'moreEnemyJungleThanOpponent': 0, 'multiKillOneSpell': 0, 'multiTurretRiftHeraldCount': 0, 'multikills': 1, 'multikillsAfterAggressiveFlash': 0, 'mythicItemUsed': 3078, 'outerTurretExecutesBefore10Minutes': 0, 'outnumberedKills': 2, 'outnumberedNexusKill': 0, 'perfectDragonSoulsTaken': 0, 'perfectGame': 0, 'pickKillWithAlly': 12, 'playedChampSelectPosition': 1, 'poroExplosions': 0, 'quickCleanse': 0, 'quickFirstTurret': 0, 'quickSoloKills': 0, 'riftHeraldTakedowns': 0, 'saveAllyFromDeath': 0, 'scuttleCrabKills': 2, 'skillshotsDodged': 115, 'skillshotsHit': 126, 'snowballsHit': 0, 'soloBaronKills': 0, 'soloKills': 1, 'stealthWardsPlaced': 3, 'survivedSingleDigitHpCount': 0, 'survivedThreeImmobilizesInFight': 2, 'takedownOnFirstTurret': 0, 'takedowns': 14, 'takedownsAfterGainingLevelAdvantage': 0, 'takedownsBeforeJungleMinionSpawn': 0, 'takedownsFirstXMinutes': 3, 'takedownsInAlcove': 0, 'takedownsInEnemyFountain': 0, 'teamBaronKills': 1, 'teamDamagePercentage': 0.2135542499391378, 'teamElderDragonKills': 0, 'teamRiftHeraldKills': 0, 'tookLargeDamageSurvived': 0, 'turretPlatesTaken': 1, 'turretTakedowns': 2, 'turretsTakenWithRiftHerald': 0, 'twentyMinionsIn3SecondsCount': 0, 'twoWardsOneSweeperCount': 0, 'unseenRecalls': 0, 'visionScoreAdvantageLaneOpponent': -0.5843740105628967, 'visionScorePerMinute': 0.3165741216537584, 'wardTakedowns': 2, 'wardTakedownsBefore20M': 0, 'wardsGuarded': 1}, 'champExperience': 18038, 'champLevel': 17, 'championId': 81, 'championName': 'Ezreal', 'championTransform': 0, 'commandPings': 6, 'consumablesPurchased': 1, 'damageDealtToBuildings': 4698, 'damageDealtToObjectives': 8069, 'damageDealtToTurrets': 4698, 'damageSelfMitigated': 13849, 'dangerPings': 0, 'deaths': 7, 'detectorWardsPlaced': 0, 'doubleKills': 1, 'dragonKills': 0, 'eligibleForProgression': True, 'enemyMissingPings': 2, 'enemyVisionPings': 0, 'firstBloodAssist': False, 'firstBloodKill': False, 'firstTowerAssist': False, 'firstTowerKill': False, 'gameEndedInEarlySurrender': False, 'gameEndedInSurrender': True, 'getBackPings': 0, 'goldEarned': 14892, 'goldSpent': 14493, 'holdPings': 0, 'individualPosition': 'BOTTOM', 'inhibitorKills': 0, 'inhibitorTakedowns': 0, 'inhibitorsLost': 2, 'item0': 6694, 'item1': 3078, 'item2': 3042, 'item3': 3074, 'item4': 0, 'item5': 3158, 'item6': 3363, 'itemsPurchased': 20, 'killingSprees': 1, 'kills': 9, 'lane': 'BOTTOM', 'largestCriticalStrike': 0, 'largestKillingSpree': 7, 'largestMultiKill': 2, 'longestTimeSpentLiving': 588, 'magicDamageDealt': 22313, 'magicDamageDealtToChampions': 7219, 'magicDamageTaken': 17291, 'missions': {'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0}, 'needVisionPings': 0, 'neutralMinionsKilled': 12, 'nexusKills': 0, 'nexusLost': 1, 'nexusTakedowns': 0, 'objectivesStolen': 0, 'objectivesStolenAssists': 0, 'onMyWayPings': 0, 'participantId': 4, 'pentaKills': 0, 'perks': {'statPerks': {'defense': 5002, 'flex': 5008, 'offense': 5005}, 'styles': [{'description': 'primaryStyle', 'selections': [{'perk': 8005, 'var1': 1454, 'var2': 832, 'var3': 622}, {'perk': 8009, 'var1': 4442, 'var2': 0, 'var3': 0}, {'perk': 9103, 'var1': 24, 'var2': 20, 'var3': 0}, {'perk': 8014, 'var1': 690, 'var2': 0, 'var3': 0}], 'style': 8000}, {'description': 'subStyle', 'selections': [{'perk': 8304, 'var1': 12, 'var2': 0, 'var3': 0}, {'perk': 8345, 'var1': 3, 'var2': 0, 'var3': 0}], 'style': 8300}]}, 'physicalDamageDealt': 153705, 'physicalDamageDealtToChampions': 26016, 'physicalDamageTaken': 16354, 'placement': 0, 'playerAugment1': 0, 'playerAugment2': 0, 'playerAugment3': 0, 'playerAugment4': 0, 'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0, 'playerSubteamId': 0, 'profileIcon': 3008, 'pushPings': 0, 'puuid': '34wqa3OKa6UWHSPIvs3i0BKlIYmR-dmz3C7uLs9o58K6O-ZnRjoBRJ4km59gerGJ546Fcr7S0ysaxw', 'quadraKills': 0, 'riotIdName': '', 'riotIdTagline': '', 'role': 'CARRY', 'sightWardsBoughtInGame': 0, 'spell1Casts': 270, 'spell2Casts': 52, 'spell3Casts': 33, 'spell4Casts': 13, 'subteamPlacement': 0, 'summoner1Casts': 5, 'summoner1Id': 4, 'summoner2Casts': 5, 'summoner2Id': 7, 'summonerId': 'bpfPS6F6No-4HKCZ6Ds9TnzdM72fm7KJaeJlEcskuNppQ_Q', 'summonerLevel': 375, 'summonerName': 'ShoCrispy', 'teamEarlySurrendered': False, 'teamId': 100, 'teamPosition': 'BOTTOM', 'timeCCingOthers': 11, 'timePlayed': 2158, 'totalAllyJungleMinionsKilled': 4, 'totalDamageDealt': 179178, 'totalDamageDealtToChampions': 33660, 'totalDamageShieldedOnTeammates': 0, 'totalDamageTaken': 34112, 'totalEnemyJungleMinionsKilled': 0, 'totalHeal': 6373, 'totalHealsOnTeammates': 195, 'totalMinionsKilled': 234, 'totalTimeCCDealt': 122, 'totalTimeSpentDead': 267, 'totalUnitsHealed': 2, 'tripleKills': 0, 'trueDamageDealt': 3159, 'trueDamageDealtToChampions': 425, 'trueDamageTaken': 467, 'turretKills': 2, 'turretTakedowns': 2, 'turretsLost': 10, 'unrealKills': 0, 'visionClearedPings': 0, 'visionScore': 11, 'visionWardsBoughtInGame': 0, 'wardsKilled': 2, 'wardsPlaced': 7, 'win': False}, {'allInPings': 0, 'assistMePings': 0, 'assists': 13, 'baitPings': 0, 'baronKills': 0, 'basicPings': 0, 'bountyLevel': 0, 'challenges': {'12AssistStreakCount': 1, 'abilityUses': 148, 'acesBefore15Minutes': 0, 'alliedJungleMonsterKills': 0, 'baronTakedowns': 0, 'blastConeOppositeOpponentCount': 0, 'bountyGold': 0, 'buffsStolen': 0, 'completeSupportQuestInTime': 0, 'controlWardTimeCoverageInRiverOrEnemyHalf': 0.6262096394361065, 'controlWardsPlaced': 11, 'damagePerMinute': 136.79166799630661, 'damageTakenOnTeamPercentage': 0.10534221619617502, 'dancedWithRiftHerald': 0, 'deathsByEnemyChamps': 7, 'dodgeSkillShotsSmallWindow': 14, 'doubleAces': 0, 'dragonTakedowns': 0, 'earliestBaron': 1657.7682029, 'earlyLaningPhaseGoldExpAdvantage': 0, 'effectiveHealAndShielding': 15736.30859375, 'elderDragonKillsWithOpposingSoul': 0, 'elderDragonMultikills': 0, 'enemyChampionImmobilizations': 10, 'enemyJungleMonsterKills': 0, 'epicMonsterKillsNearEnemyJungler': 0, 'epicMonsterKillsWithin30SecondsOfSpawn': 0, 'epicMonsterSteals': 0, 'epicMonsterStolenWithoutSmite': 0, 'firstTurretKilled': 0, 'flawlessAces': 0, 'fullTeamTakedown': 0, 'gameLength': 2158.4759347, 'getTakedownsInAllLanesEarlyJungleAsLaner': 0, 'goldPerMinute': 223.9522883849922, 'hadOpenNexus': 0, 'immobilizeAndKillWithAlly': 2, 'initialBuffCount': 0, 'initialCrabCount': 0, 'jungleCsBefore10Minutes': 0, 'junglerTakedownsNearDamagedEpicMonster': 0, 'kTurretsDestroyedBeforePlatesFall': 0, 'kda': 1.8571428571428572, 'killAfterHiddenWithAlly': 0, 'killParticipation': 0.43333333333333335, 'killedChampTookFullTeamDamageSurvived': 0, 'killingSprees': 0, 'killsNearEnemyTurret': 0, 'killsOnOtherLanesEarlyJungleAsLaner': 0, 'killsOnRecentlyHealedByAramPack': 0, 'killsUnderOwnTurret': 0, 'killsWithHelpFromEpicMonster': 0, 'knockEnemyIntoTeamAndKill': 0, 'landSkillShotsEarlyGame': 5, 'laneMinionsFirst10Minutes': 16, 'laningPhaseGoldExpAdvantage': 0, 'legendaryCount': 0, 'lostAnInhibitor': 0, 'maxCsAdvantageOnLaneOpponent': 18, 'maxKillDeficit': 0, 'maxLevelLeadLaneOpponent': 1, 'mejaisFullStackInTime': 0, 'moreEnemyJungleThanOpponent': 0, 'multiKillOneSpell': 0, 'multiTurretRiftHeraldCount': 0, 'multikills': 0, 'multikillsAfterAggressiveFlash': 0, 'mythicItemUsed': 2065, 'outerTurretExecutesBefore10Minutes': 0, 'outnumberedKills': 0, 'outnumberedNexusKill': 0, 'perfectDragonSoulsTaken': 0, 'perfectGame': 0, 'pickKillWithAlly': 12, 'playedChampSelectPosition': 1, 'poroExplosions': 0, 'quickCleanse': 0, 'quickFirstTurret': 0, 'quickSoloKills': 0, 'riftHeraldTakedowns': 0, 'saveAllyFromDeath': 7, 'scuttleCrabKills': 0, 'skillshotsDodged': 102, 'skillshotsHit': 21, 'snowballsHit': 0, 'soloBaronKills': 0, 'soloKills': 0, 'stealthWardsPlaced': 32, 'survivedSingleDigitHpCount': 0, 'survivedThreeImmobilizesInFight': 2, 'takedownOnFirstTurret': 0, 'takedowns': 13, 'takedownsAfterGainingLevelAdvantage': 0, 'takedownsBeforeJungleMinionSpawn': 0, 'takedownsFirstXMinutes': 3, 'takedownsInAlcove': 0, 'takedownsInEnemyFountain': 0, 'teamBaronKills': 1, 'teamDamagePercentage': 0.031220295714431984, 'teamElderDragonKills': 0, 'teamRiftHeraldKills': 0, 'tookLargeDamageSurvived': 0, 'turretPlatesTaken': 1, 'turretTakedowns': 0, 'turretsTakenWithRiftHerald': 0, 'twentyMinionsIn3SecondsCount': 0, 'twoWardsOneSweeperCount': 1, 'unseenRecalls': 0, 'visionScoreAdvantageLaneOpponent': -0.018535315990447998, 'visionScorePerMinute': 2.714572656568273, 'wardTakedowns': 11, 'wardTakedownsBefore20M': 6, 'wardsGuarded': 0}, 'champExperience': 13148, 'champLevel': 15, 'championId': 16, 'championName': 'Soraka', 'championTransform': 0, 'commandPings': 0, 'consumablesPurchased': 18, 'damageDealtToBuildings': 345, 'damageDealtToObjectives': 345, 'damageDealtToTurrets': 345, 'damageSelfMitigated': 8370, 'dangerPings': 0, 'deaths': 7, 'detectorWardsPlaced': 11, 'doubleKills': 0, 'dragonKills': 0, 'eligibleForProgression': True, 'enemyMissingPings': 5, 'enemyVisionPings': 4, 'firstBloodAssist': False, 'firstBloodKill': False, 'firstTowerAssist': False, 'firstTowerKill': False, 'gameEndedInEarlySurrender': False, 'gameEndedInSurrender': True, 'getBackPings': 0, 'goldEarned': 8056, 'goldSpent': 6600, 'holdPings': 0, 'individualPosition': 'UTILITY', 'inhibitorKills': 0, 'inhibitorTakedowns': 0, 'inhibitorsLost': 2, 'item0': 2065, 'item1': 3801, 'item2': 3860, 'item3': 0, 'item4': 3067, 'item5': 3158, 'item6': 3364, 'itemsPurchased': 34, 'killingSprees': 0, 'kills': 0, 'lane': 'BOTTOM', 'largestCriticalStrike': 0, 'largestKillingSpree': 0, 'largestMultiKill': 0, 'longestTimeSpentLiving': 565, 'magicDamageDealt': 6446, 'magicDamageDealtToChampions': 4223, 'magicDamageTaken': 10437, 'missions': {'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0}, 'needVisionPings': 0, 'neutralMinionsKilled': 0, 'nexusKills': 0, 'nexusLost': 1, 'nexusTakedowns': 0, 'objectivesStolen': 0, 'objectivesStolenAssists': 0, 'onMyWayPings': 3, 'participantId': 5, 'pentaKills': 0, 'perks': {'statPerks': {'defense': 5002, 'flex': 5008, 'offense': 5008}, 'styles': [{'description': 'primaryStyle', 'selections': [{'perk': 8214, 'var1': 652, 'var2': 1091, 'var3': 0}, {'perk': 8226, 'var1': 250, 'var2': 407, 'var3': 0}, {'perk': 8210, 'var1': 6, 'var2': 0, 'var3': 0}, {'perk': 8237, 'var1': 537, 'var2': 0, 'var3': 0}], 'style': 8200}, {'description': 'subStyle', 'selections': [{'perk': 8473, 'var1': 823, 'var2': 0, 'var3': 0}, {'perk': 8453, 'var1': 2666, 'var2': 285, 'var3': 0}], 'style': 8400}]}, 'physicalDamageDealt': 3239, 'physicalDamageDealtToChampions': 697, 'physicalDamageTaken': 8685, 'placement': 0, 'playerAugment1': 0, 'playerAugment2': 0, 'playerAugment3': 0, 'playerAugment4': 0, 'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0, 'playerSubteamId': 0, 'profileIcon': 21, 'pushPings': 0, 'puuid': 'IwYjkxE9AlLTMZ_9Mf6YVpyGcQYOeQq2azNWgQhU2BXEm0vZn6Bvrxj7jNIEr-iVQAAxAqQFa4jDHg', 'quadraKills': 0, 'riotIdName': '', 'riotIdTagline': '', 'role': 'SUPPORT', 'sightWardsBoughtInGame': 0, 'spell1Casts': 38, 'spell2Casts': 62, 'spell3Casts': 38, 'spell4Casts': 10, 'subteamPlacement': 0, 'summoner1Casts': 5, 'summoner1Id': 4, 'summoner2Casts': 7, 'summoner2Id': 3, 'summonerId': 'HMvtwffgxJcXZ-0ebi0JJC2PPidtPQ3jk8-B0UGdM94JjQ0', 'summonerLevel': 344, 'summonerName': 'FrostUlfhednar', 'teamEarlySurrendered': False, 'teamId': 100, 'teamPosition': 'UTILITY', 'timeCCingOthers': 47, 'timePlayed': 2158, 'totalAllyJungleMinionsKilled': 0, 'totalDamageDealt': 12469, 'totalDamageDealtToChampions': 4921, 'totalDamageShieldedOnTeammates': 1254, 'totalDamageTaken': 19237, 'totalEnemyJungleMinionsKilled': 0, 'totalHeal': 18933, 'totalHealsOnTeammates': 14481, 'totalMinionsKilled': 33, 'totalTimeCCDealt': 144, 'totalTimeSpentDead': 185, 'totalUnitsHealed': 5, 'tripleKills': 0, 'trueDamageDealt': 2783, 'trueDamageDealtToChampions': 0, 'trueDamageTaken': 113, 'turretKills': 0, 'turretTakedowns': 0, 'turretsLost': 10, 'unrealKills': 0, 'visionClearedPings': 0, 'visionScore': 97, 'visionWardsBoughtInGame': 12, 'wardsKilled': 10, 'wardsPlaced': 46, 'win': False}, {'allInPings': 0, 'assistMePings': 1, 'assists': 9, 'baitPings': 0, 'baronKills': 0, 'basicPings': 0, 'bountyLevel': 1, 'challenges': {'12AssistStreakCount': 0, 'abilityUses': 250, 'acesBefore15Minutes': 0, 'alliedJungleMonsterKills': 0, 'baronBuffGoldAdvantageOverThreshold': 1, 'baronTakedowns': 0, 'blastConeOppositeOpponentCount': 0, 'bountyGold': 0, 'buffsStolen': 1, 'completeSupportQuestInTime': 0, 'controlWardTimeCoverageInRiverOrEnemyHalf': 0.766887354678831, 'controlWardsPlaced': 3, 'damagePerMinute': 699.3837747303006, 'damageTakenOnTeamPercentage': 0.2687747076383493, 'dancedWithRiftHerald': 0, 'deathsByEnemyChamps': 5, 'dodgeSkillShotsSmallWindow': 9, 'doubleAces': 0, 'dragonTakedowns': 1, 'earliestBaron': 2074.9830690999997, 'earliestDragonTakedown': 1823.6520494, 'earlyLaningPhaseGoldExpAdvantage': 0, 'effectiveHealAndShielding': 0, 'elderDragonKillsWithOpposingSoul': 0, 'elderDragonMultikills': 0, 'enemyChampionImmobilizations': 70, 'enemyJungleMonsterKills': 7, 'epicMonsterKillsNearEnemyJungler': 0, 'epicMonsterKillsWithin30SecondsOfSpawn': 0, 'epicMonsterSteals': 0, 'epicMonsterStolenWithoutSmite': 0, 'firstTurretKilled': 1, 'firstTurretKilledTime': 947.9954345, 'flawlessAces': 1, 'fullTeamTakedown': 1, 'gameLength': 2158.4759347, 'getTakedownsInAllLanesEarlyJungleAsLaner': 0, 'goldPerMinute': 362.1064357395219, 'hadOpenNexus': 0, 'immobilizeAndKillWithAlly': 10, 'initialBuffCount': 0, 'initialCrabCount': 0, 'jungleCsBefore10Minutes': 0, 'junglerTakedownsNearDamagedEpicMonster': 0, 'kTurretsDestroyedBeforePlatesFall': 0, 'kda': 3.4, 'killAfterHiddenWithAlly': 0, 'killParticipation': 0.3148148148148148, 'killedChampTookFullTeamDamageSurvived': 0, 'killingSprees': 0, 'killsNearEnemyTurret': 1, 'killsOnOtherLanesEarlyJungleAsLaner': 0, 'killsOnRecentlyHealedByAramPack': 0, 'killsUnderOwnTurret': 1, 'killsWithHelpFromEpicMonster': 0, 'knockEnemyIntoTeamAndKill': 7, 'landSkillShotsEarlyGame': 5, 'laneMinionsFirst10Minutes': 66, 'laningPhaseGoldExpAdvantage': 0, 'legendaryCount': 0, 'lostAnInhibitor': 0, 'maxCsAdvantageOnLaneOpponent': 10, 'maxKillDeficit': 0, 'maxLevelLeadLaneOpponent': 2, 'mejaisFullStackInTime': 0, 'moreEnemyJungleThanOpponent': 0, 'multiKillOneSpell': 0, 'multiTurretRiftHeraldCount': 0, 'multikills': 0, 'multikillsAfterAggressiveFlash': 0, 'mythicItemUsed': 6665, 'outerTurretExecutesBefore10Minutes': 0, 'outnumberedKills': 0, 'outnumberedNexusKill': 0, 'perfectDragonSoulsTaken': 0, 'perfectGame': 0, 'pickKillWithAlly': 11, 'playedChampSelectPosition': 1, 'poroExplosions': 0, 'quickCleanse': 0, 'quickFirstTurret': 0, 'quickSoloKills': 0, 'riftHeraldTakedowns': 0, 'saveAllyFromDeath': 0, 'scuttleCrabKills': 0, 'skillshotsDodged': 84, 'skillshotsHit': 39, 'snowballsHit': 0, 'soloBaronKills': 0, 'soloKills': 3, 'soloTurretsLategame': 1, 'stealthWardsPlaced': 11, 'survivedSingleDigitHpCount': 0, 'survivedThreeImmobilizesInFight': 2, 'takedownOnFirstTurret': 0, 'takedowns': 17, 'takedownsAfterGainingLevelAdvantage': 0, 'takedownsBeforeJungleMinionSpawn': 0, 'takedownsFirstXMinutes': 4, 'takedownsInAlcove': 0, 'takedownsInEnemyFountain': 0, 'teamBaronKills': 1, 'teamDamagePercentage': 0.16521032085333118, 'teamElderDragonKills': 0, 'teamRiftHeraldKills': 1, 'tookLargeDamageSurvived': 1, 'turretPlatesTaken': 1, 'turretTakedowns': 2, 'turretsTakenWithRiftHerald': 0, 'twentyMinionsIn3SecondsCount': 0, 'twoWardsOneSweeperCount': 0, 'unseenRecalls': 0, 'visionScoreAdvantageLaneOpponent': 0.31673312187194824, 'visionScorePerMinute': 0.8961020483602145, 'wardTakedowns': 4, 'wardTakedownsBefore20M': 3, 'wardsGuarded': 1}, 'champExperience': 19027, 'champLevel': 18, 'championId': 516, 'championName': 'Ornn', 'championTransform': 0, 'commandPings': 5, 'consumablesPurchased': 6, 'damageDealtToBuildings': 4552, 'damageDealtToObjectives': 5085, 'damageDealtToTurrets': 4552, 'damageSelfMitigated': 116226, 'dangerPings': 0, 'deaths': 5, 'detectorWardsPlaced': 3, 'doubleKills': 0, 'dragonKills': 0, 'eligibleForProgression': True, 'enemyMissingPings': 1, 'enemyVisionPings': 1, 'firstBloodAssist': False, 'firstBloodKill': False, 'firstTowerAssist': False, 'firstTowerKill': False, 'gameEndedInEarlySurrender': False, 'gameEndedInSurrender': True, 'getBackPings': 0, 'goldEarned': 13026, 'goldSpent': 12125, 'holdPings': 0, 'individualPosition': 'TOP', 'inhibitorKills': 0, 'inhibitorTakedowns': 0, 'inhibitorsLost': 0, 'item0': 7026, 'item1': 3111, 'item2': 4401, 'item3': 3068, 'item4': 3076, 'item5': 3082, 'item6': 3340, 'itemsPurchased': 34, 'killingSprees': 2, 'kills': 8, 'lane': 'TOP', 'largestCriticalStrike': 0, 'largestKillingSpree': 2, 'largestMultiKill': 1, 'longestTimeSpentLiving': 708, 'magicDamageDealt': 66716, 'magicDamageDealtToChampions': 16879, 'magicDamageTaken': 25060, 'missions': {'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0}, 'needVisionPings': 0, 'neutralMinionsKilled': 8, 'nexusKills': 0, 'nexusLost': 0, 'nexusTakedowns': 0, 'objectivesStolen': 0, 'objectivesStolenAssists': 0, 'onMyWayPings': 3, 'participantId': 6, 'pentaKills': 0, 'perks': {'statPerks': {'defense': 5003, 'flex': 5002, 'offense': 5005}, 'styles': [{'description': 'primaryStyle', 'selections': [{'perk': 8437, 'var1': 2092, 'var2': 1248, 'var3': 0}, {'perk': 8446, 'var1': 3174, 'var2': 0, 'var3': 0}, {'perk': 8473, 'var1': 1068, 'var2': 0, 'var3': 0}, {'perk': 8451, 'var1': 295, 'var2': 0, 'var3': 0}], 'style': 8400}, {'description': 'subStyle', 'selections': [{'perk': 8304, 'var1': 9, 'var2': 4, 'var3': 5}, {'perk': 8345, 'var1': 3, 'var2': 0, 'var3': 0}], 'style': 8300}]}, 'physicalDamageDealt': 79000, 'physicalDamageDealtToChampions': 7148, 'physicalDamageTaken': 29488, 'placement': 0, 'playerAugment1': 0, 'playerAugment2': 0, 'playerAugment3': 0, 'playerAugment4': 0, 'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0, 'playerSubteamId': 0, 'profileIcon': 5726, 'pushPings': 0, 'puuid': 'bHhAY7DSUICKSCwWzf2EeKhtFsUq4fEOXZM4BsXK4CIXbAW0z-N2I-2yls25q9G__fugkZVhF7h_Pw', 'quadraKills': 0, 'riotIdName': '', 'riotIdTagline': '', 'role': 'SOLO', 'sightWardsBoughtInGame': 0, 'spell1Casts': 109, 'spell2Casts': 69, 'spell3Casts': 48, 'spell4Casts': 24, 'subteamPlacement': 0, 'summoner1Casts': 5, 'summoner1Id': 4, 'summoner2Casts': 6, 'summoner2Id': 12, 'summonerId': 'RMWLwLgy0eY8Fl1oC9tr3umr_jVVfXNFok39tP7JZq9A2FU', 'summonerLevel': 524, 'summonerName': 'Be Out Of Date', 'teamEarlySurrendered': False, 'teamId': 200, 'teamPosition': 'TOP', 'timeCCingOthers': 48, 'timePlayed': 2158, 'totalAllyJungleMinionsKilled': 0, 'totalDamageDealt': 156124, 'totalDamageDealtToChampions': 25160, 'totalDamageShieldedOnTeammates': 0, 'totalDamageTaken': 54909, 'totalEnemyJungleMinionsKilled': 8, 'totalHeal': 18178, 'totalHealsOnTeammates': 0, 'totalMinionsKilled': 191, 'totalTimeCCDealt': 720, 'totalTimeSpentDead': 197, 'totalUnitsHealed': 1, 'tripleKills': 0, 'trueDamageDealt': 10407, 'trueDamageDealtToChampions': 1132, 'trueDamageTaken': 361, 'turretKills': 1, 'turretTakedowns': 2, 'turretsLost': 4, 'unrealKills': 0, 'visionClearedPings': 0, 'visionScore': 32, 'visionWardsBoughtInGame': 3, 'wardsKilled': 4, 'wardsPlaced': 14, 'win': True}, {'allInPings': 0, 'assistMePings': 1, 'assists': 16, 'baitPings': 0, 'baronKills': 1, 'basicPings': 0, 'bountyLevel': 1, 'challenges': {'12AssistStreakCount': 0, 'abilityUses': 268, 'acesBefore15Minutes': 0, 'alliedJungleMonsterKills': 73, 'baronBuffGoldAdvantageOverThreshold': 1, 'baronTakedowns': 1, 'blastConeOppositeOpponentCount': 0, 'bountyGold': 300, 'buffsStolen': 1, 'completeSupportQuestInTime': 0, 'controlWardsPlaced': 0, 'damagePerMinute': 841.8503452947022, 'damageTakenOnTeamPercentage': 0.23221768010407318, 'dancedWithRiftHerald': 0, 'deathsByEnemyChamps': 3, 'dodgeSkillShotsSmallWindow': 3, 'doubleAces': 0, 'dragonTakedowns': 4, 'earliestBaron': 2074.9830690999997, 'earliestDragonTakedown': 790.9937994999999, 'earlyLaningPhaseGoldExpAdvantage': 0, 'effectiveHealAndShielding': 0, 'elderDragonKillsWithOpposingSoul': 0, 'elderDragonMultikills': 0, 'enemyChampionImmobilizations': 66, 'enemyJungleMonsterKills': 7, 'epicMonsterKillsNearEnemyJungler': 2, 'epicMonsterKillsWithin30SecondsOfSpawn': 1, 'epicMonsterSteals': 0, 'epicMonsterStolenWithoutSmite': 0, 'firstTurretKilled': 1, 'firstTurretKilledTime': 947.9954345, 'flawlessAces': 1, 'fullTeamTakedown': 1, 'gameLength': 2158.4759347, 'goldPerMinute': 465.2534063992592, 'hadOpenNexus': 0, 'immobilizeAndKillWithAlly': 17, 'initialBuffCount': 2, 'initialCrabCount': 0, 'jungleCsBefore10Minutes': 56.00000008940697, 'junglerKillsEarlyJungle': 0, 'junglerTakedownsNearDamagedEpicMonster': 1, 'kTurretsDestroyedBeforePlatesFall': 0, 'kda': 9, 'killAfterHiddenWithAlly': 3, 'killParticipation': 0.5, 'killedChampTookFullTeamDamageSurvived': 0, 'killingSprees': 1, 'killsNearEnemyTurret': 2, 'killsOnLanersEarlyJungleAsJungler': 1, 'killsOnRecentlyHealedByAramPack': 0, 'killsUnderOwnTurret': 1, 'killsWithHelpFromEpicMonster': 0, 'knockEnemyIntoTeamAndKill': 0, 'landSkillShotsEarlyGame': 0, 'laneMinionsFirst10Minutes': 4, 'laningPhaseGoldExpAdvantage': 0, 'legendaryCount': 0, 'lostAnInhibitor': 0, 'maxCsAdvantageOnLaneOpponent': 4, 'maxKillDeficit': 0, 'maxLevelLeadLaneOpponent': 1, 'mejaisFullStackInTime': 0, 'moreEnemyJungleThanOpponent': -59.00000008940697, 'multiKillOneSpell': 1, 'multiTurretRiftHeraldCount': 0, 'multikills': 0, 'multikillsAfterAggressiveFlash': 0, 'mythicItemUsed': 3152, 'outerTurretExecutesBefore10Minutes': 0, 'outnumberedKills': 2, 'outnumberedNexusKill': 0, 'perfectDragonSoulsTaken': 0, 'perfectGame': 0, 'pickKillWithAlly': 23, 'playedChampSelectPosition': 1, 'poroExplosions': 0, 'quickCleanse': 0, 'quickFirstTurret': 0, 'quickSoloKills': 0, 'riftHeraldTakedowns': 1, 'saveAllyFromDeath': 0, 'scuttleCrabKills': 2, 'shortestTimeToAceFromFirstTakedown': 42.70709119999992, 'skillshotsDodged': 48, 'skillshotsHit': 35, 'snowballsHit': 0, 'soloBaronKills': 0, 'soloKills': 1, 'stealthWardsPlaced': 0, 'survivedSingleDigitHpCount': 0, 'survivedThreeImmobilizesInFight': 0, 'takedownOnFirstTurret': 1, 'takedowns': 27, 'takedownsAfterGainingLevelAdvantage': 0, 'takedownsBeforeJungleMinionSpawn': 0, 'takedownsFirstXMinutes': 5, 'takedownsInAlcove': 0, 'takedownsInEnemyFountain': 1, 'teamBaronKills': 1, 'teamDamagePercentage': 0.19886415825168222, 'teamElderDragonKills': 0, 'teamRiftHeraldKills': 1, 'tookLargeDamageSurvived': 0, 'turretPlatesTaken': 0, 'turretTakedowns': 4, 'turretsTakenWithRiftHerald': 0, 'twentyMinionsIn3SecondsCount': 0, 'twoWardsOneSweeperCount': 0, 'unseenRecalls': 0, 'visionScoreAdvantageLaneOpponent': 0.13065576553344727, 'visionScorePerMinute': 1.047514505362541, 'wardTakedowns': 2, 'wardTakedownsBefore20M': 1, 'wardsGuarded': 0}, 'champExperience': 19685, 'champLevel': 18, 'championId': 9, 'championName': 'FiddleSticks', 'championTransform': 0, 'commandPings': 0, 'consumablesPurchased': 1, 'damageDealtToBuildings': 3537, 'damageDealtToObjectives': 49259, 'damageDealtToTurrets': 3537, 'damageSelfMitigated': 32987, 'dangerPings': 0, 'deaths': 3, 'detectorWardsPlaced': 0, 'doubleKills': 0, 'dragonKills': 4, 'eligibleForProgression': True, 'enemyMissingPings': 0, 'enemyVisionPings': 0, 'firstBloodAssist': False, 'firstBloodKill': False, 'firstTowerAssist': False, 'firstTowerKill': False, 'gameEndedInEarlySurrender': False, 'gameEndedInSurrender': True, 'getBackPings': 0, 'goldEarned': 16737, 'goldSpent': 15400, 'holdPings': 0, 'individualPosition': 'JUNGLE', 'inhibitorKills': 0, 'inhibitorTakedowns': 0, 'inhibitorsLost': 0, 'item0': 3157, 'item1': 3089, 'item2': 3020, 'item3': 3165, 'item4': 1058, 'item5': 7011, 'item6': 3330, 'itemsPurchased': 22, 'killingSprees': 3, 'kills': 11, 'lane': 'JUNGLE', 'largestCriticalStrike': 0, 'largestKillingSpree': 6, 'largestMultiKill': 1, 'longestTimeSpentLiving': 803, 'magicDamageDealt': 206168, 'magicDamageDealtToChampions': 26496, 'magicDamageTaken': 18779, 'missions': {'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0}, 'needVisionPings': 0, 'neutralMinionsKilled': 151, 'nexusKills': 0, 'nexusLost': 0, 'nexusTakedowns': 1, 'objectivesStolen': 0, 'objectivesStolenAssists': 0, 'onMyWayPings': 1, 'participantId': 7, 'pentaKills': 0, 'perks': {'statPerks': {'defense': 5002, 'flex': 5008, 'offense': 5008}, 'styles': [{'description': 'primaryStyle', 'selections': [{'perk': 8369, 'var1': 1067, 'var2': 847, 'var3': 0}, {'perk': 8313, 'var1': 0, 'var2': 0, 'var3': 0}, {'perk': 8321, 'var1': 2, 'var2': 0, 'var3': 0}, {'perk': 8347, 'var1': 0, 'var2': 0, 'var3': 0}], 'style': 8300}, {'description': 'subStyle', 'selections': [{'perk': 8126, 'var1': 983, 'var2': 0, 'var3': 0}, {'perk': 8106, 'var1': 5, 'var2': 0, 'var3': 0}], 'style': 8100}]}, 'physicalDamageDealt': 10738, 'physicalDamageDealtToChampions': 172, 'physicalDamageTaken': 28320, 'placement': 0, 'playerAugment1': 0, 'playerAugment2': 0, 'playerAugment3': 0, 'playerAugment4': 0, 'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0, 'playerSubteamId': 0, 'profileIcon': 4778, 'pushPings': 0, 'puuid': 'th3FWRsIG8MtnJZg-BaRLYqcMc5fggO4dl7z6lBCtR8fGtSmncvKqX2-G-Hm81htOfAsgqs5V393Dw', 'quadraKills': 0, 'riotIdName': '', 'riotIdTagline': '', 'role': 'NONE', 'sightWardsBoughtInGame': 0, 'spell1Casts': 33, 'spell2Casts': 145, 'spell3Casts': 76, 'spell4Casts': 14, 'subteamPlacement': 0, 'summoner1Casts': 21, 'summoner1Id': 11, 'summoner2Casts': 4, 'summoner2Id': 4, 'summonerId': 'x6VaZjFDO5pcpjldR7PGm14iVLoR5aW288n2XIvxG0linlA', 'summonerLevel': 119, 'summonerName': 'TrizzyLin', 'teamEarlySurrendered': False, 'teamId': 200, 'teamPosition': 'JUNGLE', 'timeCCingOthers': 65, 'timePlayed': 2158, 'totalAllyJungleMinionsKilled': 107, 'totalDamageDealt': 266695, 'totalDamageDealtToChampions': 30285, 'totalDamageShieldedOnTeammates': 0, 'totalDamageTaken': 47441, 'totalEnemyJungleMinionsKilled': 8, 'totalHeal': 31748, 'totalHealsOnTeammates': 0, 'totalMinionsKilled': 47, 'totalTimeCCDealt': 710, 'totalTimeSpentDead': 127, 'totalUnitsHealed': 1, 'tripleKills': 0, 'trueDamageDealt': 49787, 'trueDamageDealtToChampions': 3616, 'trueDamageTaken': 341, 'turretKills': 3, 'turretTakedowns': 4, 'turretsLost': 4, 'unrealKills': 0, 'visionClearedPings': 0, 'visionScore': 37, 'visionWardsBoughtInGame': 0, 'wardsKilled': 2, 'wardsPlaced': 0, 'win': True}, {'allInPings': 0, 'assistMePings': 0, 'assists': 7, 'baitPings': 0, 'baronKills': 0, 'basicPings': 0, 'bountyLevel': 7, 'challenges': {'12AssistStreakCount': 0, 'abilityUses': 326, 'acesBefore15Minutes': 0, 'alliedJungleMonsterKills': 0, 'baronBuffGoldAdvantageOverThreshold': 1, 'baronTakedowns': 0, 'blastConeOppositeOpponentCount': 0, 'bountyGold': 550, 'buffsStolen': 0, 'completeSupportQuestInTime': 0, 'controlWardTimeCoverageInRiverOrEnemyHalf': 0.7497515497317442, 'controlWardsPlaced': 3, 'damagePerMinute': 987.0611481573541, 'damageTakenOnTeamPercentage': 0.1956819718317292, 'dancedWithRiftHerald': 0, 'deathsByEnemyChamps': 6, 'dodgeSkillShotsSmallWindow': 9, 'doubleAces': 0, 'dragonTakedowns': 0, 'earliestBaron': 2074.9830690999997, 'earlyLaningPhaseGoldExpAdvantage': 0, 'effectiveHealAndShielding': 0, 'elderDragonKillsWithOpposingSoul': 0, 'elderDragonMultikills': 0, 'enemyChampionImmobilizations': 0, 'enemyJungleMonsterKills': 1, 'epicMonsterKillsNearEnemyJungler': 0, 'epicMonsterKillsWithin30SecondsOfSpawn': 0, 'epicMonsterSteals': 0, 'epicMonsterStolenWithoutSmite': 0, 'firstTurretKilled': 1, 'firstTurretKilledTime': 947.9954345, 'flawlessAces': 1, 'fullTeamTakedown': 1, 'gameLength': 2158.4759347, 'getTakedownsInAllLanesEarlyJungleAsLaner': 0, 'goldPerMinute': 475.0449831654081, 'hadOpenNexus': 0, 'immobilizeAndKillWithAlly': 0, 'initialBuffCount': 0, 'initialCrabCount': 0, 'jungleCsBefore10Minutes': 0, 'junglerTakedownsNearDamagedEpicMonster': 0, 'kTurretsDestroyedBeforePlatesFall': 0, 'kda': 4, 'killAfterHiddenWithAlly': 2, 'killParticipation': 0.4444444444444444, 'killedChampTookFullTeamDamageSurvived': 0, 'killingSprees': 3, 'killsNearEnemyTurret': 3, 'killsOnOtherLanesEarlyJungleAsLaner': 0, 'killsOnRecentlyHealedByAramPack': 0, 'killsUnderOwnTurret': 1, 'killsWithHelpFromEpicMonster': 0, 'knockEnemyIntoTeamAndKill': 0, 'landSkillShotsEarlyGame': 9, 'laneMinionsFirst10Minutes': 60, 'laningPhaseGoldExpAdvantage': 0, 'legendaryCount': 0, 'lostAnInhibitor': 0, 'maxCsAdvantageOnLaneOpponent': 14, 'maxKillDeficit': 0, 'maxLevelLeadLaneOpponent': 2, 'mejaisFullStackInTime': 0, 'moreEnemyJungleThanOpponent': 0, 'multiKillOneSpell': 1, 'multiTurretRiftHeraldCount': 0, 'multikills': 2, 'multikillsAfterAggressiveFlash': 0, 'mythicItemUsed': 3152, 'outerTurretExecutesBefore10Minutes': 0, 'outnumberedKills': 3, 'outnumberedNexusKill': 0, 'perfectDragonSoulsTaken': 0, 'perfectGame': 0, 'pickKillWithAlly': 16, 'playedChampSelectPosition': 1, 'poroExplosions': 0, 'quickCleanse': 0, 'quickFirstTurret': 0, 'quickSoloKills': 0, 'riftHeraldTakedowns': 0, 'saveAllyFromDeath': 0, 'scuttleCrabKills': 0, 'shortestTimeToAceFromFirstTakedown': 42.70709119999992, 'skillshotsDodged': 118, 'skillshotsHit': 43, 'snowballsHit': 0, 'soloBaronKills': 0, 'soloKills': 7, 'stealthWardsPlaced': 12, 'survivedSingleDigitHpCount': 1, 'survivedThreeImmobilizesInFight': 0, 'takedownOnFirstTurret': 0, 'takedowns': 24, 'takedownsAfterGainingLevelAdvantage': 0, 'takedownsBeforeJungleMinionSpawn': 1, 'takedownsFirstXMinutes': 5, 'takedownsInAlcove': 0, 'takedownsInEnemyFountain': 0, 'teamBaronKills': 1, 'teamDamagePercentage': 0.23316624560216415, 'teamElderDragonKills': 0, 'teamRiftHeraldKills': 1, 'tookLargeDamageSurvived': 0, 'turretPlatesTaken': 1, 'turretTakedowns': 3, 'turretsTakenWithRiftHerald': 0, 'twentyMinionsIn3SecondsCount': 0, 'twoWardsOneSweeperCount': 0, 'unseenRecalls': 0, 'visionScoreAdvantageLaneOpponent': -0.02784043550491333, 'visionScorePerMinute': 1.2208970005495394, 'wardTakedowns': 4, 'wardTakedownsBefore20M': 3, 'wardsGuarded': 1}, 'champExperience': 21409, 'champLevel': 18, 'championId': 84, 'championName': 'Akali', 'championTransform': 0, 'commandPings': 14, 'consumablesPurchased': 12, 'damageDealtToBuildings': 4590, 'damageDealtToObjectives': 4590, 'damageDealtToTurrets': 4590, 'damageSelfMitigated': 30543, 'dangerPings': 0, 'deaths': 6, 'detectorWardsPlaced': 3, 'doubleKills': 2, 'dragonKills': 0, 'eligibleForProgression': True, 'enemyMissingPings': 7, 'enemyVisionPings': 0, 'firstBloodAssist': True, 'firstBloodKill': False, 'firstTowerAssist': False, 'firstTowerKill': False, 'gameEndedInEarlySurrender': False, 'gameEndedInSurrender': True, 'getBackPings': 7, 'goldEarned': 17089, 'goldSpent': 16975, 'holdPings': 0, 'individualPosition': 'MIDDLE', 'inhibitorKills': 2, 'inhibitorTakedowns': 2, 'inhibitorsLost': 0, 'item0': 3157, 'item1': 3102, 'item2': 3152, 'item3': 3020, 'item4': 4645, 'item5': 3100, 'item6': 3340, 'itemsPurchased': 32, 'killingSprees': 4, 'kills': 17, 'lane': 'MIDDLE', 'largestCriticalStrike': 0, 'largestKillingSpree': 7, 'largestMultiKill': 2, 'longestTimeSpentLiving': 955, 'magicDamageDealt': 160128, 'magicDamageDealtToChampions': 32200, 'magicDamageTaken': 17706, 'missions': {'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0}, 'needVisionPings': 0, 'neutralMinionsKilled': 2, 'nexusKills': 0, 'nexusLost': 0, 'nexusTakedowns': 0, 'objectivesStolen': 0, 'objectivesStolenAssists': 0, 'onMyWayPings': 4, 'participantId': 8, 'pentaKills': 0, 'perks': {'statPerks': {'defense': 5002, 'flex': 5008, 'offense': 5008}, 'styles': [{'description': 'primaryStyle', 'selections': [{'perk': 8112, 'var1': 2415, 'var2': 0, 'var3': 0}, {'perk': 8139, 'var1': 931, 'var2': 0, 'var3': 0}, {'perk': 8138, 'var1': 30, 'var2': 0, 'var3': 0}, {'perk': 8105, 'var1': 13, 'var2': 5, 'var3': 0}], 'style': 8100}, {'description': 'subStyle', 'selections': [{'perk': 8473, 'var1': 1538, 'var2': 0, 'var3': 0}, {'perk': 8242, 'var1': 113, 'var2': 0, 'var3': 0}], 'style': 8400}]}, 'physicalDamageDealt': 24293, 'physicalDamageDealtToChampions': 1765, 'physicalDamageTaken': 21861, 'placement': 0, 'playerAugment1': 0, 'playerAugment2': 0, 'playerAugment3': 0, 'playerAugment4': 0, 'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0, 'playerSubteamId': 0, 'profileIcon': 4793, 'pushPings': 0, 'puuid': 'KyRnbjcRmlrkQ7fShSbsBPJUhUkVTZ2GNZrM8wJSeqm0ld97UrrEKnE1UIT-pDsOn0RDqM83ukk_hg', 'quadraKills': 0, 'riotIdName': '', 'riotIdTagline': '', 'role': 'SOLO', 'sightWardsBoughtInGame': 0, 'spell1Casts': 187, 'spell2Casts': 32, 'spell3Casts': 79, 'spell4Casts': 28, 'subteamPlacement': 0, 'summoner1Casts': 4, 'summoner1Id': 4, 'summoner2Casts': 7, 'summoner2Id': 14, 'summonerId': '_k-0mrfL0YiQzqjJDcbT-0iIE0BXGYSob-QY-V7_HP3-yketKvMhcjp6iQ', 'summonerLevel': 173, 'summonerName': 'Isabellauwu', 'teamEarlySurrendered': False, 'teamId': 200, 'teamPosition': 'MIDDLE', 'timeCCingOthers': 1, 'timePlayed': 2158, 'totalAllyJungleMinionsKilled': 0, 'totalDamageDealt': 186786, 'totalDamageDealtToChampions': 35509, 'totalDamageShieldedOnTeammates': 0, 'totalDamageTaken': 39977, 'totalEnemyJungleMinionsKilled': 2, 'totalHeal': 8509, 'totalHealsOnTeammates': 0, 'totalMinionsKilled': 196, 'totalTimeCCDealt': 290, 'totalTimeSpentDead': 262, 'totalUnitsHealed': 1, 'tripleKills': 0, 'trueDamageDealt': 2364, 'trueDamageDealtToChampions': 1543, 'trueDamageTaken': 410, 'turretKills': 3, 'turretTakedowns': 3, 'turretsLost': 4, 'unrealKills': 0, 'visionClearedPings': 0, 'visionScore': 43, 'visionWardsBoughtInGame': 3, 'wardsKilled': 4, 'wardsPlaced': 16, 'win': True}, {'allInPings': 0, 'assistMePings': 0, 'assists': 11, 'baitPings': 0, 'baronKills': 0, 'basicPings': 0, 'bountyLevel': 4, 'challenges': {'12AssistStreakCount': 0, 'abilityUses': 241, 'acesBefore15Minutes': 0, 'alliedJungleMonsterKills': 0, 'baronBuffGoldAdvantageOverThreshold': 1, 'baronTakedowns': 1, 'blastConeOppositeOpponentCount': 0, 'bountyGold': 0, 'buffsStolen': 0, 'completeSupportQuestInTime': 0, 'controlWardTimeCoverageInRiverOrEnemyHalf': 0.4617468263497337, 'controlWardsPlaced': 6, 'damagePerMinute': 789.782465445664, 'damageTakenOnTeamPercentage': 0.16153639182715482, 'dancedWithRiftHerald': 0, 'deathsByEnemyChamps': 7, 'dodgeSkillShotsSmallWindow': 1, 'doubleAces': 0, 'dragonTakedowns': 2, 'earliestBaron': 2074.9830690999997, 'earliestDragonTakedown': 1138.4310306999998, 'earlyLaningPhaseGoldExpAdvantage': 0, 'effectiveHealAndShielding': 0, 'elderDragonKillsWithOpposingSoul': 0, 'elderDragonMultikills': 0, 'enemyChampionImmobilizations': 13, 'enemyJungleMonsterKills': 0, 'epicMonsterKillsNearEnemyJungler': 0, 'epicMonsterKillsWithin30SecondsOfSpawn': 0, 'epicMonsterSteals': 0, 'epicMonsterStolenWithoutSmite': 0, 'firstTurretKilled': 1, 'firstTurretKilledTime': 947.9954345, 'flawlessAces': 1, 'fullTeamTakedown': 1, 'gameLength': 2158.4759347, 'getTakedownsInAllLanesEarlyJungleAsLaner': 0, 'goldPerMinute': 437.03229498241296, 'hadOpenNexus': 0, 'immobilizeAndKillWithAlly': 5, 'initialBuffCount': 0, 'initialCrabCount': 0, 'jungleCsBefore10Minutes': 0, 'junglerTakedownsNearDamagedEpicMonster': 0, 'kTurretsDestroyedBeforePlatesFall': 0, 'kda': 3.142857142857143, 'killAfterHiddenWithAlly': 5, 'killParticipation': 0.4074074074074074, 'killedChampTookFullTeamDamageSurvived': 0, 'killingSprees': 1, 'killsNearEnemyTurret': 3, 'killsOnOtherLanesEarlyJungleAsLaner': 0, 'killsOnRecentlyHealedByAramPack': 0, 'killsUnderOwnTurret': 2, 'killsWithHelpFromEpicMonster': 0, 'knockEnemyIntoTeamAndKill': 0, 'landSkillShotsEarlyGame': 0, 'laneMinionsFirst10Minutes': 66, 'laningPhaseGoldExpAdvantage': 0, 'legendaryCount': 0, 'lostAnInhibitor': 0, 'maxCsAdvantageOnLaneOpponent': 28, 'maxKillDeficit': 0, 'maxLevelLeadLaneOpponent': 1, 'mejaisFullStackInTime': 0, 'moreEnemyJungleThanOpponent': 0, 'multiKillOneSpell': 0, 'multiTurretRiftHeraldCount': 0, 'multikills': 0, 'multikillsAfterAggressiveFlash': 0, 'mythicItemUsed': 6671, 'outerTurretExecutesBefore10Minutes': 0, 'outnumberedKills': 0, 'outnumberedNexusKill': 0, 'perfectDragonSoulsTaken': 0, 'perfectGame': 0, 'pickKillWithAlly': 21, 'playedChampSelectPosition': 1, 'poroExplosions': 0, 'quickCleanse': 0, 'quickFirstTurret': 0, 'quickSoloKills': 0, 'riftHeraldTakedowns': 0, 'saveAllyFromDeath': 0, 'scuttleCrabKills': 0, 'shortestTimeToAceFromFirstTakedown': 56.705361999999695, 'skillshotsDodged': 49, 'skillshotsHit': 6, 'snowballsHit': 0, 'soloBaronKills': 0, 'soloKills': 0, 'stealthWardsPlaced': 5, 'survivedSingleDigitHpCount': 0, 'survivedThreeImmobilizesInFight': 0, 'takedownOnFirstTurret': 0, 'takedowns': 22, 'takedownsAfterGainingLevelAdvantage': 0, 'takedownsBeforeJungleMinionSpawn': 0, 'takedownsFirstXMinutes': 4, 'takedownsInAlcove': 0, 'takedownsInEnemyFountain': 0, 'teamBaronKills': 1, 'teamDamagePercentage': 0.18656454329517355, 'teamElderDragonKills': 0, 'teamRiftHeraldKills': 1, 'tookLargeDamageSurvived': 0, 'turretPlatesTaken': 0, 'turretTakedowns': 1, 'turretsTakenWithRiftHerald': 0, 'twentyMinionsIn3SecondsCount': 0, 'twoWardsOneSweeperCount': 0, 'unseenRecalls': 0, 'visionScoreAdvantageLaneOpponent': 1.4060091972351074, 'visionScorePerMinute': 0.7616802674476811, 'wardTakedowns': 4, 'wardTakedownsBefore20M': 2, 'wardsGuarded': 0}, 'champExperience': 20631, 'champLevel': 18, 'championId': 202, 'championName': 'Jhin', 'championTransform': 0, 'commandPings': 1, 'consumablesPurchased': 11, 'damageDealtToBuildings': 3838, 'damageDealtToObjectives': 19964, 'damageDealtToTurrets': 3838, 'damageSelfMitigated': 19635, 'dangerPings': 0, 'deaths': 7, 'detectorWardsPlaced': 6, 'doubleKills': 0, 'dragonKills': 0, 'eligibleForProgression': True, 'enemyMissingPings': 3, 'enemyVisionPings': 4, 'firstBloodAssist': False, 'firstBloodKill': False, 'firstTowerAssist': False, 'firstTowerKill': False, 'gameEndedInEarlySurrender': False, 'gameEndedInSurrender': True, 'getBackPings': 2, 'goldEarned': 15722, 'goldSpent': 13000, 'holdPings': 0, 'individualPosition': 'BOTTOM', 'inhibitorKills': 0, 'inhibitorTakedowns': 0, 'inhibitorsLost': 0, 'item0': 3009, 'item1': 7006, 'item2': 2421, 'item3': 3036, 'item4': 1031, 'item5': 3095, 'item6': 3363, 'itemsPurchased': 34, 'killingSprees': 3, 'kills': 11, 'lane': 'BOTTOM', 'largestCriticalStrike': 1756, 'largestKillingSpree': 4, 'largestMultiKill': 1, 'longestTimeSpentLiving': 757, 'magicDamageDealt': 23158, 'magicDamageDealtToChampions': 1942, 'magicDamageTaken': 11689, 'missions': {'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0}, 'needVisionPings': 0, 'neutralMinionsKilled': 0, 'nexusKills': 0, 'nexusLost': 0, 'nexusTakedowns': 1, 'objectivesStolen': 0, 'objectivesStolenAssists': 0, 'onMyWayPings': 1, 'participantId': 9, 'pentaKills': 0, 'perks': {'statPerks': {'defense': 5002, 'flex': 5008, 'offense': 5008}, 'styles': [{'description': 'primaryStyle', 'selections': [{'perk': 8021, 'var1': 2429, 'var2': 2701, 'var3': 0}, {'perk': 8009, 'var1': 2242, 'var2': 0, 'var3': 0}, {'perk': 9103, 'var1': 22, 'var2': 0, 'var3': 0}, {'perk': 8014, 'var1': 903, 'var2': 0, 'var3': 0}], 'style': 8000}, {'description': 'subStyle', 'selections': [{'perk': 8234, 'var1': 13607, 'var2': 0, 'var3': 0}, {'perk': 8236, 'var1': 28, 'var2': 0, 'var3': 0}], 'style': 8200}]}, 'physicalDamageDealt': 220826, 'physicalDamageDealtToChampions': 25016, 'physicalDamageTaken': 21014, 'placement': 0, 'playerAugment1': 0, 'playerAugment2': 0, 'playerAugment3': 0, 'playerAugment4': 0, 'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0, 'playerSubteamId': 0, 'profileIcon': 6295, 'pushPings': 0, 'puuid': 'yFM3imQ-mYn4XTiMFpOx8CQouAiGQTec1u24Ou7I-6kX8_oMBJTfPUG1MPbm_gI16oAeq-pWNnLgZQ', 'quadraKills': 0, 'riotIdName': '', 'riotIdTagline': '', 'role': 'CARRY', 'sightWardsBoughtInGame': 0, 'spell1Casts': 113, 'spell2Casts': 43, 'spell3Casts': 57, 'spell4Casts': 28, 'subteamPlacement': 0, 'summoner1Casts': 4, 'summoner1Id': 4, 'summoner2Casts': 5, 'summoner2Id': 3, 'summonerId': 'oCpSdISbpqbtJSW4kZolGTIBRcgSE2Pakk05h7cAluGhoTY', 'summonerLevel': 111, 'summonerName': 'DDDemacia', 'teamEarlySurrendered': False, 'teamId': 200, 'teamPosition': 'BOTTOM', 'timeCCingOthers': 24, 'timePlayed': 2158, 'totalAllyJungleMinionsKilled': 0, 'totalDamageDealt': 252424, 'totalDamageDealtToChampions': 28412, 'totalDamageShieldedOnTeammates': 0, 'totalDamageTaken': 33001, 'totalEnemyJungleMinionsKilled': 0, 'totalHeal': 10560, 'totalHealsOnTeammates': 0, 'totalMinionsKilled': 245, 'totalTimeCCDealt': 246, 'totalTimeSpentDead': 251, 'totalUnitsHealed': 1, 'tripleKills': 0, 'trueDamageDealt': 8440, 'trueDamageDealtToChampions': 1453, 'trueDamageTaken': 298, 'turretKills': 1, 'turretTakedowns': 1, 'turretsLost': 4, 'unrealKills': 0, 'visionClearedPings': 0, 'visionScore': 27, 'visionWardsBoughtInGame': 6, 'wardsKilled': 4, 'wardsPlaced': 12, 'win': True}, {'allInPings': 0, 'assistMePings': 0, 'assists': 24, 'baitPings': 0, 'baronKills': 0, 'basicPings': 0, 'bountyLevel': 1, 'challenges': {'12AssistStreakCount': 0, 'abilityUses': 172, 'acesBefore15Minutes': 0, 'alliedJungleMonsterKills': 0, 'baronBuffGoldAdvantageOverThreshold': 1, 'baronTakedowns': 1, 'blastConeOppositeOpponentCount': 0, 'bountyGold': 300, 'buffsStolen': 0, 'completeSupportQuestInTime': 1, 'controlWardTimeCoverageInRiverOrEnemyHalf': 0.9519530319829976, 'controlWardsPlaced': 4, 'damagePerMinute': 915.2157502045834, 'damageTakenOnTeamPercentage': 0.14178924859869352, 'dancedWithRiftHerald': 0, 'deathsByEnemyChamps': 9, 'dodgeSkillShotsSmallWindow': 4, 'doubleAces': 0, 'dragonTakedowns': 2, 'earliestBaron': 2074.9830690999997, 'earliestDragonTakedown': 790.9937994999999, 'earlyLaningPhaseGoldExpAdvantage': 0, 'effectiveHealAndShielding': 9815.6416015625, 'elderDragonKillsWithOpposingSoul': 0, 'elderDragonMultikills': 0, 'enemyChampionImmobilizations': 13, 'enemyJungleMonsterKills': 1, 'epicMonsterKillsNearEnemyJungler': 0, 'epicMonsterKillsWithin30SecondsOfSpawn': 0, 'epicMonsterSteals': 0, 'epicMonsterStolenWithoutSmite': 0, 'fasterSupportQuestCompletion': 1, 'firstTurretKilled': 1, 'firstTurretKilledTime': 947.9954345, 'flawlessAces': 1, 'fullTeamTakedown': 1, 'gameLength': 2158.4759347, 'getTakedownsInAllLanesEarlyJungleAsLaner': 0, 'goldPerMinute': 379.11716331052594, 'hadOpenNexus': 0, 'highestWardKills': 1, 'immobilizeAndKillWithAlly': 4, 'initialBuffCount': 0, 'initialCrabCount': 0, 'jungleCsBefore10Minutes': 0, 'junglerTakedownsNearDamagedEpicMonster': 0, 'kTurretsDestroyedBeforePlatesFall': 0, 'kda': 3.4444444444444446, 'killAfterHiddenWithAlly': 7, 'killParticipation': 0.5740740740740741, 'killedChampTookFullTeamDamageSurvived': 0, 'killingSprees': 0, 'killsNearEnemyTurret': 2, 'killsOnOtherLanesEarlyJungleAsLaner': 2, 'killsOnRecentlyHealedByAramPack': 0, 'killsUnderOwnTurret': 2, 'killsWithHelpFromEpicMonster': 0, 'knockEnemyIntoTeamAndKill': 0, 'landSkillShotsEarlyGame': 4, 'laneMinionsFirst10Minutes': 7, 'laningPhaseGoldExpAdvantage': 0, 'legendaryCount': 0, 'lostAnInhibitor': 0, 'maxCsAdvantageOnLaneOpponent': 11.5, 'maxKillDeficit': 0, 'maxLevelLeadLaneOpponent': 3, 'mejaisFullStackInTime': 0, 'moreEnemyJungleThanOpponent': 0, 'multiKillOneSpell': 0, 'multiTurretRiftHeraldCount': 0, 'multikills': 0, 'multikillsAfterAggressiveFlash': 0, 'mythicItemUsed': 6692, 'outerTurretExecutesBefore10Minutes': 0, 'outnumberedKills': 2, 'outnumberedNexusKill': 0, 'perfectDragonSoulsTaken': 0, 'perfectGame': 0, 'pickKillWithAlly': 26, 'playedChampSelectPosition': 1, 'poroExplosions': 0, 'quickCleanse': 0, 'quickFirstTurret': 0, 'quickSoloKills': 0, 'riftHeraldTakedowns': 0, 'saveAllyFromDeath': 3, 'scuttleCrabKills': 0, 'shortestTimeToAceFromFirstTakedown': 42.70709119999992, 'skillshotsDodged': 57, 'skillshotsHit': 23, 'snowballsHit': 0, 'soloBaronKills': 0, 'soloKills': 1, 'stealthWardsPlaced': 31, 'survivedSingleDigitHpCount': 0, 'survivedThreeImmobilizesInFight': 0, 'takedownOnFirstTurret': 0, 'takedowns': 31, 'takedownsAfterGainingLevelAdvantage': 0, 'takedownsBeforeJungleMinionSpawn': 1, 'takedownsFirstXMinutes': 7, 'takedownsInAlcove': 0, 'takedownsInEnemyFountain': 0, 'teamBaronKills': 1, 'teamDamagePercentage': 0.21619473199764894, 'teamElderDragonKills': 0, 'teamRiftHeraldKills': 1, 'tookLargeDamageSurvived': 0, 'turretPlatesTaken': 0, 'turretTakedowns': 3, 'turretsTakenWithRiftHerald': 0, 'twentyMinionsIn3SecondsCount': 0, 'twoWardsOneSweeperCount': 0, 'unseenRecalls': 0, 'visionScoreAdvantageLaneOpponent': 0.018885374069213867, 'visionScorePerMinute': 2.7658383058104525, 'wardTakedowns': 14, 'wardTakedownsBefore20M': 9, 'wardsGuarded': 0}, 'champExperience': 17124, 'champLevel': 17, 'championId': 235, 'championName': 'Senna', 'championTransform': 0, 'commandPings': 11, 'consumablesPurchased': 7, 'damageDealtToBuildings': 5734, 'damageDealtToObjectives': 14049, 'damageDealtToTurrets': 5734, 'damageSelfMitigated': 17385, 'dangerPings': 0, 'deaths': 9, 'detectorWardsPlaced': 4, 'doubleKills': 0, 'dragonKills': 0, 'eligibleForProgression': True, 'enemyMissingPings': 14, 'enemyVisionPings': 1, 'firstBloodAssist': False, 'firstBloodKill': True, 'firstTowerAssist': False, 'firstTowerKill': False, 'gameEndedInEarlySurrender': False, 'gameEndedInSurrender': True, 'getBackPings': 0, 'goldEarned': 13638, 'goldSpent': 11225, 'holdPings': 0, 'individualPosition': 'UTILITY', 'inhibitorKills': 0, 'inhibitorTakedowns': 0, 'inhibitorsLost': 0, 'item0': 3864, 'item1': 7001, 'item2': 3009, 'item3': 3033, 'item4': 3094, 'item5': 1036, 'item6': 3364, 'itemsPurchased': 27, 'killingSprees': 1, 'kills': 7, 'lane': 'BOTTOM', 'largestCriticalStrike': 625, 'largestKillingSpree': 2, 'largestMultiKill': 1, 'longestTimeSpentLiving': 754, 'magicDamageDealt': 1542, 'magicDamageDealtToChampions': 833, 'magicDamageTaken': 11637, 'missions': {'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0}, 'needVisionPings': 1, 'neutralMinionsKilled': 0, 'nexusKills': 1, 'nexusLost': 0, 'nexusTakedowns': 1, 'objectivesStolen': 0, 'objectivesStolenAssists': 0, 'onMyWayPings': 18, 'participantId': 10, 'pentaKills': 0, 'perks': {'statPerks': {'defense': 5002, 'flex': 5008, 'offense': 5005}, 'styles': [{'description': 'primaryStyle', 'selections': [{'perk': 8021, 'var1': 2316, 'var2': 2239, 'var3': 0}, {'perk': 8009, 'var1': 3391, 'var2': 0, 'var3': 0}, {'perk': 9104, 'var1': 15, 'var2': 40, 'var3': 0}, {'perk': 8017, 'var1': 2802, 'var2': 0, 'var3': 0}], 'style': 8000}, {'description': 'subStyle', 'selections': [{'perk': 8410, 'var1': 89, 'var2': 0, 'var3': 0}, {'perk': 8345, 'var1': 3, 'var2': 0, 'var3': 0}], 'style': 8300}]}, 'physicalDamageDealt': 79455, 'physicalDamageDealtToChampions': 30717, 'physicalDamageTaken': 16879, 'placement': 0, 'playerAugment1': 0, 'playerAugment2': 0, 'playerAugment3': 0, 'playerAugment4': 0, 'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0, 'playerSubteamId': 0, 'profileIcon': 5735, 'pushPings': 1, 'puuid': '1JzHac7vWznlDGVAlQMtHfJMJr39gI0_akJzwWBC9o2AJIp4GGbAbhhqMJYV-zgfCTE7pty72aAEwg', 'quadraKills': 0, 'riotIdName': '', 'riotIdTagline': '', 'role': 'SUPPORT', 'sightWardsBoughtInGame': 0, 'spell1Casts': 86, 'spell2Casts': 45, 'spell3Casts': 30, 'spell4Casts': 11, 'subteamPlacement': 0, 'summoner1Casts': 5, 'summoner1Id': 4, 'summoner2Casts': 6, 'summoner2Id': 7, 'summonerId': '3VKGbZg5IiCKeauhA81v53EIkgj_84pgHWpEzz63fGE4mUjkkzj5Z5YgDw', 'summonerLevel': 32, 'summonerName': 'onepunchvi69x', 'teamEarlySurrendered': False, 'teamId': 200, 'teamPosition': 'UTILITY', 'timeCCingOthers': 27, 'timePlayed': 2158, 'totalAllyJungleMinionsKilled': 0, 'totalDamageDealt': 83018, 'totalDamageDealtToChampions': 32924, 'totalDamageShieldedOnTeammates': 3396, 'totalDamageTaken': 28967, 'totalEnemyJungleMinionsKilled': 0, 'totalHeal': 19265, 'totalHealsOnTeammates': 6419, 'totalMinionsKilled': 44, 'totalTimeCCDealt': 263, 'totalTimeSpentDead': 315, 'totalUnitsHealed': 5, 'tripleKills': 0, 'trueDamageDealt': 2020, 'trueDamageDealtToChampions': 1373, 'trueDamageTaken': 450, 'turretKills': 2, 'turretTakedowns': 3, 'turretsLost': 4, 'unrealKills': 0, 'visionClearedPings': 0, 'visionScore': 99, 'visionWardsBoughtInGame': 5, 'wardsKilled': 14, 'wardsPlaced': 35, 'win': True}], 'platformId': 'NA1', 'queueId': 420, 'teams': [{'bans': [{'championId': 555, 'pickTurn': 1}, {'championId': 157, 'pickTurn': 2}, {'championId': -1, 'pickTurn': 3}, {'championId': 518, 'pickTurn': 4}, {'championId': 36, 'pickTurn': 5}], 'objectives': {'baron': {'first': True, 'kills': 1}, 'champion': {'first': False, 'kills': 30}, 'dragon': {'first': True, 'kills': 1}, 'horde': {'first': False, 'kills': 0}, 'inhibitor': {'first': False, 'kills': 0}, 'riftHerald': {'first': False, 'kills': 0}, 'tower': {'first': False, 'kills': 4}}, 'teamId': 100, 'win': False}, {'bans': [{'championId': 40, 'pickTurn': 6}, {'championId': 53, 'pickTurn': 7}, {'championId': 517, 'pickTurn': 8}, {'championId': 420, 'pickTurn': 9}, {'championId': 777, 'pickTurn': 10}], 'objectives': {'baron': {'first': False, 'kills': 1}, 'champion': {'first': True, 'kills': 54}, 'dragon': {'first': False, 'kills': 4}, 'horde': {'first': False, 'kills': 0}, 'inhibitor': {'first': True, 'kills': 2}, 'riftHerald': {'first': True, 'kills': 1}, 'tower': {'first': True, 'kills': 10}}, 'teamId': 200, 'win': True}], 'tournamentCode': ''}}
Match data: {'metadata': {'dataVersion': '2', 'matchId': 'NA1_4835192603', 'participants': ['yBm2l04adqR8oeG8WCcNZOflvVp9Q4ZcGSm-wXoGZhSUNstQih8jHK-uFw1hKK14VxzVxNtQnx11FA', 'PY7iMB_4ohr2vmtOB-GaUAFMb6owPm9ek4DJUBYt4IoEhxIvz3RrwACfnRDNv8uicE4HxaW-TiAtzg', 'vr_-Loy-Fc_iHTYyWWa383FyLCw6c0iKLnVREVa6Vtab4bAqMcE9XLbFaxnsgqwT5GTjELIuhm7Eyw', '34wqa3OKa6UWHSPIvs3i0BKlIYmR-dmz3C7uLs9o58K6O-ZnRjoBRJ4km59gerGJ546Fcr7S0ysaxw', 'IwYjkxE9AlLTMZ_9Mf6YVpyGcQYOeQq2azNWgQhU2BXEm0vZn6Bvrxj7jNIEr-iVQAAxAqQFa4jDHg', 'bHhAY7DSUICKSCwWzf2EeKhtFsUq4fEOXZM4BsXK4CIXbAW0z-N2I-2yls25q9G__fugkZVhF7h_Pw', 'th3FWRsIG8MtnJZg-BaRLYqcMc5fggO4dl7z6lBCtR8fGtSmncvKqX2-G-Hm81htOfAsgqs5V393Dw', 'KyRnbjcRmlrkQ7fShSbsBPJUhUkVTZ2GNZrM8wJSeqm0ld97UrrEKnE1UIT-pDsOn0RDqM83ukk_hg', 'yFM3imQ-mYn4XTiMFpOx8CQouAiGQTec1u24Ou7I-6kX8_oMBJTfPUG1MPbm_gI16oAeq-pWNnLgZQ', '1JzHac7vWznlDGVAlQMtHfJMJr39gI0_akJzwWBC9o2AJIp4GGbAbhhqMJYV-zgfCTE7pty72aAEwg']}, 'info': {'gameCreation': 1700458100222, 'gameDuration': 2158, 'gameEndTimestamp': 1700460278905, 'gameId': 4835192603, 'gameMode': 'CLASSIC', 'gameName': 'teambuilder-match-4835192603', 'gameStartTimestamp': 1700458120677, 'gameType': 'MATCHED_GAME', 'gameVersion': '13.22.541.9804', 'mapId': 11, 'participants': [{'allInPings': 0, 'assistMePings': 3, 'assists': 6, 'baitPings': 0, 'baronKills': 1, 'basicPings': 0, 'bountyLevel': 0, 'challenges': {'12AssistStreakCount': 0, 'abilityUses': 289, 'acesBefore15Minutes': 0, 'alliedJungleMonsterKills': 99, 'baronTakedowns': 1, 'blastConeOppositeOpponentCount': 0, 'bountyGold': 975, 'buffsStolen': 0, 'completeSupportQuestInTime': 0, 'controlWardTimeCoverageInRiverOrEnemyHalf': 0.797709301141369, 'controlWardsPlaced': 6, 'damagePerMinute': 1329.491672858909, 'damageTakenOnTeamPercentage': 0.28612535073262485, 'dancedWithRiftHerald': 0, 'deathsByEnemyChamps': 10, 'dodgeSkillShotsSmallWindow': 20, 'doubleAces': 0, 'dragonTakedowns': 1, 'earliestBaron': 1657.7682029, 'earliestDragonTakedown': 469.07374519999996, 'earlyLaningPhaseGoldExpAdvantage': 0, 'effectiveHealAndShielding': 0, 'elderDragonKillsWithOpposingSoul': 0, 'elderDragonMultikills': 0, 'enemyChampionImmobilizations': 21, 'enemyJungleMonsterKills': 0, 'epicMonsterKillsNearEnemyJungler': 0, 'epicMonsterKillsWithin30SecondsOfSpawn': 0, 'epicMonsterSteals': 0, 'epicMonsterStolenWithoutSmite': 0, 'firstTurretKilled': 0, 'flawlessAces': 0, 'fullTeamTakedown': 0, 'gameLength': 2158.4759347, 'goldPerMinute': 494.45858346960796, 'hadOpenNexus': 0, 'highestChampionDamage': 1, 'immobilizeAndKillWithAlly': 6, 'initialBuffCount': 2, 'initialCrabCount': 2, 'jungleCsBefore10Minutes': 74.00000008940697, 'junglerKillsEarlyJungle': 0, 'junglerTakedownsNearDamagedEpicMonster': 0, 'kTurretsDestroyedBeforePlatesFall': 0, 'kda': 2.2, 'killAfterHiddenWithAlly': 2, 'killParticipation': 0.7333333333333333, 'killedChampTookFullTeamDamageSurvived': 0, 'killingSprees': 3, 'killsNearEnemyTurret': 5, 'killsOnLanersEarlyJungleAsJungler': 0, 'killsOnRecentlyHealedByAramPack': 0, 'killsUnderOwnTurret': 1, 'killsWithHelpFromEpicMonster': 1, 'knockEnemyIntoTeamAndKill': 0, 'landSkillShotsEarlyGame': 1, 'laneMinionsFirst10Minutes': 0, 'laningPhaseGoldExpAdvantage': 0, 'legendaryCount': 0, 'lostAnInhibitor': 0, 'maxCsAdvantageOnLaneOpponent': 58.750000059604645, 'maxKillDeficit': 0, 'maxLevelLeadLaneOpponent': 2, 'mejaisFullStackInTime': 0, 'moreEnemyJungleThanOpponent': -53.00000008940697, 'multiKillOneSpell': 0, 'multiTurretRiftHeraldCount': 0, 'multikills': 3, 'multikillsAfterAggressiveFlash': 1, 'mythicItemUsed': 6631, 'outerTurretExecutesBefore10Minutes': 0, 'outnumberedKills': 6, 'outnumberedNexusKill': 0, 'perfectDragonSoulsTaken': 0, 'perfectGame': 0, 'pickKillWithAlly': 14, 'poroExplosions': 0, 'quickCleanse': 0, 'quickFirstTurret': 0, 'quickSoloKills': 0, 'riftHeraldTakedowns': 0, 'saveAllyFromDeath': 0, 'scuttleCrabKills': 3, 'skillshotsDodged': 130, 'skillshotsHit': 50, 'snowballsHit': 0, 'soloBaronKills': 0, 'soloKills': 6, 'stealthWardsPlaced': 2, 'survivedSingleDigitHpCount': 0, 'survivedThreeImmobilizesInFight': 8, 'takedownOnFirstTurret': 0, 'takedowns': 22, 'takedownsAfterGainingLevelAdvantage': 0, 'takedownsBeforeJungleMinionSpawn': 0, 'takedownsFirstXMinutes': 5, 'takedownsInAlcove': 0, 'takedownsInEnemyFountain': 0, 'teamBaronKills': 1, 'teamDamagePercentage': 0.3034331241406508, 'teamElderDragonKills': 0, 'teamRiftHeraldKills': 0, 'tookLargeDamageSurvived': 0, 'turretPlatesTaken': 0, 'turretTakedowns': 0, 'turretsTakenWithRiftHerald': 0, 'twentyMinionsIn3SecondsCount': 0, 'twoWardsOneSweeperCount': 0, 'unseenRecalls': 0, 'visionScoreAdvantageLaneOpponent': -0.11555755138397217, 'visionScorePerMinute': 0.9264663167538986, 'wardTakedowns': 3, 'wardTakedownsBefore20M': 1, 'wardsGuarded': 1}, 'champExperience': 19539, 'champLevel': 18, 'championId': 56, 'championName': 'Nocturne', 'championTransform': 0, 'commandPings': 20, 'consumablesPurchased': 7, 'damageDealtToBuildings': 0, 'damageDealtToObjectives': 27275, 'damageDealtToTurrets': 0, 'damageSelfMitigated': 72737, 'dangerPings': 0, 'deaths': 10, 'detectorWardsPlaced': 6, 'doubleKills': 3, 'dragonKills': 1, 'eligibleForProgression': True, 'enemyMissingPings': 12, 'enemyVisionPings': 3, 'firstBloodAssist': False, 'firstBloodKill': False, 'firstTowerAssist': False, 'firstTowerKill': False, 'gameEndedInEarlySurrender': False, 'gameEndedInSurrender': True, 'getBackPings': 3, 'goldEarned': 17787, 'goldSpent': 17650, 'holdPings': 0, 'individualPosition': 'JUNGLE', 'inhibitorKills': 0, 'inhibitorTakedowns': 0, 'inhibitorsLost': 2, 'item0': 6631, 'item1': 3111, 'item2': 3156, 'item3': 6696, 'item4': 6333, 'item5': 6694, 'item6': 3364, 'itemsPurchased': 28, 'killingSprees': 6, 'kills': 16, 'lane': 'JUNGLE', 'largestCriticalStrike': 0, 'largestKillingSpree': 3, 'largestMultiKill': 2, 'longestTimeSpentLiving': 790, 'magicDamageDealt': 7835, 'magicDamageDealtToChampions': 2969, 'magicDamageTaken': 14131, 'missions': {'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0}, 'needVisionPings': 0, 'neutralMinionsKilled': 171, 'nexusKills': 0, 'nexusLost': 1, 'nexusTakedowns': 0, 'objectivesStolen': 0, 'objectivesStolenAssists': 0, 'onMyWayPings': 13, 'participantId': 1, 'pentaKills': 0, 'perks': {'statPerks': {'defense': 5003, 'flex': 5008, 'offense': 5005}, 'styles': [{'description': 'primaryStyle', 'selections': [{'perk': 8008, 'var1': 116, 'var2': 33, 'var3': 0}, {'perk': 9111, 'var1': 2039, 'var2': 440, 'var3': 0}, {'perk': 9104, 'var1': 13, 'var2': 20, 'var3': 0}, {'perk': 8014, 'var1': 1227, 'var2': 0, 'var3': 0}], 'style': 8000}, {'description': 'subStyle', 'selections': [{'perk': 8138, 'var1': 18, 'var2': 0, 'var3': 0}, {'perk': 8106, 'var1': 5, 'var2': 0, 'var3': 0}], 'style': 8100}]}, 'physicalDamageDealt': 266332, 'physicalDamageDealtToChampions': 43491, 'physicalDamageTaken': 30914, 'placement': 0, 'playerAugment1': 0, 'playerAugment2': 0, 'playerAugment3': 0, 'playerAugment4': 0, 'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0, 'playerSubteamId': 0, 'profileIcon': 5675, 'pushPings': 0, 'puuid': 'yBm2l04adqR8oeG8WCcNZOflvVp9Q4ZcGSm-wXoGZhSUNstQih8jHK-uFw1hKK14VxzVxNtQnx11FA', 'quadraKills': 0, 'riotIdName': '', 'riotIdTagline': '', 'role': 'NONE', 'sightWardsBoughtInGame': 0, 'spell1Casts': 141, 'spell2Casts': 45, 'spell3Casts': 68, 'spell4Casts': 35, 'subteamPlacement': 0, 'summoner1Casts': 4, 'summoner1Id': 4, 'summoner2Casts': 20, 'summoner2Id': 11, 'summonerId': '3acL0Rdj_8MNNSo0lwD4JQL15Ik_KjMI4dJrSdbu0kLxIZQ', 'summonerLevel': 374, 'summonerName': 'Yessss Sirrr ', 'teamEarlySurrendered': False, 'teamId': 100, 'teamPosition': 'JUNGLE', 'timeCCingOthers': 311, 'timePlayed': 2158, 'totalAllyJungleMinionsKilled': 141, 'totalDamageDealt': 318237, 'totalDamageDealtToChampions': 47827, 'totalDamageShieldedOnTeammates': 0, 'totalDamageTaken': 52251, 'totalEnemyJungleMinionsKilled': 0, 'totalHeal': 18751, 'totalHealsOnTeammates': 0, 'totalMinionsKilled': 72, 'totalTimeCCDealt': 1213, 'totalTimeSpentDead': 398, 'totalUnitsHealed': 1, 'tripleKills': 0, 'trueDamageDealt': 44069, 'trueDamageDealtToChampions': 1366, 'trueDamageTaken': 7205, 'turretKills': 0, 'turretTakedowns': 0, 'turretsLost': 10, 'unrealKills': 0, 'visionClearedPings': 0, 'visionScore': 33, 'visionWardsBoughtInGame': 6, 'wardsKilled': 3, 'wardsPlaced': 8, 'win': False}, {'allInPings': 0, 'assistMePings': 9, 'assists': 3, 'baitPings': 0, 'baronKills': 0, 'basicPings': 0, 'bountyLevel': 0, 'challenges': {'12AssistStreakCount': 0, 'abilityUses': 647, 'acesBefore15Minutes': 0, 'alliedJungleMonsterKills': 3, 'baronTakedowns': 1, 'blastConeOppositeOpponentCount': 0, 'bountyGold': 800, 'buffsStolen': 0, 'completeSupportQuestInTime': 0, 'controlWardsPlaced': 0, 'damagePerMinute': 1083.557185778894, 'damageTakenOnTeamPercentage': 0.22598831396423713, 'dancedWithRiftHerald': 0, 'deathsByEnemyChamps': 12, 'dodgeSkillShotsSmallWindow': 11, 'doubleAces': 0, 'dragonTakedowns': 0, 'earliestBaron': 1657.7682029, 'earlyLaningPhaseGoldExpAdvantage': 0, 'effectiveHealAndShielding': 0, 'elderDragonKillsWithOpposingSoul': 0, 'elderDragonMultikills': 0, 'enemyChampionImmobilizations': 12, 'enemyJungleMonsterKills': 0, 'epicMonsterKillsNearEnemyJungler': 0, 'epicMonsterKillsWithin30SecondsOfSpawn': 0, 'epicMonsterSteals': 0, 'epicMonsterStolenWithoutSmite': 0, 'firstTurretKilled': 0, 'flawlessAces': 0, 'fullTeamTakedown': 0, 'gameLength': 2158.4759347, 'getTakedownsInAllLanesEarlyJungleAsLaner': 0, 'goldPerMinute': 349.06166838604037, 'hadOpenNexus': 0, 'immobilizeAndKillWithAlly': 1, 'initialBuffCount': 0, 'initialCrabCount': 0, 'jungleCsBefore10Minutes': 0, 'junglerTakedownsNearDamagedEpicMonster': 0, 'kTurretsDestroyedBeforePlatesFall': 0, 'kda': 0.5833333333333334, 'killAfterHiddenWithAlly': 1, 'killParticipation': 0.23333333333333334, 'killedChampTookFullTeamDamageSurvived': 0, 'killingSprees': 0, 'killsNearEnemyTurret': 0, 'killsOnOtherLanesEarlyJungleAsLaner': 0, 'killsOnRecentlyHealedByAramPack': 0, 'killsUnderOwnTurret': 1, 'killsWithHelpFromEpicMonster': 0, 'knockEnemyIntoTeamAndKill': 0, 'landSkillShotsEarlyGame': 4, 'laneMinionsFirst10Minutes': 72, 'laningPhaseGoldExpAdvantage': 0, 'legendaryCount': 0, 'lostAnInhibitor': 0, 'maxCsAdvantageOnLaneOpponent': 48, 'maxKillDeficit': 0, 'maxLevelLeadLaneOpponent': 1, 'mejaisFullStackInTime': 0, 'moreEnemyJungleThanOpponent': 0, 'multiKillOneSpell': 0, 'multiTurretRiftHeraldCount': 0, 'multikills': 0, 'multikillsAfterAggressiveFlash': 0, 'mythicItemUsed': 6653, 'outerTurretExecutesBefore10Minutes': 0, 'outnumberedKills': 0, 'outnumberedNexusKill': 0, 'perfectDragonSoulsTaken': 0, 'perfectGame': 0, 'pickKillWithAlly': 5, 'poroExplosions': 0, 'quickCleanse': 0, 'quickFirstTurret': 0, 'quickSoloKills': 0, 'riftHeraldTakedowns': 0, 'saveAllyFromDeath': 0, 'scuttleCrabKills': 0, 'skillshotsDodged': 71, 'skillshotsHit': 21, 'snowballsHit': 0, 'soloBaronKills': 0, 'soloKills': 2, 'stealthWardsPlaced': 12, 'survivedSingleDigitHpCount': 0, 'survivedThreeImmobilizesInFight': 6, 'takedownOnFirstTurret': 0, 'takedowns': 7, 'takedownsAfterGainingLevelAdvantage': 0, 'takedownsBeforeJungleMinionSpawn': 0, 'takedownsFirstXMinutes': 2, 'takedownsInAlcove': 1, 'takedownsInEnemyFountain': 0, 'teamBaronKills': 1, 'teamDamagePercentage': 0.24730289687255055, 'teamElderDragonKills': 0, 'teamRiftHeraldKills': 0, 'tookLargeDamageSurvived': 0, 'turretPlatesTaken': 0, 'turretTakedowns': 0, 'turretsTakenWithRiftHerald': 0, 'twentyMinionsIn3SecondsCount': 0, 'twoWardsOneSweeperCount': 0, 'unseenRecalls': 0, 'visionScoreAdvantageLaneOpponent': -0.24054467678070068, 'visionScorePerMinute': 0.6805494736539871, 'wardTakedowns': 2, 'wardTakedownsBefore20M': 2, 'wardsGuarded': 0}, 'champExperience': 16642, 'champLevel': 17, 'championId': 69, 'championName': 'Cassiopeia', 'championTransform': 0, 'commandPings': 0, 'consumablesPurchased': 3, 'damageDealtToBuildings': 0, 'damageDealtToObjectives': 11116, 'damageDealtToTurrets': 0, 'damageSelfMitigated': 23139, 'dangerPings': 0, 'deaths': 12, 'detectorWardsPlaced': 0, 'doubleKills': 0, 'dragonKills': 0, 'eligibleForProgression': True, 'enemyMissingPings': 9, 'enemyVisionPings': 0, 'firstBloodAssist': False, 'firstBloodKill': False, 'firstTowerAssist': False, 'firstTowerKill': False, 'gameEndedInEarlySurrender': False, 'gameEndedInSurrender': True, 'getBackPings': 6, 'goldEarned': 12557, 'goldSpent': 11450, 'holdPings': 0, 'individualPosition': 'TOP', 'inhibitorKills': 0, 'inhibitorTakedowns': 0, 'inhibitorsLost': 2, 'item0': 3040, 'item1': 6653, 'item2': 3116, 'item3': 1058, 'item4': 1058, 'item5': 0, 'item6': 3340, 'itemsPurchased': 21, 'killingSprees': 0, 'kills': 4, 'lane': 'JUNGLE', 'largestCriticalStrike': 0, 'largestKillingSpree': 0, 'largestMultiKill': 1, 'longestTimeSpentLiving': 336, 'magicDamageDealt': 142566, 'magicDamageDealtToChampions': 35481, 'magicDamageTaken': 20471, 'missions': {'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0}, 'needVisionPings': 1, 'neutralMinionsKilled': 4, 'nexusKills': 0, 'nexusLost': 1, 'nexusTakedowns': 0, 'objectivesStolen': 0, 'objectivesStolenAssists': 0, 'onMyWayPings': 6, 'participantId': 2, 'pentaKills': 0, 'perks': {'statPerks': {'defense': 5003, 'flex': 5008, 'offense': 5008}, 'styles': [{'description': 'primaryStyle', 'selections': [{'perk': 8010, 'var1': 928, 'var2': 0, 'var3': 0}, {'perk': 8009, 'var1': 3947, 'var2': 0, 'var3': 0}, {'perk': 9105, 'var1': 21, 'var2': 30, 'var3': 0}, {'perk': 8299, 'var1': 1829, 'var2': 0, 'var3': 0}], 'style': 8000}, {'description': 'subStyle', 'selections': [{'perk': 8226, 'var1': 250, 'var2': 2017, 'var3': 0}, {'perk': 8210, 'var1': 9, 'var2': 0, 'var3': 0}], 'style': 8200}]}, 'physicalDamageDealt': 21357, 'physicalDamageDealtToChampions': 3429, 'physicalDamageTaken': 19979, 'placement': 0, 'playerAugment1': 0, 'playerAugment2': 0, 'playerAugment3': 0, 'playerAugment4': 0, 'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0, 'playerSubteamId': 0, 'profileIcon': 11, 'pushPings': 0, 'puuid': 'PY7iMB_4ohr2vmtOB-GaUAFMb6owPm9ek4DJUBYt4IoEhxIvz3RrwACfnRDNv8uicE4HxaW-TiAtzg', 'quadraKills': 0, 'riotIdName': '', 'riotIdTagline': '', 'role': 'NONE', 'sightWardsBoughtInGame': 0, 'spell1Casts': 187, 'spell2Casts': 37, 'spell3Casts': 407, 'spell4Casts': 16, 'subteamPlacement': 0, 'summoner1Casts': 6, 'summoner1Id': 4, 'summoner2Casts': 5, 'summoner2Id': 12, 'summonerId': 'XzKi7SrTEcXzb4uXfz-VWXO3SMgGG-faQOAip9lB4rthsK8tT5i9Hdo99w', 'summonerLevel': 39, 'summonerName': 'Icky Viktor', 'teamEarlySurrendered': False, 'teamId': 100, 'teamPosition': 'TOP', 'timeCCingOthers': 54, 'timePlayed': 2158, 'totalAllyJungleMinionsKilled': 4, 'totalDamageDealt': 164317, 'totalDamageDealtToChampions': 38980, 'totalDamageShieldedOnTeammates': 0, 'totalDamageTaken': 41269, 'totalEnemyJungleMinionsKilled': 0, 'totalHeal': 5872, 'totalHealsOnTeammates': 0, 'totalMinionsKilled': 226, 'totalTimeCCDealt': 473, 'totalTimeSpentDead': 417, 'totalUnitsHealed': 1, 'tripleKills': 0, 'trueDamageDealt': 393, 'trueDamageDealtToChampions': 70, 'trueDamageTaken': 818, 'turretKills': 0, 'turretTakedowns': 0, 'turretsLost': 10, 'unrealKills': 0, 'visionClearedPings': 0, 'visionScore': 24, 'visionWardsBoughtInGame': 0, 'wardsKilled': 2, 'wardsPlaced': 12, 'win': False}, {'allInPings': 0, 'assistMePings': 2, 'assists': 7, 'baitPings': 0, 'baronKills': 0, 'basicPings': 0, 'bountyLevel': 0, 'challenges': {'12AssistStreakCount': 0, 'abilityUses': 327, 'acesBefore15Minutes': 0, 'alliedJungleMonsterKills': 0, 'baronTakedowns': 0, 'blastConeOppositeOpponentCount': 0, 'bountyGold': 0, 'buffsStolen': 0, 'completeSupportQuestInTime': 0, 'controlWardTimeCoverageInRiverOrEnemyHalf': 0.753495733287408, 'controlWardsPlaced': 3, 'damagePerMinute': 895.9700743750896, 'damageTakenOnTeamPercentage': 0.1957433771205483, 'dancedWithRiftHerald': 0, 'deathsByEnemyChamps': 18, 'dodgeSkillShotsSmallWindow': 35, 'doubleAces': 0, 'dragonTakedowns': 0, 'earliestBaron': 1657.7682029, 'earlyLaningPhaseGoldExpAdvantage': 0, 'effectiveHealAndShielding': 0, 'elderDragonKillsWithOpposingSoul': 0, 'elderDragonMultikills': 0, 'enemyChampionImmobilizations': 15, 'enemyJungleMonsterKills': 0, 'epicMonsterKillsNearEnemyJungler': 0, 'epicMonsterKillsWithin30SecondsOfSpawn': 0, 'epicMonsterSteals': 0, 'epicMonsterStolenWithoutSmite': 0, 'firstTurretKilled': 0, 'flawlessAces': 0, 'fullTeamTakedown': 0, 'gameLength': 2158.4759347, 'getTakedownsInAllLanesEarlyJungleAsLaner': 0, 'goldPerMinute': 352.98218638392865, 'hadOpenNexus': 0, 'immobilizeAndKillWithAlly': 3, 'initialBuffCount': 0, 'initialCrabCount': 0, 'jungleCsBefore10Minutes': 0, 'junglerTakedownsNearDamagedEpicMonster': 0, 'kTurretsDestroyedBeforePlatesFall': 0, 'kda': 0.4444444444444444, 'killAfterHiddenWithAlly': 2, 'killParticipation': 0.26666666666666666, 'killedChampTookFullTeamDamageSurvived': 0, 'killingSprees': 0, 'killsNearEnemyTurret': 0, 'killsOnOtherLanesEarlyJungleAsLaner': 0, 'killsOnRecentlyHealedByAramPack': 0, 'killsUnderOwnTurret': 0, 'killsWithHelpFromEpicMonster': 0, 'knockEnemyIntoTeamAndKill': 0, 'landSkillShotsEarlyGame': 20, 'laneMinionsFirst10Minutes': 68, 'laningPhaseGoldExpAdvantage': 0, 'legendaryCount': 0, 'lostAnInhibitor': 0, 'maxCsAdvantageOnLaneOpponent': 20, 'maxKillDeficit': 0, 'maxLevelLeadLaneOpponent': 1, 'mejaisFullStackInTime': 0, 'moreEnemyJungleThanOpponent': 0, 'multiKillOneSpell': 0, 'multiTurretRiftHeraldCount': 0, 'multikills': 0, 'multikillsAfterAggressiveFlash': 0, 'mythicItemUsed': 6653, 'outerTurretExecutesBefore10Minutes': 0, 'outnumberedKills': 0, 'outnumberedNexusKill': 0, 'perfectDragonSoulsTaken': 0, 'perfectGame': 0, 'pickKillWithAlly': 7, 'playedChampSelectPosition': 1, 'poroExplosions': 0, 'quickCleanse': 0, 'quickFirstTurret': 0, 'quickSoloKills': 0, 'riftHeraldTakedowns': 0, 'saveAllyFromDeath': 0, 'scuttleCrabKills': 0, 'skillshotsDodged': 252, 'skillshotsHit': 103, 'snowballsHit': 0, 'soloBaronKills': 0, 'soloKills': 0, 'stealthWardsPlaced': 13, 'survivedSingleDigitHpCount': 0, 'survivedThreeImmobilizesInFight': 0, 'takedownOnFirstTurret': 0, 'takedowns': 8, 'takedownsAfterGainingLevelAdvantage': 0, 'takedownsBeforeJungleMinionSpawn': 0, 'takedownsFirstXMinutes': 3, 'takedownsInAlcove': 0, 'takedownsInEnemyFountain': 0, 'teamBaronKills': 1, 'teamDamagePercentage': 0.20448943333322883, 'teamElderDragonKills': 0, 'teamRiftHeraldKills': 0, 'tookLargeDamageSurvived': 0, 'turretPlatesTaken': 1, 'turretTakedowns': 2, 'turretsTakenWithRiftHerald': 0, 'twentyMinionsIn3SecondsCount': 0, 'twoWardsOneSweeperCount': 0, 'unseenRecalls': 0, 'visionScoreAdvantageLaneOpponent': 0.02863776683807373, 'visionScorePerMinute': 1.2558606946186222, 'wardTakedowns': 4, 'wardTakedownsBefore20M': 3, 'wardsGuarded': 1}, 'champExperience': 16730, 'champLevel': 17, 'championId': 74, 'championName': 'Heimerdinger', 'championTransform': 0, 'commandPings': 1, 'consumablesPurchased': 10, 'damageDealtToBuildings': 3837, 'damageDealtToObjectives': 6511, 'damageDealtToTurrets': 3837, 'damageSelfMitigated': 16102, 'dangerPings': 0, 'deaths': 18, 'detectorWardsPlaced': 3, 'doubleKills': 0, 'dragonKills': 0, 'eligibleForProgression': True, 'enemyMissingPings': 4, 'enemyVisionPings': 0, 'firstBloodAssist': False, 'firstBloodKill': False, 'firstTowerAssist': False, 'firstTowerKill': False, 'gameEndedInEarlySurrender': False, 'gameEndedInSurrender': True, 'getBackPings': 1, 'goldEarned': 12698, 'goldSpent': 11275, 'holdPings': 0, 'individualPosition': 'MIDDLE', 'inhibitorKills': 0, 'inhibitorTakedowns': 0, 'inhibitorsLost': 2, 'item0': 2003, 'item1': 1056, 'item2': 3157, 'item3': 3165, 'item4': 6653, 'item5': 3020, 'item6': 3340, 'itemsPurchased': 24, 'killingSprees': 0, 'kills': 1, 'lane': 'MIDDLE', 'largestCriticalStrike': 0, 'largestKillingSpree': 0, 'largestMultiKill': 1, 'longestTimeSpentLiving': 223, 'magicDamageDealt': 179651, 'magicDamageDealtToChampions': 31095, 'magicDamageTaken': 19591, 'missions': {'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0}, 'needVisionPings': 0, 'neutralMinionsKilled': 0, 'nexusKills': 0, 'nexusLost': 1, 'nexusTakedowns': 0, 'objectivesStolen': 0, 'objectivesStolenAssists': 0, 'onMyWayPings': 3, 'participantId': 3, 'pentaKills': 0, 'perks': {'statPerks': {'defense': 5003, 'flex': 5008, 'offense': 5007}, 'styles': [{'description': 'primaryStyle', 'selections': [{'perk': 8229, 'var1': 2231, 'var2': 0, 'var3': 0}, {'perk': 8226, 'var1': 250, 'var2': 1372, 'var3': 0}, {'perk': 8210, 'var1': 13, 'var2': 0, 'var3': 0}, {'perk': 8237, 'var1': 1265, 'var2': 0, 'var3': 0}], 'style': 8200}, {'description': 'subStyle', 'selections': [{'perk': 8135, 'var1': 320, 'var2': 4, 'var3': 0}, {'perk': 8139, 'var1': 1191, 'var2': 0, 'var3': 0}], 'style': 8100}]}, 'physicalDamageDealt': 15007, 'physicalDamageDealtToChampions': 1136, 'physicalDamageTaken': 14864, 'placement': 0, 'playerAugment1': 0, 'playerAugment2': 0, 'playerAugment3': 0, 'playerAugment4': 0, 'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0, 'playerSubteamId': 0, 'profileIcon': 21, 'pushPings': 0, 'puuid': 'vr_-Loy-Fc_iHTYyWWa383FyLCw6c0iKLnVREVa6Vtab4bAqMcE9XLbFaxnsgqwT5GTjELIuhm7Eyw', 'quadraKills': 0, 'riotIdName': '', 'riotIdTagline': '', 'role': 'SOLO', 'sightWardsBoughtInGame': 0, 'spell1Casts': 123, 'spell2Casts': 114, 'spell3Casts': 72, 'spell4Casts': 18, 'subteamPlacement': 0, 'summoner1Casts': 4, 'summoner1Id': 4, 'summoner2Casts': 4, 'summoner2Id': 12, 'summonerId': '2OYBKao4wuXO3WVqfy58hycAmwlBe3mEV-mP2ky7nYscfAo', 'summonerLevel': 447, 'summonerName': 'Russ2k', 'teamEarlySurrendered': False, 'teamId': 100, 'teamPosition': 'MIDDLE', 'timeCCingOthers': 28, 'timePlayed': 2158, 'totalAllyJungleMinionsKilled': 0, 'totalDamageDealt': 196298, 'totalDamageDealtToChampions': 32232, 'totalDamageShieldedOnTeammates': 0, 'totalDamageTaken': 35745, 'totalEnemyJungleMinionsKilled': 0, 'totalHeal': 1002, 'totalHealsOnTeammates': 0, 'totalMinionsKilled': 212, 'totalTimeCCDealt': 378, 'totalTimeSpentDead': 594, 'totalUnitsHealed': 1, 'tripleKills': 0, 'trueDamageDealt': 1640, 'trueDamageDealtToChampions': 0, 'trueDamageTaken': 1290, 'turretKills': 2, 'turretTakedowns': 2, 'turretsLost': 10, 'unrealKills': 0, 'visionClearedPings': 0, 'visionScore': 45, 'visionWardsBoughtInGame': 3, 'wardsKilled': 4, 'wardsPlaced': 16, 'win': False}, {'allInPings': 0, 'assistMePings': 0, 'assists': 5, 'baitPings': 0, 'baronKills': 0, 'basicPings': 0, 'bountyLevel': 0, 'challenges': {'12AssistStreakCount': 0, 'abilityUses': 368, 'acesBefore15Minutes': 0, 'alliedJungleMonsterKills': 3, 'baronTakedowns': 1, 'blastConeOppositeOpponentCount': 0, 'bountyGold': 300, 'buffsStolen': 0, 'completeSupportQuestInTime': 0, 'controlWardsPlaced': 0, 'damagePerMinute': 935.6875515875076, 'damageTakenOnTeamPercentage': 0.18680074198641475, 'dancedWithRiftHerald': 0, 'deathsByEnemyChamps': 7, 'dodgeSkillShotsSmallWindow': 18, 'doubleAces': 0, 'dragonTakedowns': 0, 'earliestBaron': 1657.7682029, 'earlyLaningPhaseGoldExpAdvantage': 0, 'effectiveHealAndShielding': 195.80001831054688, 'elderDragonKillsWithOpposingSoul': 0, 'elderDragonMultikills': 0, 'enemyChampionImmobilizations': 0, 'enemyJungleMonsterKills': 0, 'epicMonsterKillsNearEnemyJungler': 0, 'epicMonsterKillsWithin30SecondsOfSpawn': 0, 'epicMonsterSteals': 0, 'epicMonsterStolenWithoutSmite': 0, 'firstTurretKilled': 0, 'flawlessAces': 0, 'fullTeamTakedown': 0, 'gameLength': 2158.4759347, 'getTakedownsInAllLanesEarlyJungleAsLaner': 0, 'goldPerMinute': 413.9642878386732, 'hadOpenNexus': 0, 'immobilizeAndKillWithAlly': 0, 'initialBuffCount': 0, 'initialCrabCount': 0, 'jungleCsBefore10Minutes': 0, 'junglerTakedownsNearDamagedEpicMonster': 0, 'kTurretsDestroyedBeforePlatesFall': 0, 'kda': 2, 'killAfterHiddenWithAlly': 1, 'killParticipation': 0.4666666666666667, 'killedChampTookFullTeamDamageSurvived': 0, 'killingSprees': 1, 'killsNearEnemyTurret': 0, 'killsOnOtherLanesEarlyJungleAsLaner': 0, 'killsOnRecentlyHealedByAramPack': 0, 'killsUnderOwnTurret': 3, 'killsWithHelpFromEpicMonster': 0, 'knockEnemyIntoTeamAndKill': 0, 'landSkillShotsEarlyGame': 13, 'laneMinionsFirst10Minutes': 55, 'laningPhaseGoldExpAdvantage': 0, 'legendaryCount': 0, 'lostAnInhibitor': 0, 'maxCsAdvantageOnLaneOpponent': 9, 'maxKillDeficit': 0, 'maxLevelLeadLaneOpponent': 2, 'mejaisFullStackInTime': 0, 'moreEnemyJungleThanOpponent': 0, 'multiKillOneSpell': 0, 'multiTurretRiftHeraldCount': 0, 'multikills': 1, 'multikillsAfterAggressiveFlash': 0, 'mythicItemUsed': 3078, 'outerTurretExecutesBefore10Minutes': 0, 'outnumberedKills': 2, 'outnumberedNexusKill': 0, 'perfectDragonSoulsTaken': 0, 'perfectGame': 0, 'pickKillWithAlly': 12, 'playedChampSelectPosition': 1, 'poroExplosions': 0, 'quickCleanse': 0, 'quickFirstTurret': 0, 'quickSoloKills': 0, 'riftHeraldTakedowns': 0, 'saveAllyFromDeath': 0, 'scuttleCrabKills': 2, 'skillshotsDodged': 115, 'skillshotsHit': 126, 'snowballsHit': 0, 'soloBaronKills': 0, 'soloKills': 1, 'stealthWardsPlaced': 3, 'survivedSingleDigitHpCount': 0, 'survivedThreeImmobilizesInFight': 2, 'takedownOnFirstTurret': 0, 'takedowns': 14, 'takedownsAfterGainingLevelAdvantage': 0, 'takedownsBeforeJungleMinionSpawn': 0, 'takedownsFirstXMinutes': 3, 'takedownsInAlcove': 0, 'takedownsInEnemyFountain': 0, 'teamBaronKills': 1, 'teamDamagePercentage': 0.2135542499391378, 'teamElderDragonKills': 0, 'teamRiftHeraldKills': 0, 'tookLargeDamageSurvived': 0, 'turretPlatesTaken': 1, 'turretTakedowns': 2, 'turretsTakenWithRiftHerald': 0, 'twentyMinionsIn3SecondsCount': 0, 'twoWardsOneSweeperCount': 0, 'unseenRecalls': 0, 'visionScoreAdvantageLaneOpponent': -0.5843740105628967, 'visionScorePerMinute': 0.3165741216537584, 'wardTakedowns': 2, 'wardTakedownsBefore20M': 0, 'wardsGuarded': 1}, 'champExperience': 18038, 'champLevel': 17, 'championId': 81, 'championName': 'Ezreal', 'championTransform': 0, 'commandPings': 6, 'consumablesPurchased': 1, 'damageDealtToBuildings': 4698, 'damageDealtToObjectives': 8069, 'damageDealtToTurrets': 4698, 'damageSelfMitigated': 13849, 'dangerPings': 0, 'deaths': 7, 'detectorWardsPlaced': 0, 'doubleKills': 1, 'dragonKills': 0, 'eligibleForProgression': True, 'enemyMissingPings': 2, 'enemyVisionPings': 0, 'firstBloodAssist': False, 'firstBloodKill': False, 'firstTowerAssist': False, 'firstTowerKill': False, 'gameEndedInEarlySurrender': False, 'gameEndedInSurrender': True, 'getBackPings': 0, 'goldEarned': 14892, 'goldSpent': 14493, 'holdPings': 0, 'individualPosition': 'BOTTOM', 'inhibitorKills': 0, 'inhibitorTakedowns': 0, 'inhibitorsLost': 2, 'item0': 6694, 'item1': 3078, 'item2': 3042, 'item3': 3074, 'item4': 0, 'item5': 3158, 'item6': 3363, 'itemsPurchased': 20, 'killingSprees': 1, 'kills': 9, 'lane': 'BOTTOM', 'largestCriticalStrike': 0, 'largestKillingSpree': 7, 'largestMultiKill': 2, 'longestTimeSpentLiving': 588, 'magicDamageDealt': 22313, 'magicDamageDealtToChampions': 7219, 'magicDamageTaken': 17291, 'missions': {'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0}, 'needVisionPings': 0, 'neutralMinionsKilled': 12, 'nexusKills': 0, 'nexusLost': 1, 'nexusTakedowns': 0, 'objectivesStolen': 0, 'objectivesStolenAssists': 0, 'onMyWayPings': 0, 'participantId': 4, 'pentaKills': 0, 'perks': {'statPerks': {'defense': 5002, 'flex': 5008, 'offense': 5005}, 'styles': [{'description': 'primaryStyle', 'selections': [{'perk': 8005, 'var1': 1454, 'var2': 832, 'var3': 622}, {'perk': 8009, 'var1': 4442, 'var2': 0, 'var3': 0}, {'perk': 9103, 'var1': 24, 'var2': 20, 'var3': 0}, {'perk': 8014, 'var1': 690, 'var2': 0, 'var3': 0}], 'style': 8000}, {'description': 'subStyle', 'selections': [{'perk': 8304, 'var1': 12, 'var2': 0, 'var3': 0}, {'perk': 8345, 'var1': 3, 'var2': 0, 'var3': 0}], 'style': 8300}]}, 'physicalDamageDealt': 153705, 'physicalDamageDealtToChampions': 26016, 'physicalDamageTaken': 16354, 'placement': 0, 'playerAugment1': 0, 'playerAugment2': 0, 'playerAugment3': 0, 'playerAugment4': 0, 'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0, 'playerSubteamId': 0, 'profileIcon': 3008, 'pushPings': 0, 'puuid': '34wqa3OKa6UWHSPIvs3i0BKlIYmR-dmz3C7uLs9o58K6O-ZnRjoBRJ4km59gerGJ546Fcr7S0ysaxw', 'quadraKills': 0, 'riotIdName': '', 'riotIdTagline': '', 'role': 'CARRY', 'sightWardsBoughtInGame': 0, 'spell1Casts': 270, 'spell2Casts': 52, 'spell3Casts': 33, 'spell4Casts': 13, 'subteamPlacement': 0, 'summoner1Casts': 5, 'summoner1Id': 4, 'summoner2Casts': 5, 'summoner2Id': 7, 'summonerId': 'bpfPS6F6No-4HKCZ6Ds9TnzdM72fm7KJaeJlEcskuNppQ_Q', 'summonerLevel': 375, 'summonerName': 'ShoCrispy', 'teamEarlySurrendered': False, 'teamId': 100, 'teamPosition': 'BOTTOM', 'timeCCingOthers': 11, 'timePlayed': 2158, 'totalAllyJungleMinionsKilled': 4, 'totalDamageDealt': 179178, 'totalDamageDealtToChampions': 33660, 'totalDamageShieldedOnTeammates': 0, 'totalDamageTaken': 34112, 'totalEnemyJungleMinionsKilled': 0, 'totalHeal': 6373, 'totalHealsOnTeammates': 195, 'totalMinionsKilled': 234, 'totalTimeCCDealt': 122, 'totalTimeSpentDead': 267, 'totalUnitsHealed': 2, 'tripleKills': 0, 'trueDamageDealt': 3159, 'trueDamageDealtToChampions': 425, 'trueDamageTaken': 467, 'turretKills': 2, 'turretTakedowns': 2, 'turretsLost': 10, 'unrealKills': 0, 'visionClearedPings': 0, 'visionScore': 11, 'visionWardsBoughtInGame': 0, 'wardsKilled': 2, 'wardsPlaced': 7, 'win': False}, {'allInPings': 0, 'assistMePings': 0, 'assists': 13, 'baitPings': 0, 'baronKills': 0, 'basicPings': 0, 'bountyLevel': 0, 'challenges': {'12AssistStreakCount': 1, 'abilityUses': 148, 'acesBefore15Minutes': 0, 'alliedJungleMonsterKills': 0, 'baronTakedowns': 0, 'blastConeOppositeOpponentCount': 0, 'bountyGold': 0, 'buffsStolen': 0, 'completeSupportQuestInTime': 0, 'controlWardTimeCoverageInRiverOrEnemyHalf': 0.6262096394361065, 'controlWardsPlaced': 11, 'damagePerMinute': 136.79166799630661, 'damageTakenOnTeamPercentage': 0.10534221619617502, 'dancedWithRiftHerald': 0, 'deathsByEnemyChamps': 7, 'dodgeSkillShotsSmallWindow': 14, 'doubleAces': 0, 'dragonTakedowns': 0, 'earliestBaron': 1657.7682029, 'earlyLaningPhaseGoldExpAdvantage': 0, 'effectiveHealAndShielding': 15736.30859375, 'elderDragonKillsWithOpposingSoul': 0, 'elderDragonMultikills': 0, 'enemyChampionImmobilizations': 10, 'enemyJungleMonsterKills': 0, 'epicMonsterKillsNearEnemyJungler': 0, 'epicMonsterKillsWithin30SecondsOfSpawn': 0, 'epicMonsterSteals': 0, 'epicMonsterStolenWithoutSmite': 0, 'firstTurretKilled': 0, 'flawlessAces': 0, 'fullTeamTakedown': 0, 'gameLength': 2158.4759347, 'getTakedownsInAllLanesEarlyJungleAsLaner': 0, 'goldPerMinute': 223.9522883849922, 'hadOpenNexus': 0, 'immobilizeAndKillWithAlly': 2, 'initialBuffCount': 0, 'initialCrabCount': 0, 'jungleCsBefore10Minutes': 0, 'junglerTakedownsNearDamagedEpicMonster': 0, 'kTurretsDestroyedBeforePlatesFall': 0, 'kda': 1.8571428571428572, 'killAfterHiddenWithAlly': 0, 'killParticipation': 0.43333333333333335, 'killedChampTookFullTeamDamageSurvived': 0, 'killingSprees': 0, 'killsNearEnemyTurret': 0, 'killsOnOtherLanesEarlyJungleAsLaner': 0, 'killsOnRecentlyHealedByAramPack': 0, 'killsUnderOwnTurret': 0, 'killsWithHelpFromEpicMonster': 0, 'knockEnemyIntoTeamAndKill': 0, 'landSkillShotsEarlyGame': 5, 'laneMinionsFirst10Minutes': 16, 'laningPhaseGoldExpAdvantage': 0, 'legendaryCount': 0, 'lostAnInhibitor': 0, 'maxCsAdvantageOnLaneOpponent': 18, 'maxKillDeficit': 0, 'maxLevelLeadLaneOpponent': 1, 'mejaisFullStackInTime': 0, 'moreEnemyJungleThanOpponent': 0, 'multiKillOneSpell': 0, 'multiTurretRiftHeraldCount': 0, 'multikills': 0, 'multikillsAfterAggressiveFlash': 0, 'mythicItemUsed': 2065, 'outerTurretExecutesBefore10Minutes': 0, 'outnumberedKills': 0, 'outnumberedNexusKill': 0, 'perfectDragonSoulsTaken': 0, 'perfectGame': 0, 'pickKillWithAlly': 12, 'playedChampSelectPosition': 1, 'poroExplosions': 0, 'quickCleanse': 0, 'quickFirstTurret': 0, 'quickSoloKills': 0, 'riftHeraldTakedowns': 0, 'saveAllyFromDeath': 7, 'scuttleCrabKills': 0, 'skillshotsDodged': 102, 'skillshotsHit': 21, 'snowballsHit': 0, 'soloBaronKills': 0, 'soloKills': 0, 'stealthWardsPlaced': 32, 'survivedSingleDigitHpCount': 0, 'survivedThreeImmobilizesInFight': 2, 'takedownOnFirstTurret': 0, 'takedowns': 13, 'takedownsAfterGainingLevelAdvantage': 0, 'takedownsBeforeJungleMinionSpawn': 0, 'takedownsFirstXMinutes': 3, 'takedownsInAlcove': 0, 'takedownsInEnemyFountain': 0, 'teamBaronKills': 1, 'teamDamagePercentage': 0.031220295714431984, 'teamElderDragonKills': 0, 'teamRiftHeraldKills': 0, 'tookLargeDamageSurvived': 0, 'turretPlatesTaken': 1, 'turretTakedowns': 0, 'turretsTakenWithRiftHerald': 0, 'twentyMinionsIn3SecondsCount': 0, 'twoWardsOneSweeperCount': 1, 'unseenRecalls': 0, 'visionScoreAdvantageLaneOpponent': -0.018535315990447998, 'visionScorePerMinute': 2.714572656568273, 'wardTakedowns': 11, 'wardTakedownsBefore20M': 6, 'wardsGuarded': 0}, 'champExperience': 13148, 'champLevel': 15, 'championId': 16, 'championName': 'Soraka', 'championTransform': 0, 'commandPings': 0, 'consumablesPurchased': 18, 'damageDealtToBuildings': 345, 'damageDealtToObjectives': 345, 'damageDealtToTurrets': 345, 'damageSelfMitigated': 8370, 'dangerPings': 0, 'deaths': 7, 'detectorWardsPlaced': 11, 'doubleKills': 0, 'dragonKills': 0, 'eligibleForProgression': True, 'enemyMissingPings': 5, 'enemyVisionPings': 4, 'firstBloodAssist': False, 'firstBloodKill': False, 'firstTowerAssist': False, 'firstTowerKill': False, 'gameEndedInEarlySurrender': False, 'gameEndedInSurrender': True, 'getBackPings': 0, 'goldEarned': 8056, 'goldSpent': 6600, 'holdPings': 0, 'individualPosition': 'UTILITY', 'inhibitorKills': 0, 'inhibitorTakedowns': 0, 'inhibitorsLost': 2, 'item0': 2065, 'item1': 3801, 'item2': 3860, 'item3': 0, 'item4': 3067, 'item5': 3158, 'item6': 3364, 'itemsPurchased': 34, 'killingSprees': 0, 'kills': 0, 'lane': 'BOTTOM', 'largestCriticalStrike': 0, 'largestKillingSpree': 0, 'largestMultiKill': 0, 'longestTimeSpentLiving': 565, 'magicDamageDealt': 6446, 'magicDamageDealtToChampions': 4223, 'magicDamageTaken': 10437, 'missions': {'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0}, 'needVisionPings': 0, 'neutralMinionsKilled': 0, 'nexusKills': 0, 'nexusLost': 1, 'nexusTakedowns': 0, 'objectivesStolen': 0, 'objectivesStolenAssists': 0, 'onMyWayPings': 3, 'participantId': 5, 'pentaKills': 0, 'perks': {'statPerks': {'defense': 5002, 'flex': 5008, 'offense': 5008}, 'styles': [{'description': 'primaryStyle', 'selections': [{'perk': 8214, 'var1': 652, 'var2': 1091, 'var3': 0}, {'perk': 8226, 'var1': 250, 'var2': 407, 'var3': 0}, {'perk': 8210, 'var1': 6, 'var2': 0, 'var3': 0}, {'perk': 8237, 'var1': 537, 'var2': 0, 'var3': 0}], 'style': 8200}, {'description': 'subStyle', 'selections': [{'perk': 8473, 'var1': 823, 'var2': 0, 'var3': 0}, {'perk': 8453, 'var1': 2666, 'var2': 285, 'var3': 0}], 'style': 8400}]}, 'physicalDamageDealt': 3239, 'physicalDamageDealtToChampions': 697, 'physicalDamageTaken': 8685, 'placement': 0, 'playerAugment1': 0, 'playerAugment2': 0, 'playerAugment3': 0, 'playerAugment4': 0, 'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0, 'playerSubteamId': 0, 'profileIcon': 21, 'pushPings': 0, 'puuid': 'IwYjkxE9AlLTMZ_9Mf6YVpyGcQYOeQq2azNWgQhU2BXEm0vZn6Bvrxj7jNIEr-iVQAAxAqQFa4jDHg', 'quadraKills': 0, 'riotIdName': '', 'riotIdTagline': '', 'role': 'SUPPORT', 'sightWardsBoughtInGame': 0, 'spell1Casts': 38, 'spell2Casts': 62, 'spell3Casts': 38, 'spell4Casts': 10, 'subteamPlacement': 0, 'summoner1Casts': 5, 'summoner1Id': 4, 'summoner2Casts': 7, 'summoner2Id': 3, 'summonerId': 'HMvtwffgxJcXZ-0ebi0JJC2PPidtPQ3jk8-B0UGdM94JjQ0', 'summonerLevel': 344, 'summonerName': 'FrostUlfhednar', 'teamEarlySurrendered': False, 'teamId': 100, 'teamPosition': 'UTILITY', 'timeCCingOthers': 47, 'timePlayed': 2158, 'totalAllyJungleMinionsKilled': 0, 'totalDamageDealt': 12469, 'totalDamageDealtToChampions': 4921, 'totalDamageShieldedOnTeammates': 1254, 'totalDamageTaken': 19237, 'totalEnemyJungleMinionsKilled': 0, 'totalHeal': 18933, 'totalHealsOnTeammates': 14481, 'totalMinionsKilled': 33, 'totalTimeCCDealt': 144, 'totalTimeSpentDead': 185, 'totalUnitsHealed': 5, 'tripleKills': 0, 'trueDamageDealt': 2783, 'trueDamageDealtToChampions': 0, 'trueDamageTaken': 113, 'turretKills': 0, 'turretTakedowns': 0, 'turretsLost': 10, 'unrealKills': 0, 'visionClearedPings': 0, 'visionScore': 97, 'visionWardsBoughtInGame': 12, 'wardsKilled': 10, 'wardsPlaced': 46, 'win': False}, {'allInPings': 0, 'assistMePings': 1, 'assists': 9, 'baitPings': 0, 'baronKills': 0, 'basicPings': 0, 'bountyLevel': 1, 'challenges': {'12AssistStreakCount': 0, 'abilityUses': 250, 'acesBefore15Minutes': 0, 'alliedJungleMonsterKills': 0, 'baronBuffGoldAdvantageOverThreshold': 1, 'baronTakedowns': 0, 'blastConeOppositeOpponentCount': 0, 'bountyGold': 0, 'buffsStolen': 1, 'completeSupportQuestInTime': 0, 'controlWardTimeCoverageInRiverOrEnemyHalf': 0.766887354678831, 'controlWardsPlaced': 3, 'damagePerMinute': 699.3837747303006, 'damageTakenOnTeamPercentage': 0.2687747076383493, 'dancedWithRiftHerald': 0, 'deathsByEnemyChamps': 5, 'dodgeSkillShotsSmallWindow': 9, 'doubleAces': 0, 'dragonTakedowns': 1, 'earliestBaron': 2074.9830690999997, 'earliestDragonTakedown': 1823.6520494, 'earlyLaningPhaseGoldExpAdvantage': 0, 'effectiveHealAndShielding': 0, 'elderDragonKillsWithOpposingSoul': 0, 'elderDragonMultikills': 0, 'enemyChampionImmobilizations': 70, 'enemyJungleMonsterKills': 7, 'epicMonsterKillsNearEnemyJungler': 0, 'epicMonsterKillsWithin30SecondsOfSpawn': 0, 'epicMonsterSteals': 0, 'epicMonsterStolenWithoutSmite': 0, 'firstTurretKilled': 1, 'firstTurretKilledTime': 947.9954345, 'flawlessAces': 1, 'fullTeamTakedown': 1, 'gameLength': 2158.4759347, 'getTakedownsInAllLanesEarlyJungleAsLaner': 0, 'goldPerMinute': 362.1064357395219, 'hadOpenNexus': 0, 'immobilizeAndKillWithAlly': 10, 'initialBuffCount': 0, 'initialCrabCount': 0, 'jungleCsBefore10Minutes': 0, 'junglerTakedownsNearDamagedEpicMonster': 0, 'kTurretsDestroyedBeforePlatesFall': 0, 'kda': 3.4, 'killAfterHiddenWithAlly': 0, 'killParticipation': 0.3148148148148148, 'killedChampTookFullTeamDamageSurvived': 0, 'killingSprees': 0, 'killsNearEnemyTurret': 1, 'killsOnOtherLanesEarlyJungleAsLaner': 0, 'killsOnRecentlyHealedByAramPack': 0, 'killsUnderOwnTurret': 1, 'killsWithHelpFromEpicMonster': 0, 'knockEnemyIntoTeamAndKill': 7, 'landSkillShotsEarlyGame': 5, 'laneMinionsFirst10Minutes': 66, 'laningPhaseGoldExpAdvantage': 0, 'legendaryCount': 0, 'lostAnInhibitor': 0, 'maxCsAdvantageOnLaneOpponent': 10, 'maxKillDeficit': 0, 'maxLevelLeadLaneOpponent': 2, 'mejaisFullStackInTime': 0, 'moreEnemyJungleThanOpponent': 0, 'multiKillOneSpell': 0, 'multiTurretRiftHeraldCount': 0, 'multikills': 0, 'multikillsAfterAggressiveFlash': 0, 'mythicItemUsed': 6665, 'outerTurretExecutesBefore10Minutes': 0, 'outnumberedKills': 0, 'outnumberedNexusKill': 0, 'perfectDragonSoulsTaken': 0, 'perfectGame': 0, 'pickKillWithAlly': 11, 'playedChampSelectPosition': 1, 'poroExplosions': 0, 'quickCleanse': 0, 'quickFirstTurret': 0, 'quickSoloKills': 0, 'riftHeraldTakedowns': 0, 'saveAllyFromDeath': 0, 'scuttleCrabKills': 0, 'skillshotsDodged': 84, 'skillshotsHit': 39, 'snowballsHit': 0, 'soloBaronKills': 0, 'soloKills': 3, 'soloTurretsLategame': 1, 'stealthWardsPlaced': 11, 'survivedSingleDigitHpCount': 0, 'survivedThreeImmobilizesInFight': 2, 'takedownOnFirstTurret': 0, 'takedowns': 17, 'takedownsAfterGainingLevelAdvantage': 0, 'takedownsBeforeJungleMinionSpawn': 0, 'takedownsFirstXMinutes': 4, 'takedownsInAlcove': 0, 'takedownsInEnemyFountain': 0, 'teamBaronKills': 1, 'teamDamagePercentage': 0.16521032085333118, 'teamElderDragonKills': 0, 'teamRiftHeraldKills': 1, 'tookLargeDamageSurvived': 1, 'turretPlatesTaken': 1, 'turretTakedowns': 2, 'turretsTakenWithRiftHerald': 0, 'twentyMinionsIn3SecondsCount': 0, 'twoWardsOneSweeperCount': 0, 'unseenRecalls': 0, 'visionScoreAdvantageLaneOpponent': 0.31673312187194824, 'visionScorePerMinute': 0.8961020483602145, 'wardTakedowns': 4, 'wardTakedownsBefore20M': 3, 'wardsGuarded': 1}, 'champExperience': 19027, 'champLevel': 18, 'championId': 516, 'championName': 'Ornn', 'championTransform': 0, 'commandPings': 5, 'consumablesPurchased': 6, 'damageDealtToBuildings': 4552, 'damageDealtToObjectives': 5085, 'damageDealtToTurrets': 4552, 'damageSelfMitigated': 116226, 'dangerPings': 0, 'deaths': 5, 'detectorWardsPlaced': 3, 'doubleKills': 0, 'dragonKills': 0, 'eligibleForProgression': True, 'enemyMissingPings': 1, 'enemyVisionPings': 1, 'firstBloodAssist': False, 'firstBloodKill': False, 'firstTowerAssist': False, 'firstTowerKill': False, 'gameEndedInEarlySurrender': False, 'gameEndedInSurrender': True, 'getBackPings': 0, 'goldEarned': 13026, 'goldSpent': 12125, 'holdPings': 0, 'individualPosition': 'TOP', 'inhibitorKills': 0, 'inhibitorTakedowns': 0, 'inhibitorsLost': 0, 'item0': 7026, 'item1': 3111, 'item2': 4401, 'item3': 3068, 'item4': 3076, 'item5': 3082, 'item6': 3340, 'itemsPurchased': 34, 'killingSprees': 2, 'kills': 8, 'lane': 'TOP', 'largestCriticalStrike': 0, 'largestKillingSpree': 2, 'largestMultiKill': 1, 'longestTimeSpentLiving': 708, 'magicDamageDealt': 66716, 'magicDamageDealtToChampions': 16879, 'magicDamageTaken': 25060, 'missions': {'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0}, 'needVisionPings': 0, 'neutralMinionsKilled': 8, 'nexusKills': 0, 'nexusLost': 0, 'nexusTakedowns': 0, 'objectivesStolen': 0, 'objectivesStolenAssists': 0, 'onMyWayPings': 3, 'participantId': 6, 'pentaKills': 0, 'perks': {'statPerks': {'defense': 5003, 'flex': 5002, 'offense': 5005}, 'styles': [{'description': 'primaryStyle', 'selections': [{'perk': 8437, 'var1': 2092, 'var2': 1248, 'var3': 0}, {'perk': 8446, 'var1': 3174, 'var2': 0, 'var3': 0}, {'perk': 8473, 'var1': 1068, 'var2': 0, 'var3': 0}, {'perk': 8451, 'var1': 295, 'var2': 0, 'var3': 0}], 'style': 8400}, {'description': 'subStyle', 'selections': [{'perk': 8304, 'var1': 9, 'var2': 4, 'var3': 5}, {'perk': 8345, 'var1': 3, 'var2': 0, 'var3': 0}], 'style': 8300}]}, 'physicalDamageDealt': 79000, 'physicalDamageDealtToChampions': 7148, 'physicalDamageTaken': 29488, 'placement': 0, 'playerAugment1': 0, 'playerAugment2': 0, 'playerAugment3': 0, 'playerAugment4': 0, 'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0, 'playerSubteamId': 0, 'profileIcon': 5726, 'pushPings': 0, 'puuid': 'bHhAY7DSUICKSCwWzf2EeKhtFsUq4fEOXZM4BsXK4CIXbAW0z-N2I-2yls25q9G__fugkZVhF7h_Pw', 'quadraKills': 0, 'riotIdName': '', 'riotIdTagline': '', 'role': 'SOLO', 'sightWardsBoughtInGame': 0, 'spell1Casts': 109, 'spell2Casts': 69, 'spell3Casts': 48, 'spell4Casts': 24, 'subteamPlacement': 0, 'summoner1Casts': 5, 'summoner1Id': 4, 'summoner2Casts': 6, 'summoner2Id': 12, 'summonerId': 'RMWLwLgy0eY8Fl1oC9tr3umr_jVVfXNFok39tP7JZq9A2FU', 'summonerLevel': 524, 'summonerName': 'Be Out Of Date', 'teamEarlySurrendered': False, 'teamId': 200, 'teamPosition': 'TOP', 'timeCCingOthers': 48, 'timePlayed': 2158, 'totalAllyJungleMinionsKilled': 0, 'totalDamageDealt': 156124, 'totalDamageDealtToChampions': 25160, 'totalDamageShieldedOnTeammates': 0, 'totalDamageTaken': 54909, 'totalEnemyJungleMinionsKilled': 8, 'totalHeal': 18178, 'totalHealsOnTeammates': 0, 'totalMinionsKilled': 191, 'totalTimeCCDealt': 720, 'totalTimeSpentDead': 197, 'totalUnitsHealed': 1, 'tripleKills': 0, 'trueDamageDealt': 10407, 'trueDamageDealtToChampions': 1132, 'trueDamageTaken': 361, 'turretKills': 1, 'turretTakedowns': 2, 'turretsLost': 4, 'unrealKills': 0, 'visionClearedPings': 0, 'visionScore': 32, 'visionWardsBoughtInGame': 3, 'wardsKilled': 4, 'wardsPlaced': 14, 'win': True}, {'allInPings': 0, 'assistMePings': 1, 'assists': 16, 'baitPings': 0, 'baronKills': 1, 'basicPings': 0, 'bountyLevel': 1, 'challenges': {'12AssistStreakCount': 0, 'abilityUses': 268, 'acesBefore15Minutes': 0, 'alliedJungleMonsterKills': 73, 'baronBuffGoldAdvantageOverThreshold': 1, 'baronTakedowns': 1, 'blastConeOppositeOpponentCount': 0, 'bountyGold': 300, 'buffsStolen': 1, 'completeSupportQuestInTime': 0, 'controlWardsPlaced': 0, 'damagePerMinute': 841.8503452947022, 'damageTakenOnTeamPercentage': 0.23221768010407318, 'dancedWithRiftHerald': 0, 'deathsByEnemyChamps': 3, 'dodgeSkillShotsSmallWindow': 3, 'doubleAces': 0, 'dragonTakedowns': 4, 'earliestBaron': 2074.9830690999997, 'earliestDragonTakedown': 790.9937994999999, 'earlyLaningPhaseGoldExpAdvantage': 0, 'effectiveHealAndShielding': 0, 'elderDragonKillsWithOpposingSoul': 0, 'elderDragonMultikills': 0, 'enemyChampionImmobilizations': 66, 'enemyJungleMonsterKills': 7, 'epicMonsterKillsNearEnemyJungler': 2, 'epicMonsterKillsWithin30SecondsOfSpawn': 1, 'epicMonsterSteals': 0, 'epicMonsterStolenWithoutSmite': 0, 'firstTurretKilled': 1, 'firstTurretKilledTime': 947.9954345, 'flawlessAces': 1, 'fullTeamTakedown': 1, 'gameLength': 2158.4759347, 'goldPerMinute': 465.2534063992592, 'hadOpenNexus': 0, 'immobilizeAndKillWithAlly': 17, 'initialBuffCount': 2, 'initialCrabCount': 0, 'jungleCsBefore10Minutes': 56.00000008940697, 'junglerKillsEarlyJungle': 0, 'junglerTakedownsNearDamagedEpicMonster': 1, 'kTurretsDestroyedBeforePlatesFall': 0, 'kda': 9, 'killAfterHiddenWithAlly': 3, 'killParticipation': 0.5, 'killedChampTookFullTeamDamageSurvived': 0, 'killingSprees': 1, 'killsNearEnemyTurret': 2, 'killsOnLanersEarlyJungleAsJungler': 1, 'killsOnRecentlyHealedByAramPack': 0, 'killsUnderOwnTurret': 1, 'killsWithHelpFromEpicMonster': 0, 'knockEnemyIntoTeamAndKill': 0, 'landSkillShotsEarlyGame': 0, 'laneMinionsFirst10Minutes': 4, 'laningPhaseGoldExpAdvantage': 0, 'legendaryCount': 0, 'lostAnInhibitor': 0, 'maxCsAdvantageOnLaneOpponent': 4, 'maxKillDeficit': 0, 'maxLevelLeadLaneOpponent': 1, 'mejaisFullStackInTime': 0, 'moreEnemyJungleThanOpponent': -59.00000008940697, 'multiKillOneSpell': 1, 'multiTurretRiftHeraldCount': 0, 'multikills': 0, 'multikillsAfterAggressiveFlash': 0, 'mythicItemUsed': 3152, 'outerTurretExecutesBefore10Minutes': 0, 'outnumberedKills': 2, 'outnumberedNexusKill': 0, 'perfectDragonSoulsTaken': 0, 'perfectGame': 0, 'pickKillWithAlly': 23, 'playedChampSelectPosition': 1, 'poroExplosions': 0, 'quickCleanse': 0, 'quickFirstTurret': 0, 'quickSoloKills': 0, 'riftHeraldTakedowns': 1, 'saveAllyFromDeath': 0, 'scuttleCrabKills': 2, 'shortestTimeToAceFromFirstTakedown': 42.70709119999992, 'skillshotsDodged': 48, 'skillshotsHit': 35, 'snowballsHit': 0, 'soloBaronKills': 0, 'soloKills': 1, 'stealthWardsPlaced': 0, 'survivedSingleDigitHpCount': 0, 'survivedThreeImmobilizesInFight': 0, 'takedownOnFirstTurret': 1, 'takedowns': 27, 'takedownsAfterGainingLevelAdvantage': 0, 'takedownsBeforeJungleMinionSpawn': 0, 'takedownsFirstXMinutes': 5, 'takedownsInAlcove': 0, 'takedownsInEnemyFountain': 1, 'teamBaronKills': 1, 'teamDamagePercentage': 0.19886415825168222, 'teamElderDragonKills': 0, 'teamRiftHeraldKills': 1, 'tookLargeDamageSurvived': 0, 'turretPlatesTaken': 0, 'turretTakedowns': 4, 'turretsTakenWithRiftHerald': 0, 'twentyMinionsIn3SecondsCount': 0, 'twoWardsOneSweeperCount': 0, 'unseenRecalls': 0, 'visionScoreAdvantageLaneOpponent': 0.13065576553344727, 'visionScorePerMinute': 1.047514505362541, 'wardTakedowns': 2, 'wardTakedownsBefore20M': 1, 'wardsGuarded': 0}, 'champExperience': 19685, 'champLevel': 18, 'championId': 9, 'championName': 'FiddleSticks', 'championTransform': 0, 'commandPings': 0, 'consumablesPurchased': 1, 'damageDealtToBuildings': 3537, 'damageDealtToObjectives': 49259, 'damageDealtToTurrets': 3537, 'damageSelfMitigated': 32987, 'dangerPings': 0, 'deaths': 3, 'detectorWardsPlaced': 0, 'doubleKills': 0, 'dragonKills': 4, 'eligibleForProgression': True, 'enemyMissingPings': 0, 'enemyVisionPings': 0, 'firstBloodAssist': False, 'firstBloodKill': False, 'firstTowerAssist': False, 'firstTowerKill': False, 'gameEndedInEarlySurrender': False, 'gameEndedInSurrender': True, 'getBackPings': 0, 'goldEarned': 16737, 'goldSpent': 15400, 'holdPings': 0, 'individualPosition': 'JUNGLE', 'inhibitorKills': 0, 'inhibitorTakedowns': 0, 'inhibitorsLost': 0, 'item0': 3157, 'item1': 3089, 'item2': 3020, 'item3': 3165, 'item4': 1058, 'item5': 7011, 'item6': 3330, 'itemsPurchased': 22, 'killingSprees': 3, 'kills': 11, 'lane': 'JUNGLE', 'largestCriticalStrike': 0, 'largestKillingSpree': 6, 'largestMultiKill': 1, 'longestTimeSpentLiving': 803, 'magicDamageDealt': 206168, 'magicDamageDealtToChampions': 26496, 'magicDamageTaken': 18779, 'missions': {'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0}, 'needVisionPings': 0, 'neutralMinionsKilled': 151, 'nexusKills': 0, 'nexusLost': 0, 'nexusTakedowns': 1, 'objectivesStolen': 0, 'objectivesStolenAssists': 0, 'onMyWayPings': 1, 'participantId': 7, 'pentaKills': 0, 'perks': {'statPerks': {'defense': 5002, 'flex': 5008, 'offense': 5008}, 'styles': [{'description': 'primaryStyle', 'selections': [{'perk': 8369, 'var1': 1067, 'var2': 847, 'var3': 0}, {'perk': 8313, 'var1': 0, 'var2': 0, 'var3': 0}, {'perk': 8321, 'var1': 2, 'var2': 0, 'var3': 0}, {'perk': 8347, 'var1': 0, 'var2': 0, 'var3': 0}], 'style': 8300}, {'description': 'subStyle', 'selections': [{'perk': 8126, 'var1': 983, 'var2': 0, 'var3': 0}, {'perk': 8106, 'var1': 5, 'var2': 0, 'var3': 0}], 'style': 8100}]}, 'physicalDamageDealt': 10738, 'physicalDamageDealtToChampions': 172, 'physicalDamageTaken': 28320, 'placement': 0, 'playerAugment1': 0, 'playerAugment2': 0, 'playerAugment3': 0, 'playerAugment4': 0, 'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0, 'playerSubteamId': 0, 'profileIcon': 4778, 'pushPings': 0, 'puuid': 'th3FWRsIG8MtnJZg-BaRLYqcMc5fggO4dl7z6lBCtR8fGtSmncvKqX2-G-Hm81htOfAsgqs5V393Dw', 'quadraKills': 0, 'riotIdName': '', 'riotIdTagline': '', 'role': 'NONE', 'sightWardsBoughtInGame': 0, 'spell1Casts': 33, 'spell2Casts': 145, 'spell3Casts': 76, 'spell4Casts': 14, 'subteamPlacement': 0, 'summoner1Casts': 21, 'summoner1Id': 11, 'summoner2Casts': 4, 'summoner2Id': 4, 'summonerId': 'x6VaZjFDO5pcpjldR7PGm14iVLoR5aW288n2XIvxG0linlA', 'summonerLevel': 119, 'summonerName': 'TrizzyLin', 'teamEarlySurrendered': False, 'teamId': 200, 'teamPosition': 'JUNGLE', 'timeCCingOthers': 65, 'timePlayed': 2158, 'totalAllyJungleMinionsKilled': 107, 'totalDamageDealt': 266695, 'totalDamageDealtToChampions': 30285, 'totalDamageShieldedOnTeammates': 0, 'totalDamageTaken': 47441, 'totalEnemyJungleMinionsKilled': 8, 'totalHeal': 31748, 'totalHealsOnTeammates': 0, 'totalMinionsKilled': 47, 'totalTimeCCDealt': 710, 'totalTimeSpentDead': 127, 'totalUnitsHealed': 1, 'tripleKills': 0, 'trueDamageDealt': 49787, 'trueDamageDealtToChampions': 3616, 'trueDamageTaken': 341, 'turretKills': 3, 'turretTakedowns': 4, 'turretsLost': 4, 'unrealKills': 0, 'visionClearedPings': 0, 'visionScore': 37, 'visionWardsBoughtInGame': 0, 'wardsKilled': 2, 'wardsPlaced': 0, 'win': True}, {'allInPings': 0, 'assistMePings': 0, 'assists': 7, 'baitPings': 0, 'baronKills': 0, 'basicPings': 0, 'bountyLevel': 7, 'challenges': {'12AssistStreakCount': 0, 'abilityUses': 326, 'acesBefore15Minutes': 0, 'alliedJungleMonsterKills': 0, 'baronBuffGoldAdvantageOverThreshold': 1, 'baronTakedowns': 0, 'blastConeOppositeOpponentCount': 0, 'bountyGold': 550, 'buffsStolen': 0, 'completeSupportQuestInTime': 0, 'controlWardTimeCoverageInRiverOrEnemyHalf': 0.7497515497317442, 'controlWardsPlaced': 3, 'damagePerMinute': 987.0611481573541, 'damageTakenOnTeamPercentage': 0.1956819718317292, 'dancedWithRiftHerald': 0, 'deathsByEnemyChamps': 6, 'dodgeSkillShotsSmallWindow': 9, 'doubleAces': 0, 'dragonTakedowns': 0, 'earliestBaron': 2074.9830690999997, 'earlyLaningPhaseGoldExpAdvantage': 0, 'effectiveHealAndShielding': 0, 'elderDragonKillsWithOpposingSoul': 0, 'elderDragonMultikills': 0, 'enemyChampionImmobilizations': 0, 'enemyJungleMonsterKills': 1, 'epicMonsterKillsNearEnemyJungler': 0, 'epicMonsterKillsWithin30SecondsOfSpawn': 0, 'epicMonsterSteals': 0, 'epicMonsterStolenWithoutSmite': 0, 'firstTurretKilled': 1, 'firstTurretKilledTime': 947.9954345, 'flawlessAces': 1, 'fullTeamTakedown': 1, 'gameLength': 2158.4759347, 'getTakedownsInAllLanesEarlyJungleAsLaner': 0, 'goldPerMinute': 475.0449831654081, 'hadOpenNexus': 0, 'immobilizeAndKillWithAlly': 0, 'initialBuffCount': 0, 'initialCrabCount': 0, 'jungleCsBefore10Minutes': 0, 'junglerTakedownsNearDamagedEpicMonster': 0, 'kTurretsDestroyedBeforePlatesFall': 0, 'kda': 4, 'killAfterHiddenWithAlly': 2, 'killParticipation': 0.4444444444444444, 'killedChampTookFullTeamDamageSurvived': 0, 'killingSprees': 3, 'killsNearEnemyTurret': 3, 'killsOnOtherLanesEarlyJungleAsLaner': 0, 'killsOnRecentlyHealedByAramPack': 0, 'killsUnderOwnTurret': 1, 'killsWithHelpFromEpicMonster': 0, 'knockEnemyIntoTeamAndKill': 0, 'landSkillShotsEarlyGame': 9, 'laneMinionsFirst10Minutes': 60, 'laningPhaseGoldExpAdvantage': 0, 'legendaryCount': 0, 'lostAnInhibitor': 0, 'maxCsAdvantageOnLaneOpponent': 14, 'maxKillDeficit': 0, 'maxLevelLeadLaneOpponent': 2, 'mejaisFullStackInTime': 0, 'moreEnemyJungleThanOpponent': 0, 'multiKillOneSpell': 1, 'multiTurretRiftHeraldCount': 0, 'multikills': 2, 'multikillsAfterAggressiveFlash': 0, 'mythicItemUsed': 3152, 'outerTurretExecutesBefore10Minutes': 0, 'outnumberedKills': 3, 'outnumberedNexusKill': 0, 'perfectDragonSoulsTaken': 0, 'perfectGame': 0, 'pickKillWithAlly': 16, 'playedChampSelectPosition': 1, 'poroExplosions': 0, 'quickCleanse': 0, 'quickFirstTurret': 0, 'quickSoloKills': 0, 'riftHeraldTakedowns': 0, 'saveAllyFromDeath': 0, 'scuttleCrabKills': 0, 'shortestTimeToAceFromFirstTakedown': 42.70709119999992, 'skillshotsDodged': 118, 'skillshotsHit': 43, 'snowballsHit': 0, 'soloBaronKills': 0, 'soloKills': 7, 'stealthWardsPlaced': 12, 'survivedSingleDigitHpCount': 1, 'survivedThreeImmobilizesInFight': 0, 'takedownOnFirstTurret': 0, 'takedowns': 24, 'takedownsAfterGainingLevelAdvantage': 0, 'takedownsBeforeJungleMinionSpawn': 1, 'takedownsFirstXMinutes': 5, 'takedownsInAlcove': 0, 'takedownsInEnemyFountain': 0, 'teamBaronKills': 1, 'teamDamagePercentage': 0.23316624560216415, 'teamElderDragonKills': 0, 'teamRiftHeraldKills': 1, 'tookLargeDamageSurvived': 0, 'turretPlatesTaken': 1, 'turretTakedowns': 3, 'turretsTakenWithRiftHerald': 0, 'twentyMinionsIn3SecondsCount': 0, 'twoWardsOneSweeperCount': 0, 'unseenRecalls': 0, 'visionScoreAdvantageLaneOpponent': -0.02784043550491333, 'visionScorePerMinute': 1.2208970005495394, 'wardTakedowns': 4, 'wardTakedownsBefore20M': 3, 'wardsGuarded': 1}, 'champExperience': 21409, 'champLevel': 18, 'championId': 84, 'championName': 'Akali', 'championTransform': 0, 'commandPings': 14, 'consumablesPurchased': 12, 'damageDealtToBuildings': 4590, 'damageDealtToObjectives': 4590, 'damageDealtToTurrets': 4590, 'damageSelfMitigated': 30543, 'dangerPings': 0, 'deaths': 6, 'detectorWardsPlaced': 3, 'doubleKills': 2, 'dragonKills': 0, 'eligibleForProgression': True, 'enemyMissingPings': 7, 'enemyVisionPings': 0, 'firstBloodAssist': True, 'firstBloodKill': False, 'firstTowerAssist': False, 'firstTowerKill': False, 'gameEndedInEarlySurrender': False, 'gameEndedInSurrender': True, 'getBackPings': 7, 'goldEarned': 17089, 'goldSpent': 16975, 'holdPings': 0, 'individualPosition': 'MIDDLE', 'inhibitorKills': 2, 'inhibitorTakedowns': 2, 'inhibitorsLost': 0, 'item0': 3157, 'item1': 3102, 'item2': 3152, 'item3': 3020, 'item4': 4645, 'item5': 3100, 'item6': 3340, 'itemsPurchased': 32, 'killingSprees': 4, 'kills': 17, 'lane': 'MIDDLE', 'largestCriticalStrike': 0, 'largestKillingSpree': 7, 'largestMultiKill': 2, 'longestTimeSpentLiving': 955, 'magicDamageDealt': 160128, 'magicDamageDealtToChampions': 32200, 'magicDamageTaken': 17706, 'missions': {'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0}, 'needVisionPings': 0, 'neutralMinionsKilled': 2, 'nexusKills': 0, 'nexusLost': 0, 'nexusTakedowns': 0, 'objectivesStolen': 0, 'objectivesStolenAssists': 0, 'onMyWayPings': 4, 'participantId': 8, 'pentaKills': 0, 'perks': {'statPerks': {'defense': 5002, 'flex': 5008, 'offense': 5008}, 'styles': [{'description': 'primaryStyle', 'selections': [{'perk': 8112, 'var1': 2415, 'var2': 0, 'var3': 0}, {'perk': 8139, 'var1': 931, 'var2': 0, 'var3': 0}, {'perk': 8138, 'var1': 30, 'var2': 0, 'var3': 0}, {'perk': 8105, 'var1': 13, 'var2': 5, 'var3': 0}], 'style': 8100}, {'description': 'subStyle', 'selections': [{'perk': 8473, 'var1': 1538, 'var2': 0, 'var3': 0}, {'perk': 8242, 'var1': 113, 'var2': 0, 'var3': 0}], 'style': 8400}]}, 'physicalDamageDealt': 24293, 'physicalDamageDealtToChampions': 1765, 'physicalDamageTaken': 21861, 'placement': 0, 'playerAugment1': 0, 'playerAugment2': 0, 'playerAugment3': 0, 'playerAugment4': 0, 'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0, 'playerSubteamId': 0, 'profileIcon': 4793, 'pushPings': 0, 'puuid': 'KyRnbjcRmlrkQ7fShSbsBPJUhUkVTZ2GNZrM8wJSeqm0ld97UrrEKnE1UIT-pDsOn0RDqM83ukk_hg', 'quadraKills': 0, 'riotIdName': '', 'riotIdTagline': '', 'role': 'SOLO', 'sightWardsBoughtInGame': 0, 'spell1Casts': 187, 'spell2Casts': 32, 'spell3Casts': 79, 'spell4Casts': 28, 'subteamPlacement': 0, 'summoner1Casts': 4, 'summoner1Id': 4, 'summoner2Casts': 7, 'summoner2Id': 14, 'summonerId': '_k-0mrfL0YiQzqjJDcbT-0iIE0BXGYSob-QY-V7_HP3-yketKvMhcjp6iQ', 'summonerLevel': 173, 'summonerName': 'Isabellauwu', 'teamEarlySurrendered': False, 'teamId': 200, 'teamPosition': 'MIDDLE', 'timeCCingOthers': 1, 'timePlayed': 2158, 'totalAllyJungleMinionsKilled': 0, 'totalDamageDealt': 186786, 'totalDamageDealtToChampions': 35509, 'totalDamageShieldedOnTeammates': 0, 'totalDamageTaken': 39977, 'totalEnemyJungleMinionsKilled': 2, 'totalHeal': 8509, 'totalHealsOnTeammates': 0, 'totalMinionsKilled': 196, 'totalTimeCCDealt': 290, 'totalTimeSpentDead': 262, 'totalUnitsHealed': 1, 'tripleKills': 0, 'trueDamageDealt': 2364, 'trueDamageDealtToChampions': 1543, 'trueDamageTaken': 410, 'turretKills': 3, 'turretTakedowns': 3, 'turretsLost': 4, 'unrealKills': 0, 'visionClearedPings': 0, 'visionScore': 43, 'visionWardsBoughtInGame': 3, 'wardsKilled': 4, 'wardsPlaced': 16, 'win': True}, {'allInPings': 0, 'assistMePings': 0, 'assists': 11, 'baitPings': 0, 'baronKills': 0, 'basicPings': 0, 'bountyLevel': 4, 'challenges': {'12AssistStreakCount': 0, 'abilityUses': 241, 'acesBefore15Minutes': 0, 'alliedJungleMonsterKills': 0, 'baronBuffGoldAdvantageOverThreshold': 1, 'baronTakedowns': 1, 'blastConeOppositeOpponentCount': 0, 'bountyGold': 0, 'buffsStolen': 0, 'completeSupportQuestInTime': 0, 'controlWardTimeCoverageInRiverOrEnemyHalf': 0.4617468263497337, 'controlWardsPlaced': 6, 'damagePerMinute': 789.782465445664, 'damageTakenOnTeamPercentage': 0.16153639182715482, 'dancedWithRiftHerald': 0, 'deathsByEnemyChamps': 7, 'dodgeSkillShotsSmallWindow': 1, 'doubleAces': 0, 'dragonTakedowns': 2, 'earliestBaron': 2074.9830690999997, 'earliestDragonTakedown': 1138.4310306999998, 'earlyLaningPhaseGoldExpAdvantage': 0, 'effectiveHealAndShielding': 0, 'elderDragonKillsWithOpposingSoul': 0, 'elderDragonMultikills': 0, 'enemyChampionImmobilizations': 13, 'enemyJungleMonsterKills': 0, 'epicMonsterKillsNearEnemyJungler': 0, 'epicMonsterKillsWithin30SecondsOfSpawn': 0, 'epicMonsterSteals': 0, 'epicMonsterStolenWithoutSmite': 0, 'firstTurretKilled': 1, 'firstTurretKilledTime': 947.9954345, 'flawlessAces': 1, 'fullTeamTakedown': 1, 'gameLength': 2158.4759347, 'getTakedownsInAllLanesEarlyJungleAsLaner': 0, 'goldPerMinute': 437.03229498241296, 'hadOpenNexus': 0, 'immobilizeAndKillWithAlly': 5, 'initialBuffCount': 0, 'initialCrabCount': 0, 'jungleCsBefore10Minutes': 0, 'junglerTakedownsNearDamagedEpicMonster': 0, 'kTurretsDestroyedBeforePlatesFall': 0, 'kda': 3.142857142857143, 'killAfterHiddenWithAlly': 5, 'killParticipation': 0.4074074074074074, 'killedChampTookFullTeamDamageSurvived': 0, 'killingSprees': 1, 'killsNearEnemyTurret': 3, 'killsOnOtherLanesEarlyJungleAsLaner': 0, 'killsOnRecentlyHealedByAramPack': 0, 'killsUnderOwnTurret': 2, 'killsWithHelpFromEpicMonster': 0, 'knockEnemyIntoTeamAndKill': 0, 'landSkillShotsEarlyGame': 0, 'laneMinionsFirst10Minutes': 66, 'laningPhaseGoldExpAdvantage': 0, 'legendaryCount': 0, 'lostAnInhibitor': 0, 'maxCsAdvantageOnLaneOpponent': 28, 'maxKillDeficit': 0, 'maxLevelLeadLaneOpponent': 1, 'mejaisFullStackInTime': 0, 'moreEnemyJungleThanOpponent': 0, 'multiKillOneSpell': 0, 'multiTurretRiftHeraldCount': 0, 'multikills': 0, 'multikillsAfterAggressiveFlash': 0, 'mythicItemUsed': 6671, 'outerTurretExecutesBefore10Minutes': 0, 'outnumberedKills': 0, 'outnumberedNexusKill': 0, 'perfectDragonSoulsTaken': 0, 'perfectGame': 0, 'pickKillWithAlly': 21, 'playedChampSelectPosition': 1, 'poroExplosions': 0, 'quickCleanse': 0, 'quickFirstTurret': 0, 'quickSoloKills': 0, 'riftHeraldTakedowns': 0, 'saveAllyFromDeath': 0, 'scuttleCrabKills': 0, 'shortestTimeToAceFromFirstTakedown': 56.705361999999695, 'skillshotsDodged': 49, 'skillshotsHit': 6, 'snowballsHit': 0, 'soloBaronKills': 0, 'soloKills': 0, 'stealthWardsPlaced': 5, 'survivedSingleDigitHpCount': 0, 'survivedThreeImmobilizesInFight': 0, 'takedownOnFirstTurret': 0, 'takedowns': 22, 'takedownsAfterGainingLevelAdvantage': 0, 'takedownsBeforeJungleMinionSpawn': 0, 'takedownsFirstXMinutes': 4, 'takedownsInAlcove': 0, 'takedownsInEnemyFountain': 0, 'teamBaronKills': 1, 'teamDamagePercentage': 0.18656454329517355, 'teamElderDragonKills': 0, 'teamRiftHeraldKills': 1, 'tookLargeDamageSurvived': 0, 'turretPlatesTaken': 0, 'turretTakedowns': 1, 'turretsTakenWithRiftHerald': 0, 'twentyMinionsIn3SecondsCount': 0, 'twoWardsOneSweeperCount': 0, 'unseenRecalls': 0, 'visionScoreAdvantageLaneOpponent': 1.4060091972351074, 'visionScorePerMinute': 0.7616802674476811, 'wardTakedowns': 4, 'wardTakedownsBefore20M': 2, 'wardsGuarded': 0}, 'champExperience': 20631, 'champLevel': 18, 'championId': 202, 'championName': 'Jhin', 'championTransform': 0, 'commandPings': 1, 'consumablesPurchased': 11, 'damageDealtToBuildings': 3838, 'damageDealtToObjectives': 19964, 'damageDealtToTurrets': 3838, 'damageSelfMitigated': 19635, 'dangerPings': 0, 'deaths': 7, 'detectorWardsPlaced': 6, 'doubleKills': 0, 'dragonKills': 0, 'eligibleForProgression': True, 'enemyMissingPings': 3, 'enemyVisionPings': 4, 'firstBloodAssist': False, 'firstBloodKill': False, 'firstTowerAssist': False, 'firstTowerKill': False, 'gameEndedInEarlySurrender': False, 'gameEndedInSurrender': True, 'getBackPings': 2, 'goldEarned': 15722, 'goldSpent': 13000, 'holdPings': 0, 'individualPosition': 'BOTTOM', 'inhibitorKills': 0, 'inhibitorTakedowns': 0, 'inhibitorsLost': 0, 'item0': 3009, 'item1': 7006, 'item2': 2421, 'item3': 3036, 'item4': 1031, 'item5': 3095, 'item6': 3363, 'itemsPurchased': 34, 'killingSprees': 3, 'kills': 11, 'lane': 'BOTTOM', 'largestCriticalStrike': 1756, 'largestKillingSpree': 4, 'largestMultiKill': 1, 'longestTimeSpentLiving': 757, 'magicDamageDealt': 23158, 'magicDamageDealtToChampions': 1942, 'magicDamageTaken': 11689, 'missions': {'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0}, 'needVisionPings': 0, 'neutralMinionsKilled': 0, 'nexusKills': 0, 'nexusLost': 0, 'nexusTakedowns': 1, 'objectivesStolen': 0, 'objectivesStolenAssists': 0, 'onMyWayPings': 1, 'participantId': 9, 'pentaKills': 0, 'perks': {'statPerks': {'defense': 5002, 'flex': 5008, 'offense': 5008}, 'styles': [{'description': 'primaryStyle', 'selections': [{'perk': 8021, 'var1': 2429, 'var2': 2701, 'var3': 0}, {'perk': 8009, 'var1': 2242, 'var2': 0, 'var3': 0}, {'perk': 9103, 'var1': 22, 'var2': 0, 'var3': 0}, {'perk': 8014, 'var1': 903, 'var2': 0, 'var3': 0}], 'style': 8000}, {'description': 'subStyle', 'selections': [{'perk': 8234, 'var1': 13607, 'var2': 0, 'var3': 0}, {'perk': 8236, 'var1': 28, 'var2': 0, 'var3': 0}], 'style': 8200}]}, 'physicalDamageDealt': 220826, 'physicalDamageDealtToChampions': 25016, 'physicalDamageTaken': 21014, 'placement': 0, 'playerAugment1': 0, 'playerAugment2': 0, 'playerAugment3': 0, 'playerAugment4': 0, 'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0, 'playerSubteamId': 0, 'profileIcon': 6295, 'pushPings': 0, 'puuid': 'yFM3imQ-mYn4XTiMFpOx8CQouAiGQTec1u24Ou7I-6kX8_oMBJTfPUG1MPbm_gI16oAeq-pWNnLgZQ', 'quadraKills': 0, 'riotIdName': '', 'riotIdTagline': '', 'role': 'CARRY', 'sightWardsBoughtInGame': 0, 'spell1Casts': 113, 'spell2Casts': 43, 'spell3Casts': 57, 'spell4Casts': 28, 'subteamPlacement': 0, 'summoner1Casts': 4, 'summoner1Id': 4, 'summoner2Casts': 5, 'summoner2Id': 3, 'summonerId': 'oCpSdISbpqbtJSW4kZolGTIBRcgSE2Pakk05h7cAluGhoTY', 'summonerLevel': 111, 'summonerName': 'DDDemacia', 'teamEarlySurrendered': False, 'teamId': 200, 'teamPosition': 'BOTTOM', 'timeCCingOthers': 24, 'timePlayed': 2158, 'totalAllyJungleMinionsKilled': 0, 'totalDamageDealt': 252424, 'totalDamageDealtToChampions': 28412, 'totalDamageShieldedOnTeammates': 0, 'totalDamageTaken': 33001, 'totalEnemyJungleMinionsKilled': 0, 'totalHeal': 10560, 'totalHealsOnTeammates': 0, 'totalMinionsKilled': 245, 'totalTimeCCDealt': 246, 'totalTimeSpentDead': 251, 'totalUnitsHealed': 1, 'tripleKills': 0, 'trueDamageDealt': 8440, 'trueDamageDealtToChampions': 1453, 'trueDamageTaken': 298, 'turretKills': 1, 'turretTakedowns': 1, 'turretsLost': 4, 'unrealKills': 0, 'visionClearedPings': 0, 'visionScore': 27, 'visionWardsBoughtInGame': 6, 'wardsKilled': 4, 'wardsPlaced': 12, 'win': True}, {'allInPings': 0, 'assistMePings': 0, 'assists': 24, 'baitPings': 0, 'baronKills': 0, 'basicPings': 0, 'bountyLevel': 1, 'challenges': {'12AssistStreakCount': 0, 'abilityUses': 172, 'acesBefore15Minutes': 0, 'alliedJungleMonsterKills': 0, 'baronBuffGoldAdvantageOverThreshold': 1, 'baronTakedowns': 1, 'blastConeOppositeOpponentCount': 0, 'bountyGold': 300, 'buffsStolen': 0, 'completeSupportQuestInTime': 1, 'controlWardTimeCoverageInRiverOrEnemyHalf': 0.9519530319829976, 'controlWardsPlaced': 4, 'damagePerMinute': 915.2157502045834, 'damageTakenOnTeamPercentage': 0.14178924859869352, 'dancedWithRiftHerald': 0, 'deathsByEnemyChamps': 9, 'dodgeSkillShotsSmallWindow': 4, 'doubleAces': 0, 'dragonTakedowns': 2, 'earliestBaron': 2074.9830690999997, 'earliestDragonTakedown': 790.9937994999999, 'earlyLaningPhaseGoldExpAdvantage': 0, 'effectiveHealAndShielding': 9815.6416015625, 'elderDragonKillsWithOpposingSoul': 0, 'elderDragonMultikills': 0, 'enemyChampionImmobilizations': 13, 'enemyJungleMonsterKills': 1, 'epicMonsterKillsNearEnemyJungler': 0, 'epicMonsterKillsWithin30SecondsOfSpawn': 0, 'epicMonsterSteals': 0, 'epicMonsterStolenWithoutSmite': 0, 'fasterSupportQuestCompletion': 1, 'firstTurretKilled': 1, 'firstTurretKilledTime': 947.9954345, 'flawlessAces': 1, 'fullTeamTakedown': 1, 'gameLength': 2158.4759347, 'getTakedownsInAllLanesEarlyJungleAsLaner': 0, 'goldPerMinute': 379.11716331052594, 'hadOpenNexus': 0, 'highestWardKills': 1, 'immobilizeAndKillWithAlly': 4, 'initialBuffCount': 0, 'initialCrabCount': 0, 'jungleCsBefore10Minutes': 0, 'junglerTakedownsNearDamagedEpicMonster': 0, 'kTurretsDestroyedBeforePlatesFall': 0, 'kda': 3.4444444444444446, 'killAfterHiddenWithAlly': 7, 'killParticipation': 0.5740740740740741, 'killedChampTookFullTeamDamageSurvived': 0, 'killingSprees': 0, 'killsNearEnemyTurret': 2, 'killsOnOtherLanesEarlyJungleAsLaner': 2, 'killsOnRecentlyHealedByAramPack': 0, 'killsUnderOwnTurret': 2, 'killsWithHelpFromEpicMonster': 0, 'knockEnemyIntoTeamAndKill': 0, 'landSkillShotsEarlyGame': 4, 'laneMinionsFirst10Minutes': 7, 'laningPhaseGoldExpAdvantage': 0, 'legendaryCount': 0, 'lostAnInhibitor': 0, 'maxCsAdvantageOnLaneOpponent': 11.5, 'maxKillDeficit': 0, 'maxLevelLeadLaneOpponent': 3, 'mejaisFullStackInTime': 0, 'moreEnemyJungleThanOpponent': 0, 'multiKillOneSpell': 0, 'multiTurretRiftHeraldCount': 0, 'multikills': 0, 'multikillsAfterAggressiveFlash': 0, 'mythicItemUsed': 6692, 'outerTurretExecutesBefore10Minutes': 0, 'outnumberedKills': 2, 'outnumberedNexusKill': 0, 'perfectDragonSoulsTaken': 0, 'perfectGame': 0, 'pickKillWithAlly': 26, 'playedChampSelectPosition': 1, 'poroExplosions': 0, 'quickCleanse': 0, 'quickFirstTurret': 0, 'quickSoloKills': 0, 'riftHeraldTakedowns': 0, 'saveAllyFromDeath': 3, 'scuttleCrabKills': 0, 'shortestTimeToAceFromFirstTakedown': 42.70709119999992, 'skillshotsDodged': 57, 'skillshotsHit': 23, 'snowballsHit': 0, 'soloBaronKills': 0, 'soloKills': 1, 'stealthWardsPlaced': 31, 'survivedSingleDigitHpCount': 0, 'survivedThreeImmobilizesInFight': 0, 'takedownOnFirstTurret': 0, 'takedowns': 31, 'takedownsAfterGainingLevelAdvantage': 0, 'takedownsBeforeJungleMinionSpawn': 1, 'takedownsFirstXMinutes': 7, 'takedownsInAlcove': 0, 'takedownsInEnemyFountain': 0, 'teamBaronKills': 1, 'teamDamagePercentage': 0.21619473199764894, 'teamElderDragonKills': 0, 'teamRiftHeraldKills': 1, 'tookLargeDamageSurvived': 0, 'turretPlatesTaken': 0, 'turretTakedowns': 3, 'turretsTakenWithRiftHerald': 0, 'twentyMinionsIn3SecondsCount': 0, 'twoWardsOneSweeperCount': 0, 'unseenRecalls': 0, 'visionScoreAdvantageLaneOpponent': 0.018885374069213867, 'visionScorePerMinute': 2.7658383058104525, 'wardTakedowns': 14, 'wardTakedownsBefore20M': 9, 'wardsGuarded': 0}, 'champExperience': 17124, 'champLevel': 17, 'championId': 235, 'championName': 'Senna', 'championTransform': 0, 'commandPings': 11, 'consumablesPurchased': 7, 'damageDealtToBuildings': 5734, 'damageDealtToObjectives': 14049, 'damageDealtToTurrets': 5734, 'damageSelfMitigated': 17385, 'dangerPings': 0, 'deaths': 9, 'detectorWardsPlaced': 4, 'doubleKills': 0, 'dragonKills': 0, 'eligibleForProgression': True, 'enemyMissingPings': 14, 'enemyVisionPings': 1, 'firstBloodAssist': False, 'firstBloodKill': True, 'firstTowerAssist': False, 'firstTowerKill': False, 'gameEndedInEarlySurrender': False, 'gameEndedInSurrender': True, 'getBackPings': 0, 'goldEarned': 13638, 'goldSpent': 11225, 'holdPings': 0, 'individualPosition': 'UTILITY', 'inhibitorKills': 0, 'inhibitorTakedowns': 0, 'inhibitorsLost': 0, 'item0': 3864, 'item1': 7001, 'item2': 3009, 'item3': 3033, 'item4': 3094, 'item5': 1036, 'item6': 3364, 'itemsPurchased': 27, 'killingSprees': 1, 'kills': 7, 'lane': 'BOTTOM', 'largestCriticalStrike': 625, 'largestKillingSpree': 2, 'largestMultiKill': 1, 'longestTimeSpentLiving': 754, 'magicDamageDealt': 1542, 'magicDamageDealtToChampions': 833, 'magicDamageTaken': 11637, 'missions': {'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0}, 'needVisionPings': 1, 'neutralMinionsKilled': 0, 'nexusKills': 1, 'nexusLost': 0, 'nexusTakedowns': 1, 'objectivesStolen': 0, 'objectivesStolenAssists': 0, 'onMyWayPings': 18, 'participantId': 10, 'pentaKills': 0, 'perks': {'statPerks': {'defense': 5002, 'flex': 5008, 'offense': 5005}, 'styles': [{'description': 'primaryStyle', 'selections': [{'perk': 8021, 'var1': 2316, 'var2': 2239, 'var3': 0}, {'perk': 8009, 'var1': 3391, 'var2': 0, 'var3': 0}, {'perk': 9104, 'var1': 15, 'var2': 40, 'var3': 0}, {'perk': 8017, 'var1': 2802, 'var2': 0, 'var3': 0}], 'style': 8000}, {'description': 'subStyle', 'selections': [{'perk': 8410, 'var1': 89, 'var2': 0, 'var3': 0}, {'perk': 8345, 'var1': 3, 'var2': 0, 'var3': 0}], 'style': 8300}]}, 'physicalDamageDealt': 79455, 'physicalDamageDealtToChampions': 30717, 'physicalDamageTaken': 16879, 'placement': 0, 'playerAugment1': 0, 'playerAugment2': 0, 'playerAugment3': 0, 'playerAugment4': 0, 'playerScore0': 0, 'playerScore1': 0, 'playerScore10': 0, 'playerScore11': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0, 'playerSubteamId': 0, 'profileIcon': 5735, 'pushPings': 1, 'puuid': '1JzHac7vWznlDGVAlQMtHfJMJr39gI0_akJzwWBC9o2AJIp4GGbAbhhqMJYV-zgfCTE7pty72aAEwg', 'quadraKills': 0, 'riotIdName': '', 'riotIdTagline': '', 'role': 'SUPPORT', 'sightWardsBoughtInGame': 0, 'spell1Casts': 86, 'spell2Casts': 45, 'spell3Casts': 30, 'spell4Casts': 11, 'subteamPlacement': 0, 'summoner1Casts': 5, 'summoner1Id': 4, 'summoner2Casts': 6, 'summoner2Id': 7, 'summonerId': '3VKGbZg5IiCKeauhA81v53EIkgj_84pgHWpEzz63fGE4mUjkkzj5Z5YgDw', 'summonerLevel': 32, 'summonerName': 'onepunchvi69x', 'teamEarlySurrendered': False, 'teamId': 200, 'teamPosition': 'UTILITY', 'timeCCingOthers': 27, 'timePlayed': 2158, 'totalAllyJungleMinionsKilled': 0, 'totalDamageDealt': 83018, 'totalDamageDealtToChampions': 32924, 'totalDamageShieldedOnTeammates': 3396, 'totalDamageTaken': 28967, 'totalEnemyJungleMinionsKilled': 0, 'totalHeal': 19265, 'totalHealsOnTeammates': 6419, 'totalMinionsKilled': 44, 'totalTimeCCDealt': 263, 'totalTimeSpentDead': 315, 'totalUnitsHealed': 5, 'tripleKills': 0, 'trueDamageDealt': 2020, 'trueDamageDealtToChampions': 1373, 'trueDamageTaken': 450, 'turretKills': 2, 'turretTakedowns': 3, 'turretsLost': 4, 'unrealKills': 0, 'visionClearedPings': 0, 'visionScore': 99, 'visionWardsBoughtInGame': 5, 'wardsKilled': 14, 'wardsPlaced': 35, 'win': True}], 'platformId': 'NA1', 'queueId': 420, 'teams': [{'bans': [{'championId': 555, 'pickTurn': 1}, {'championId': 157, 'pickTurn': 2}, {'championId': -1, 'pickTurn': 3}, {'championId': 518, 'pickTurn': 4}, {'championId': 36, 'pickTurn': 5}], 'objectives': {'baron': {'first': True, 'kills': 1}, 'champion': {'first': False, 'kills': 30}, 'dragon': {'first': True, 'kills': 1}, 'horde': {'first': False, 'kills': 0}, 'inhibitor': {'first': False, 'kills': 0}, 'riftHerald': {'first': False, 'kills': 0}, 'tower': {'first': False, 'kills': 4}}, 'teamId': 100, 'win': False}, {'bans': [{'championId': 40, 'pickTurn': 6}, {'championId': 53, 'pickTurn': 7}, {'championId': 517, 'pickTurn': 8}, {'championId': 420, 'pickTurn': 9}, {'championId': 777, 'pickTurn': 10}], 'objectives': {'baron': {'first': False, 'kills': 1}, 'champion': {'first': True, 'kills': 54}, 'dragon': {'first': False, 'kills': 4}, 'horde': {'first': False, 'kills': 0}, 'inhibitor': {'first': True, 'kills': 2}, 'riftHerald': {'first': True, 'kills': 1}, 'tower': {'first': True, 'kills': 10}}, 'teamId': 200, 'win': True}], 'tournamentCode': ''}}